In [1]:
# All imports

from __future__ import print_function, division

import matplotlib.pyplot as plt
import numpy as np
import glob


import SimpleITK as sitk
import pandas as pd
import glob
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans

import csv
from scipy import ndimage, misc
from tqdm import tqdm

import numba
from numba import njit, prange

import os
import skimage.io as io
import skimage.transform as trans
import numpy as np

from skimage.measure import label
from scipy.io import loadmat
from scipy.ndimage import zoom
#from scipy.misc import imresize
import pywt

import csv
import random
import time
%matplotlib inline  

from scipy import ndimage, misc

import pywt
#import hdf5storage

import scipy.io as sio
from skimage.filters import threshold_otsu

import pywt
import numpy as np
#import pydicom
import matplotlib.pyplot as plt
import SimpleITK as sitk
import skimage.io as io
#from sklearn.decomposition import PCA
import collections, numpy
import warnings
from scipy import ndimage, misc
warnings.filterwarnings('ignore')
import copy


import os
import glob
import uuid
import numpy as np


import numpy
import warnings

import functools
import pickle
import time


import numpy as np

import argparse
import torch
import torch.utils.data as data_utils
import torch.optim as optim
from torch.autograd import Variable

import torch.nn as nn

np.random.seed(0)
#torch.manual_seed(0)

In [155]:
# [STAR] Pytorch Models for training

import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable

import matplotlib.pyplot as plt
import numpy as np
from torchsummary import summary

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            #nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
            #nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

class Down(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)

class Up(nn.Module):
    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels , in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)


    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)
    
class SUNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(SUNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes  = n_classes
        self.bilinear   = bilinear

        self.inc = DoubleConv(n_channels, 16)
        self.down1 = Down(16, 32)
        self.down2 = Down(32, 64)
        self.down3 = Down(64, 128)
        factor = 2 if bilinear else 1
        self.down4 = Down(128, 256 // factor)
        self.up1 = Up(256, 128 // factor, bilinear)
        self.up2 = Up(128, 64 // factor, bilinear)
        self.up3 = Up(64, 32 // factor, bilinear)
        self.up4 = Up(32, 16, bilinear)
        self.outc = OutConv(16, n_classes)
        #self.out_sigmoid = nn.Sigmoid()
        self.out_softmax = nn.LogSoftmax(dim=1)
        
        self.gn1 = nn.GroupNorm(8, 16)
        self.gn2 = nn.GroupNorm(16, 32)
        self.gn3 = nn.GroupNorm(32, 64)
        self.gn4 = nn.GroupNorm(64, 128)
        self.gn5 = nn.GroupNorm(32, 64)
        self.gn6 = nn.GroupNorm(16, 32)
        self.gn7 = nn.GroupNorm(8, 16)
        
        self.dp1 = nn.Dropout(p=0.4)
        self.dp2 = nn.Dropout(p=0.4)
    
    def forward(self, x):
        x1 = self.inc(x)
        x1 = self.gn1(x1)
        
        x2 = self.down1(x1)
        x2 = self.gn2(x2)
        
        x3 = self.down2(x2)
        x3 = self.gn3(x3)
       
        x4 = self.down3(x3)
        x4 = self.gn4(x4)
       
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.gn5(x)
       
        x = self.up2(x, x3)
        x = self.gn6(x)
            
        x = self.up3(x, x2)
        x = self.gn7(x)
        
        x  = self.up4(x, x1)
        
        logits = self.outc(x)
        #out    = self.out_softmax(logits)
        return logits

class AttnDecoderRNN_old(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=256, bilinear=True):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        self.bilinear = bilinear
        self.n_classes = 1

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size*2, self.max_length)
        
        self.attn_24 = nn.Linear(self.hidden_size*4, self.hidden_size*2)
        
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        
        self.attn_combine_bilstm = nn.Linear(self.hidden_size * 3, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)
       # self.hidden = nn.Parameter(torch.randn(4,256,256).cuda()),nn.Parameter(torch.randn(4,256,256).cuda())
       
        self.lsgn_a = nn.GroupNorm(128,256)
    
        self.down5 = Down(128,256)
        
        factor = 2 if bilinear else 1
                
        self.ups4 = nn.ConvTranspose2d(256 , 256 // 2, kernel_size=2, stride=2)
        self.upsconv4 = DoubleConv(256,128)

        self.lstm = nn.LSTM(256,256,batch_first=False,bidirectional=True,num_layers=1).cuda()
    
    def forward(self, input,hidden,encoder_outputs):
        
        h = torch.unsqueeze(hidden,0)
        
        embedded = input
        
        embedded = self.dropout(embedded)

        hidden_bilstm = h[0]
        
        
        hidden_bilinn =  hidden_bilstm
        
        hidden_bilinn = self.attn(hidden_bilinn)
    
        hidden_bilinn = self.lsgn_a(hidden_bilinn)

        attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden_bilinn), 1)), dim=1)
        
        
        
        attn_weights  = self.lsgn_a(attn_weights)
    
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),encoder_outputs.unsqueeze(0))
        
   #     print('attn_applied: encoder outputs',attn_applied[0].shape,encoder_outputs[0].shape)

        output = torch.cat((embedded[0], attn_applied[0]), 1)
  #      print('The output shape is : ',output.shape)
        
        output = self.attn_combine_bilstm(output).unsqueeze(0)
 #      print('The output shape after is : ',output.shape)
        
    
        hidden_bi = hidden_bilinn.unsqueeze(0)
        
        output = F.relu(output)
        
        #print("output and hidden before lstm ",output.shape,hidden_bi.shape)

        output, hidden = self.gru(output, hidden_bi)
        
        output = F.log_softmax(self.out(output[0]), dim=1)
        output = self.lsgn_a(output)
        
       #output = self.lsgn_a(output)
    
        return output,hidden


    def initHidden(self):
        return torch.randn(4, 256, self.hidden_size, device=device)

############### MAIN MODEL ##############
class UNetDoubleSmallGroupNormdifferent_old(nn.Module):
    def __init__(self, n_channels, n_classes,bilinear=True):
        
        super(UNetDoubleSmallGroupNormdifferent, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 16)
        self.down1 = Down(16, 32)
        self.down2 = Down(32, 64)
        self.down3 = Down(64, 128)
        factor = 2 if bilinear else 1
        self.down4 = Down(128, 256 // factor)

        
        self.down5 = Down(128,256)
        
        
        self.up1 = Up(256, 128 // factor, bilinear)
        self.up2 = Up(128, 64 // factor, bilinear)
        self.up3 = Up(64, 32 // factor, bilinear)
        self.up4 = Up(32, 16, bilinear)
        self.outc = OutConv(16, n_classes)
        #self.out_sigmoid = nn.Sigmoid()
        self.out_softmax = nn.LogSoftmax(dim=1)
        
        self.lsgn1 = nn.GroupNorm(128,256)
        
        self.lsgn2 = nn.GroupNorm(64,256)
        
        
        self.gn1 = nn.GroupNorm(8, 16)
        self.gn2 = nn.GroupNorm(16, 32)
        self.gn3 = nn.GroupNorm(32, 64)
        self.gn4 = nn.GroupNorm(64, 128)
        self.gn5 = nn.GroupNorm(32, 64)
        self.gn6 = nn.GroupNorm(16, 32)
        self.gn7 = nn.GroupNorm(8, 16)
   
    def forward(self, x):
        x1 = self.inc(x)
       # x1 = self.gn1(x1)
       
        x2 = self.down1(x1)
       # x2 = self.gn2(x2)
       
        x3 = self.down2(x2)
       # x3 = self.gn3(x3)
       
        x4 = self.down3(x3)
       # x4 = self.gn4(x4)
       
        x5 = self.down4(x4)
        
        #x5 = torch.squeeze(x5)
        x5 = self.down5(x5)
        #x5 = self.down6(x5)
        
        #print('x5 shape is :',x5.shape)
        
        xlst = x5.reshape([4,256,256])

        lstm = nn.LSTM(256,256,batch_first= True,bidirectional=True,num_layers=1).cuda()
                
        #print('xlst',xlst.shape)    
        
        xlst = self.lsgn1(xlst)
        
        ylst = lstm(xlst)
        
        
        #print(hidden)
        
        f = np.asarray(ylst)
        
        h  = torch.cuda.FloatTensor(ylst[0])
        
        
        h = torch.squeeze(h)
        
        encoder_o = f[0]
        
        a = np.zeros((4,256,256))

        a = torch.from_numpy(a)
        a.cuda()
        
        for i in range(4):
    
            oo,b = attn_decoder1.forward(xlst,h[i],encoder_o[i])
            oo = self.lsgn2(oo)
            a[i] = oo
        
            
        a = a.unsqueeze(0)
        a = a.reshape([4,256,16,16])
        
        
        
        x5 = a  
        x5 = x5.cuda()
        
        
        x5 = x5.type(torch.cuda.FloatTensor)
 
        
        
        x5 = self.lsgn2(x5)
        
        ups4 = nn.ConvTranspose2d(256 , 256 // 2, kernel_size=2, stride=2)
        upsconv4 = DoubleConv(256,128)

        ups4 = ups4.cuda()
        
        opt = ups4(x5)
        
        x5 = opt
        
        x = self.up1(x5, x4)
        #x = self.gn5(x)
        
        x = self.up2(x, x3)
       # x = self.gn6(x)
       
        x = self.up3(x, x2)
        #x = self.gn7(x)
       
        x = self.up4(x, x1)
        logits = self.outc(x)
        #out    = self.out_softmax(logits)
        return logits

class UNetDoubleSmallGroupNormdifferent(nn.Module):
    def __init__(self, n_channels, n_classes,bilinear=True):
        super(UNetDoubleSmallGroupNormdifferent, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc     = DoubleConv(n_channels, 16)
        self.down1   = Down(16, 32)
        self.downnew = Down(16,16)
        self.down2   = Down(32, 64)
        self.down3   = Down(64, 128)
        factor = 2 if bilinear else 1
        self.down4   = Down(128, 256 // factor) 
        self.upsam   = nn.Upsample(scale_factor=2, mode='nearest')
        
        self.down5 = Down(128,256)
        self.ups3  = nn.ConvTranspose2d(1 , 1, kernel_size=2, stride=2)
        self.ups4  = nn.ConvTranspose2d(256 , 256 // 2, kernel_size=2, stride=2)
        
        self.up1 = Up(256, 128 // factor, bilinear)
        self.up2 = Up(128, 64 // factor, bilinear)
        self.up3 = Up(64, 32 // factor, bilinear)
        self.up4 = Up(32, 16, bilinear)
        self.outc = OutConv(16, n_classes)
        #self.out_sigmoid = nn.Sigmoid()
        self.out_softmax = nn.LogSoftmax(dim=1)
        
        self.lsgn1 = nn.GroupNorm(64,128)
        self.lsgn2 = nn.GroupNorm(64,1024)
        self.lsgn3 = nn.GroupNorm(64,1024)
        
        self.gn1 = nn.GroupNorm(8, 16)
        self.gn2 = nn.GroupNorm(16, 32)
        self.gn3 = nn.GroupNorm(32, 64)
        self.gn4 = nn.GroupNorm(64, 128)
        self.gn5 = nn.GroupNorm(32, 64)
        self.gn6 = nn.GroupNorm(16, 32)
        self.gn7 = nn.GroupNorm(8, 16)
        self.gn8 = nn.GroupNorm(4,8)
   
    def forward(self, x):
        #x = self.upsam()
        
        x1 = self.inc(x)
        #x1 = self.gn1(x1)
       
        x2 = self.down1(x1)
        #x2 = self.gn2(x2)
       
        x3 = self.down2(x2)
        #x3 = self.gn3(x3)
       
        x4 = self.down3(x3)
        #x4 = self.gn4(x4)
       
        x5 = self.down4(x4)
        #x5 = self.gn
        #x5 = torch.squeeze(x5)
        #x5 = self.down5(x5)
        #x5 = self.down6(x5)
        #print('x5:',x5.shape)
        
        xlst = x5.reshape([4,128,1024])
        

        lstm = nn.LSTM(1024,1024,batch_first= True,bidirectional=True,num_layers=1).cuda()
        
        xlst = self.lsgn1(xlst)
        ylst = lstm(xlst)
        
        f = np.asarray(ylst)
        
        h  = torch.cuda.FloatTensor(ylst[0])
        h = torch.squeeze(h)
        
        encoder_o = f[0]
        
        a = np.zeros((4,128,1024))
        #a = ndarray((4,128,1024))

        a = torch.from_numpy(a)
        a.cuda()
        
        for i in range(4):
            oo,b = attn_decoder1.forward(xlst,h[i],encoder_o[i])
            oo   = self.lsgn2(oo)
            a[i] = oo
        
            
        a = a.unsqueeze(0)
        a = a.reshape([4,128,32,32])
        
        
        x5 = a  
        x5 = x5.cuda()
        
        
        x5 = x5.type(torch.cuda.FloatTensor)
        #x5 = self.lsgn3(x5)
        
        #x5 = self.ups4(x5)
    
        x = self.up1(x5, x4)
        #x = self.gn5(x)
        
        x = self.up2(x, x3)
        #x = self.gn6(x)
       
        x = self.up3(x, x2)
        #x = self.gn7(x)
       
        x = self.up4(x, x1)
        #x = self.gn7(x)

        #x = self.downnew(x)
        
        #out    = self.out_softmax(logits)
        
        logits = self.outc(x)
        
        return logits

class UNetDoubleSmallWithoutGN(nn.Module):
    def __init__(self, n_channels, n_classes,bilinear=True):
        
        super(UNetDoubleSmallWithoutGN, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc   = DoubleConv(n_channels, 16)
        self.down1 = Down(16, 32)
        self.down2 = Down(32, 64)
        self.down3 = Down(64, 128)
        factor = 2 if bilinear else 1
        self.down4 = Down(128, 256 // factor)
        self.down5 = Down(128,256)
        
        self.up1 = Up(256, 128 // factor, bilinear)
        self.up2 = Up(128, 64 // factor, bilinear)
        self.up3 = Up(64, 32 // factor, bilinear)
        self.up4 = Up(32, 16, bilinear)
        self.outc = OutConv(16, n_classes)
        
    def forward(self, x):
        x1 = self.inc(x)
       # x1 = self.gn1(x1)
       
        x2 = self.down1(x1)
       # x2 = self.gn2(x2)
       
        x3 = self.down2(x2)
       # x3 = self.gn3(x3)
       
        x4 = self.down3(x3)
       # x4 = self.gn4(x4)
       
        x5 = self.down4(x4)
        
        #x5 = torch.squeeze(x5)
        x5 = self.down5(x5)
        #x5 = self.down6(x5)
        
        ups4     = nn.ConvTranspose2d(256 , 256 // 2, kernel_size=2, stride=2)
        upsconv4 = DoubleConv(256,128)
        ups4 = ups4.cuda()
        
        opt = ups4(x5)
        
        x5 = opt
        
        x = self.up1(x5, x4)
        #x = self.gn5(x)
        
        x = self.up2(x, x3)
       # x = self.gn6(x)
       
        x = self.up3(x, x2)
        #x = self.gn7(x)
       
        x = self.up4(x, x1)
        logits = self.outc(x)
        #out    = self.out_softmax(logits)
        return logits

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=128, bilinear=True):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p   = dropout_p
        self.max_length  = max_length
        self.bilinear    = bilinear
        self.n_classes   = 1

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn      = nn.Linear(2048, 1024)
        
        self.attn2   = nn.Linear(1024, 128)
        
        self.attn_24 = nn.Linear(self.hidden_size*4, self.hidden_size*2)
        
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        
        self.attn_combine_bilstm = nn.Linear(3072, 1024)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru     = nn.GRU(1024, 1024)
        self.out     = nn.Linear(1024, 1024)
       # self.hidden = nn.Parameter(torch.randn(4,256,256).cuda()),nn.Parameter(torch.randn(4,256,256).cuda())
       
        #self.lsgn_a = nn.GroupNorm(512,1024)
        self.lsbn_a1 = nn.BatchNorm1d(1024)
        #self.lsgn_a2 = nn.GroupNorm(512,1024)
        
        #self.lsgn_in = nn.GroupNorm(64,128)
        self.lsbn_in1 = nn.BatchNorm1d(2048)
        self.lsbn_in2 = nn.BatchNorm1d(1024)
        
        
        self.lsbn_in3 = nn.BatchNorm1d(128)#nn.GroupNorm(64,   128)
        self.lsbn_in4 = nn.BatchNorm1d(128)#nn.GroupNorm(64,   128)
        self.lsbn_in5 = nn.BatchNorm1d(1024)#nn.GroupNorm(512,  1024)
        
        self.down5 = Down(128,256)
        
        factor = 2 if bilinear else 1
                
        self.ups4     = nn.ConvTranspose2d(256 , 256 // 2, kernel_size=2, stride=2)
        self.upsconv4 = DoubleConv(256,128)

        self.lstm = nn.LSTM(256,256,batch_first=False,bidirectional=True,num_layers=1).cuda()
    
    def forward(self, input,hidden,encoder_outputs):
        
        h        = torch.unsqueeze(hidden, 0)
        embedded = input
        #embedded = self.lsgn_in1(embedded)
        embedded = self.dropout(embedded)
        
        hidden_bilstm = h[0]
        hidden_bilinn = hidden_bilstm
        
        hidden_bilinn = self.attn(hidden_bilinn)
        hidden_bilinn = self.lsbn_a1(hidden_bilinn)
        
        hidden_bi     = hidden_bilinn.unsqueeze(0)
        
        #print(hidden_bilinn.shape)
        
        attn_weights  = torch.cat((embedded[0], hidden_bilinn), 1)
        attn_weights  = self.lsbn_in1(attn_weights)
        
        attn_weights  = self.attn(attn_weights)
        attn_weights  = self.lsbn_in2(attn_weights)
        
        attn_weights  = F.softmax(attn_weights, dim=1)
        
        attn_weights  = self.attn2(attn_weights)
        attn_weights  = self.lsbn_in3(attn_weights)
        
        #print(attn_weights.unsqueeze(0).shape,encoder_outputs.unsqueeze(0).shape)
    
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))
        
        #print('attn_applied: encoder outputs',attn_applied[0].shape,encoder_outputs[0].shape)

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        
        output = self.attn_combine_bilstm(output).unsqueeze(0)
        output = F.relu(output)
        output = self.lsbn_in4(output)
        
        output, hidden = self.gru(output, hidden_bi)
        
        output = self.out(output[0])
        output = self.lsbn_in5(output)
        output = F.log_softmax(output, dim=1)
        return output, hidden


    def initHidden(self):
        return torch.randn(4, 256, self.hidden_size, device=device)
#model = SUNet(1, 1)
#model.cuda()

In [ ]:
# [STAR] For training different models for comparison on MOSMEDDATA dataset

import skimage
import torch.optim as optim
from tqdm import tqdm
import random
from skimage.transform import rotate, AffineTransform, warp
from scipy.stats import entropy
import numpy as np

import torch.optim as optim
from tqdm import tqdm
import random
from skimage.transform import rotate, AffineTransform, warp
from scipy.stats import entropy


basepath         = '/media/yu-hao/WindowsData/COVID_MOSCOW/COVID_MOSCOW/COVID19_1110/'
basepath_models  = '/media/yu-hao/WindowsData/COVID_MOSCOW/COVID_MOSCOW/COVID19_1110/models/single_models/'


def read_training_data(read_ids):
    x_array = []
    y_array = []
    
    for p in read_ids:
        name = basepath+'masks/'
        name = name+'study_'+p+'_mask.nii.gz'
        
        mask = sitk.GetArrayFromImage(sitk.ReadImage(name))
        vol  = sitk.GetArrayFromImage(sitk.ReadImage(name.replace('_mask.nii.gz', '.nii.gz').replace('masks', 'studies/CT-1')))
        
        for t in range(mask.shape[0]):
            temp  = np.count_nonzero(mask[t].flatten())
            if temp > 0:
                x_array.append(np.expand_dims(vol[t], axis=0))
                y_array.append(np.expand_dims(mask[t], axis=0))

    x_array = (np.array(x_array)+1024.0)/1024.0
    y_array = np.array(y_array)
    
    return x_array, y_array

def dice(im1, im2):
    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)
    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)
    return 2. * intersection.sum() / (im1.sum() + im2.sum()+0.00001)

def dice_loss(pred, target, smooth = 1.):
    pred = F.sigmoid(pred)
    
    pred   = pred.contiguous()
    target = target.contiguous()    
    intersection = (pred * target).sum(dim=2).sum(dim=2)
    loss = (1 - ((2. * intersection + smooth) / (pred.sum(dim=2).sum(dim=2) + target.sum(dim=2).sum(dim=2) + smooth)))
    return loss.mean()

def read_training_data_unlabelled(read_ids):
    x_array          = []
    x_array_lungmask = []
    
    names   = [x.split('_')[0] for x in read_ids]
    types   = [x.split('_')[1] for x in read_ids]
    count   = 0
    
    for p in names:
        name     = basepath+'studies/'+types[count]+'/'
        maskname = name+'study_'+p+'_mask.nii.gz'
        volname  = name+'study_'+p+'.nii.gz'
        
        mask = sitk.GetArrayFromImage(sitk.ReadImage(maskname))
        vol  = sitk.GetArrayFromImage(sitk.ReadImage(volname))
        mask[mask > 0] = 1
        
        for t in range(mask.shape[0]):
            if True:#t % 1 == 0:
                temp  = np.count_nonzero(mask[t].flatten())
                if temp > 0: # Check if lung region is present
                    x_array.append(np.expand_dims(vol[t], axis=0))
                    x_array_lungmask.append(np.expand_dims(mask[t], axis=0))
        
        count = count+1

    x_array          = (np.array(x_array)+1024.0)/1024.0
    x_array_lungmask = np.array(x_array_lungmask)
    
    return x_array, x_array_lungmask

def get_prediction(model, valx):
    output_array   = []
    batch_size     = 1
    
    model.eval()
    
    for ik in range(len(valx)//batch_size):
        x = valx[ik*batch_size:(ik+1)*batch_size, :, :, :]
        x = torch.tensor(x, device=device).float()

        output = model.forward(x)
        output = torch.sigmoid(output)
        output = output.data.cpu().numpy()
        #output[output > 0.5]= 1
        #output[output < 0.5]= 0
        
        for k in range(output.shape[0]):
            output_array.append(output[k, 0])
    
    output_array = np.array(output_array)
    output_array = np.expand_dims(output_array, 1)
    
    return output_array

def get_predictions(models, valx):
    output_array   = []
    batch_size     = 1
    
    for i in range(5):
        models[i].eval()
    
    for ik in range(len(valx)//batch_size):
        x = valx[ik*batch_size:(ik+1)*batch_size, :, :, :]
        x = torch.tensor(x, device=device).float()
        
        outputs = []
        for k in range(5):
            output = models[k].forward(x)
            output = torch.sigmoid(output)
            output = output.data.cpu().numpy()
            outputs.append(output)
        
        output_sum = np.zeros(outputs[0].shape, dtype='float16')
        for k in range(5):
            output_sum = output_sum+outputs[k]
        output_sum = output_sum/5.0
        
        for k in range(output.shape[0]):
            output_array.append(output_sum[k, 0])
    
    output_array = np.array(output_array)
    output_array = np.expand_dims(output_array, 1)
    
    return output_array

def get_filtered(valx, valy):
    valxf = []
    valyf = []
    
    for i in range(valx.shape[0]):
        if np.count_nonzero(valy[i]) > 0:
            valxf.append(valx[i])
            valyf.append(valy[i])
    return np.array(valxf), np.array(valyf)

def evaluate_result(model, valx, valy):
    model.eval()
    
    val_dice       = []
    batch_size     = 1
    for ik in range(len(valx)//batch_size):
        x = valx[ik*batch_size:(ik+1)*batch_size, :, :, :]
        y = valy[ik*batch_size:(ik+1)*batch_size, :, :, :]

        x = torch.tensor(x, device=device).float()

        output = model.forward(x)

        output = torch.sigmoid(output)        
        output = output.data.cpu().numpy()

        output[output < 0.5] = 0
        output[output > 0.5] = 1
        
        for pk in range(output.shape[0]):
            dt = dice(y[pk, 0, :, :], output[pk, 0, :, :])
            val_dice.append(dt)
    return val_dice

def train_model(model, batch_size, optimizer, criterion, trainx, trainy):
    loss_array = []
    
    model.train()
    
    for i in range(len(trainx)//batch_size):
        x = trainx[i*batch_size:(i+1)*batch_size, :, :, :]
        y = trainy[i*batch_size:(i+1)*batch_size, :, :, :]
        
        x = torch.tensor(x, device=device).float()
        y = torch.tensor(y, device=device).float()
        
        optimizer.zero_grad()
        output = model.forward(x)        
        loss   = criterion(output , y)
        loss.backward()
        
        loss_array.append(loss.item())
        optimizer.step()
    
    loss_array = np.mean(loss_array)
    return loss_array

def prepare_batch(batch_size, k_means, trainx_l, trainy_l, h):
    a = []
    b = []
    
    for i in range(int(batch_size/2)):
        idx = random.randint(0, trainx_l.shape[0]-1)
        c   = k_means.predict(np.reshape(trainx_l[idx].astype('float32'), [1, 512*512]))[0]
        
        a.append(trainx_l[idx])
        b.append(trainy_l[idx])
        
        idx = random.randint(0, len(h[c])-1)
        t1  = np.expand_dims(np.load(h[c][idx]), 0)
        t2  = np.expand_dims(np.load(h[c][idx].replace('-x', '-y')), 0)
        
        a.append(t1)
        b.append(t2)
   
    a1 = np.array(a).astype('float16')
    b1 = np.array(b).astype('float16')
   
    return a1, b1

def store_cluster_slices(model_teacher, k_means, version):
    epoch_array = np.arange(79)
    all_labels  = []
    step_size   = 10 
    count       = 0
    
    for epoch in epoch_array:
        temp_index               = epoch%(int(len(unlabelled_ids)/step_size))
        trainx, trainx_lungmask  = read_training_data_unlabelled(unlabelled_ids[temp_index*step_size:temp_index*step_size+step_size])
        trainy                   = get_prediction(model_teacher, trainx)
        
        #trainy = np.load('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/PREDICTION-NUMPY/'+str(epoch)+'.npy')
        trainy = np.reshape(trainy, [trainy.shape[0], 512*512])
        #print(epoch, trainy.shape, trainx.shape)
        
        l1     = k_means.predict(trainy)
        
        for jt, t in enumerate(l1):
            temp  = np.reshape(trainy[jt], [512, 512]).astype('float16')
            np.save('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/CLUSTER-NUMPY-'+str(version)+'/'+str(t)+'-'+str(count)+'-y.npy', temp)
            
            temp  = np.reshape(trainx[jt], [512, 512]).astype('float16')
            np.save('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/CLUSTER-NUMPY-'+str(version)+'/'+str(t)+'-'+str(count)+'-x.npy', temp)
            
            count = count+1
    
    return

def prepare_hash(version):
    all_cluster_files = glob.glob('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/CLUSTER-NUMPY-'+str(version)+'/*.npy')
    print('Version ', version, 'File name counts ', len(all_cluster_files))
    filename_hash = {}
    for i in range(50):
        filename_hash[i] = []

    for t in all_cluster_files:
        filename_hash[int(t.split('/')[-1].split('-')[0])].append(t)
    
    return filename_hash

def get_all_covid_lesions(valx, valy, lesion_size):
    lesion_shapes_x = []
    lesion_shapes_y = []
    
    for i in range(valy.shape[0]):
        tx           = valx[i, 0]
        blobs        = valy[i, 0]
        blobs_labels = skimage.measure.label(blobs, background=0)
        propsa       = skimage.measure.regionprops(blobs_labels)
        
        for k in range(len(propsa)):
            temp = (blobs_labels == propsa[k].label).astype('uint8')
            
            temp_size = np.count_nonzero(temp.flatten().astype('uint8'))
            if temp_size < lesion_size and temp_size > 5:
                slice_x, slice_y = ndimage.find_objects(temp == 1)[0]
                
                roi_y = 1-temp[slice_x, slice_y]
                roi_x = tx[slice_x, slice_y]*temp[slice_x, slice_y]
                
                lesion_shapes_x.append(roi_x)
                lesion_shapes_y.append(roi_y)
                
                lesion_shapes_x.append(roi_x.T)
                lesion_shapes_y.append(roi_y.T)
                
                lesion_shapes_x.append(np.rot90(roi_x, 180))
                lesion_shapes_y.append(np.rot90(roi_y, 180))
    
    return lesion_shapes_x, lesion_shapes_y

def get_augmented_slice(batch_size, read_ids, lesion_shapes_x, lesion_shapes_y):
    x_array          = []
    x_array_lungmask = []
    
    index   = random.randint(0, len(read_ids)-1)
    #print(read_ids[index])
    
    p       = read_ids[index].split('_')[0]
    types   = 'CT-1'#read_ids[index].split('_')[1]
    count   = 0
    
    name     = basepath+'studies/'+types+'/'
    maskname = name+'study_'+p+'_mask.nii.gz'
    volname  = name+'study_'+p+'.nii.gz'
    
    segmentation_mask = basepath+'masks/'
    segmentation_mask = segmentation_mask+'study_'+p+'_mask.nii.gz'
    
    mask     = sitk.GetArrayFromImage(sitk.ReadImage(maskname))
    vol      = (sitk.GetArrayFromImage(sitk.ReadImage(volname))+1024.0)/1024.0
    segmentation_mask = sitk.GetArrayFromImage(sitk.ReadImage(segmentation_mask))
    
    mask[mask > 0] = 1
    count          = 0
    
    while(count < batch_size):
        t     = np.random.randint(0, mask.shape[0]-1)
        temp  = np.count_nonzero(mask[t].flatten())
        
        # Check if lung region is present
        if temp > 0:
            st  = vol[t]
            i,j = np.nonzero(mask[t])
            
            index = random.randint(0, len(i)-1)
            
            i = i[index]
            j = j[index]
            
            lesion_index = random.randint(0, len(lesion_shapes_x)-1)
            
            lesion_x     = lesion_shapes_x[lesion_index]
            lesion_y     = lesion_shapes_y[lesion_index]
            
            sx     = int(lesion_x.shape[0]/2)
            sy     = int(lesion_x.shape[1]/2)
            
            if st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy].shape == lesion_x.shape:
                st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  =  lesion_y*st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]
                st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  =  lesion_x + st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]

                m1 = segmentation_mask[t]#np.zeros(st.shape)
                m1[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  += 1-lesion_y
                m1         = m1*mask[t]
                m1[m1 > 0] = 1

                x_array.append(np.expand_dims(st,          axis=0))
                x_array_lungmask.append(np.expand_dims(m1, axis=0))

                count = count+1

    x_array          = np.array(x_array)
    x_array_lungmask = np.array(x_array_lungmask)
    
    return x_array, x_array_lungmask

def get_multiple_augmented_slice(batch_size, read_ids, lesion_shapes_x, lesion_shapes_y):
    x_array          = []
    x_array_lungmask = []
    
    index   = random.randint(0, len(read_ids)-1)
    #print(read_ids[index])
    
    p       = read_ids[index].split('_')[0]
    types   = 'CT-1'#read_ids[index].split('_')[1]
    count   = 0
    
    name     = basepath+'studies/'+types+'/'
    maskname = name+'study_'+p+'_mask.nii.gz'
    volname  = name+'study_'+p+'.nii.gz'
    
    segmentation_mask = basepath+'masks/'
    segmentation_mask = segmentation_mask+'study_'+p+'_mask.nii.gz'
    
    mask     = sitk.GetArrayFromImage(sitk.ReadImage(maskname))
    vol      = (sitk.GetArrayFromImage(sitk.ReadImage(volname))+1024.0)/1024.0
    segmentation_mask = sitk.GetArrayFromImage(sitk.ReadImage(segmentation_mask))
    
    mask[mask > 0] = 1
    count          = 0
    
    while(count < batch_size):
        t     = np.random.randint(0, mask.shape[0]-1)
        temp  = np.count_nonzero(mask[t].flatten())
        
        # Check if lung region is present
        if temp > 0:
            st  = vol[t]
            #segmen
            ipl, jpl = np.nonzero(mask[t])
            
            lesion_count = random.randint(0, 5)
            temp_count   = 0
            
            while(temp_count < lesion_count):
                index = random.randint(0, len(ipl)-1)

                i = ipl[index]
                j = jpl[index]

                lesion_index = random.randint(0, len(lesion_shapes_x)-1)

                lesion_x     = lesion_shapes_x[lesion_index]
                lesion_y     = lesion_shapes_y[lesion_index]

                sx     = int(lesion_x.shape[0]/2)
                sy     = int(lesion_x.shape[1]/2)

                if st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy].shape == lesion_x.shape:
                    st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  =  lesion_y*st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]
                    st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  =  lesion_x + st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]

                    m1 = segmentation_mask[t]#np.zeros(st.shape)
                    m1[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  += 1-lesion_y
                    m1         = m1*mask[t]
                    m1[m1 > 0] = 1
                    segmentation_mask[t] = m1
                    temp_count           = temp_count + 1
            
            x_array.append(np.expand_dims(st,          axis=0))
            x_array_lungmask.append(np.expand_dims(m1, axis=0))
            
            count = count+1

    x_array          = np.array(x_array)
    x_array_lungmask = np.array(x_array_lungmask)
    
    return x_array, x_array_lungmask

def plot_figure_slope(model_save_name):
    N = 2
    a = val_dice_array1#np.convolve(val_dice_array1, np.ones((N,))/N, mode='valid')
    b = train_dice_array1#np.convolve(train_dice_array1, np.ones((N,))/N, mode='valid')
    c = test_dice_array1#np.convolve(test_dice_array1, np.ones((N,))/N, mode='valid')
    
    temp  = 0
    slope = 0
    #np.abs(np.abs(b[i]-b[i-1])-np.abs(a[i]-a[i-1])) < 0.1 and
    for i in range(1, len(a)):
        if b[i] >= b[i-1] and a[i] >= a[i-1]:
            temp  = i#np.argmax(a)
            slope = b[i]-b[i-1]-(a[i]-a[i-1])
            #print(i, slope, np.abs(b[i]-b[i-1]), np.abs(a[i]-a[i-1]), b[i], b[i-1])
    
    import matplotlib.pyplot as plt
    plt.plot(a)
    plt.plot(b)
    plt.plot(c)
    plt.ylabel('some numbers')
    plt.annotate('Index '+str(temp), xy=(0.75, 0.25), xycoords='axes fraction')
    plt.annotate('Train '+str(round(b[temp], 3)), xy=(0.75, 0.20), xycoords='axes fraction')
    plt.annotate('Val   '+str(round(a[temp], 3)), xy=(0.75, 0.15), xycoords='axes fraction')
    plt.annotate('Test  '+str(round(c[temp], 3)), xy=(0.75, 0.10), xycoords='axes fraction')
    plt.annotate('Slope '+str(round(slope, 3)),   xy=(0.75, 0.05), xycoords='axes fraction')
    #plt.text(6, 0, )
    #plt.text(6, 0.1, 'Val   '+str(round(a[temp], 3)))
    #plt.text(6, 0.2, 'Train '+str(round(b[temp], 3)))
    #plt.text(6, 0.3, 'Test  '+str(round(c[temp], 3)))
    
    plt.savefig(model_save_name+".png")
    
    plt.close()
    plt.clf()
    
    return

def sort_data(trainx1, trainy1):
    # Sort the data
    X = trainx1
    Y = trainy1
    r = [t for t in sorted(zip(Y,X), key=lambda pair: np.sum(pair[0].flatten()))]
    
    trainx = []
    trainy = []
    
    for i in range(len(X)):
        trainy.append(r[i][0])
        trainx.append(r[i][1])
    
    trainx = np.array(trainx)
    trainy = np.array(trainy)
    
    return trainx, trainy

def plot_figure(model_save_name):
    a = list(val_dice_array)#np.convolve(val_dice_array1, np.ones((N,))/N, mode='valid')
    b = list(train_dice_array)#np.convolve(train_dice_array1, np.ones((N,))/N, mode='valid')
    c = list(test_dice_array)#np.convolve(test_dice_array1, np.ones((N,))/N, mode='valid')
    
    #a.insert(0, 0)
    #b.insert(0, 0)
    #c.insert(0, 0)
#     temp  = 0
#     slope = 0
#     #np.abs(np.abs(b[i]-b[i-1])-np.abs(a[i]-a[i-1])) < 0.1 and
#     for i in range(1, len(a)):
#         if b[i] >= b[i-1] and a[i] >= a[i-1]:
#             temp  = i#np.argmax(a)
#             slope = b[i]-b[i-1]-(a[i]-a[i-1])
#             #print(i, slope, np.abs(b[i]-b[i-1]), np.abs(a[i]-a[i-1]), b[i], b[i-1])
    
    # Take arg max for semi model
    temp = np.argmax(a)
    
    import matplotlib.pyplot as plt
    plt.plot(a)
    plt.plot(b)
    plt.plot(c)
    plt.ylabel('some numbers')
    plt.annotate('Index '+str(temp), xy=(0.75, 0.25), xycoords='axes fraction')
    plt.annotate('Train '+str(round(b[temp], 3)), xy=(0.75, 0.20), xycoords='axes fraction')
    plt.annotate('Val   '+str(round(a[temp], 3)), xy=(0.75, 0.15), xycoords='axes fraction')
    plt.annotate('Test  '+str(round(c[temp], 3)), xy=(0.75, 0.10), xycoords='axes fraction')
    #plt.annotate('Slope '+str(round(slope, 3)),   xy=(0.75, 0.05), xycoords='axes fraction')
    #plt.text(6, 0, )
    #plt.text(6, 0.1, 'Val   '+str(round(a[temp], 3)))
    #plt.text(6, 0.2, 'Train '+str(round(b[temp], 3)))
    #plt.text(6, 0.3, 'Test  '+str(round(c[temp], 3)))
    
    plt.savefig(model_save_name+".png")
    
    plt.close()
    plt.clf()
    
    return
device         = torch.device("cuda:0")


train_ids      = np.load(basepath+'TRAIN.npy')
val_ids        = np.load(basepath+'VALIDATION.npy')
test_ids       = np.load(basepath+'TEST.npy')
unlabelled_ids = np.load(basepath+'NOTLABELLED.npy')
#nocovid_ids    = np.load(basepath+'NOCOVID.npy')


unlabelled_ids     = unlabelled_ids
train_ids          = train_ids#[:4]
val_ids            = val_ids
test_ids           = test_ids

trainx_l, trainy_l = read_training_data(train_ids)
valx, valy         = read_training_data(val_ids)
testx, testy       = read_training_data(test_ids)

print(trainx_l.shape, valx.shape, testx.shape)

def init_normal(m):
    if type(m) == nn.Conv2d:
        nn.init.kaiming_normal_(m.weight)
    if type(m) == nn.Linear:
        #nn.init.kaiming_normal_(m.weight)
        nn.init.xavier_uniform_(m.weight)
            
prev_max        = -1000
model_student   = SUNet(1, 1)
model_student.cuda()

optimizer_student  = optim.Adam(model_student.parameters(), lr=0.0001)
criterion          = nn.BCEWithLogitsLoss(torch.ones([1]).cuda())


val_dice_array   = []
train_dice_array = []
test_dice_array  = []

trainx, trainy   = sort_data(trainx_l, trainy_l)
total_epochs = 100


teacher_dice_array = []
for epoch in range(total_epochs):
    if epoch%10 ==1:
        print(epoch)

    train_loss    = train_model(model_student, 2, optimizer_student, criterion, trainx, trainy)
    
    val_dice      = evaluate_result(model_student, valx, valy)
    student_dice1 = evaluate_result(model_student, trainx, trainy)
    
    
    train_dice_array.append(np.mean(student_dice1))
    val_dice_array.append(np.mean(val_dice))
    
    model_save_name = "tmi-compare-sunet"
    
    if np.mean(val_dice) > prev_max:
        print("Step %d  Dice %.5f > %f  Train Dice %f " % (epoch, np.mean(val_dice), prev_max, np.mean(student_dice1)))
        prev_max     = np.mean(val_dice)
        torch.save(model_student.state_dict(), basepath_models+model_save_name+'-'+str(epoch)+".pt")

    #np.save(model_save_name+'_train.npy',      train_dice_array)
    #np.save(model_save_name+'_validation.npy', val_dice_array)

In [111]:
# np.save('/media/yu-hao/WindowsData/COVID_MOSCOW/COVID_MOSCOW/COVID19_1110/trainx_mosmed.npy', trainx_l)
# np.save('/media/yu-hao/WindowsData/COVID_MOSCOW/COVID_MOSCOW/COVID19_1110/trainy_mosmed.npy', trainy_l)
# np.save('/media/yu-hao/WindowsData/COVID_MOSCOW/COVID_MOSCOW/COVID19_1110/valx_mosmed.npy', valx)
# np.save('/media/yu-hao/WindowsData/COVID_MOSCOW/COVID_MOSCOW/COVID19_1110/valy_mosmed.npy', valy)
# np.save('/media/yu-hao/WindowsData/COVID_MOSCOW/COVID_MOSCOW/COVID19_1110/testx_mosmed.npy', testx)
# np.save('/media/yu-hao/WindowsData/COVID_MOSCOW/COVID_MOSCOW/COVID19_1110/testy_mosmed.npy', testy)

In [ ]:
# [STAR] For training LSTM model on MOSMEDDATA dataset

import skimage
import torch.optim as optim
from tqdm import tqdm
import random
from skimage.transform import rotate, AffineTransform, warp
from scipy.stats import entropy
import numpy as np

import torch.optim as optim
from tqdm import tqdm
import random
from skimage.transform import rotate, AffineTransform, warp
from scipy.stats import entropy


basepath         = '/media/yu-hao/WindowsData/COVID_MOSCOW/COVID_MOSCOW/COVID19_1110/'
basepath_models  = '/media/yu-hao/WindowsData/COVID_MOSCOW/COVID_MOSCOW/COVID19_1110/models/single_models/'


def read_training_data(read_ids):
    x_array = []
    y_array = []
    
    for p in read_ids:
        name = basepath+'masks/'
        name = name+'study_'+p+'_mask.nii.gz'
        
        mask = sitk.GetArrayFromImage(sitk.ReadImage(name))
        vol  = sitk.GetArrayFromImage(sitk.ReadImage(name.replace('_mask.nii.gz', '.nii.gz').replace('masks', 'studies/CT-1')))
        
        for t in range(mask.shape[0]):
            temp  = np.count_nonzero(mask[t].flatten())
            if temp > 1000:
                x_array.append(np.expand_dims(vol[t], axis=0))
                y_array.append(np.expand_dims(mask[t], axis=0))

    x_array = (np.array(x_array)+1024.0)/1024.0
    x_array[x_array < 0] = 0
    y_array = np.array(y_array)
    
    return x_array, y_array

def dice(im1, im2):
    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)
    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)
    return 2. * intersection.sum() / (im1.sum() + im2.sum()+0.00001)

def dice_loss(pred, target, smooth = 1.):
    pred = F.sigmoid(pred)
    
    pred   = pred.contiguous()
    target = target.contiguous()    
    intersection = (pred * target).sum(dim=2).sum(dim=2)
    loss = (1 - ((2. * intersection + smooth) / (pred.sum(dim=2).sum(dim=2) + target.sum(dim=2).sum(dim=2) + smooth)))
    return loss.mean()

def read_training_data_unlabelled(read_ids):
    x_array          = []
    x_array_lungmask = []
    
    names   = [x.split('_')[0] for x in read_ids]
    types   = [x.split('_')[1] for x in read_ids]
    count   = 0
    
    for p in names:
        name     = basepath+'studies/'+types[count]+'/'
        maskname = name+'study_'+p+'_mask.nii.gz'
        volname  = name+'study_'+p+'.nii.gz'
        
        mask = sitk.GetArrayFromImage(sitk.ReadImage(maskname))
        vol  = sitk.GetArrayFromImage(sitk.ReadImage(volname))
        mask[mask > 0] = 1
        
        for t in range(mask.shape[0]):
            if True:#t % 1 == 0:
                temp  = np.count_nonzero(mask[t].flatten())
                if temp > 0: # Check if lung region is present
                    x_array.append(np.expand_dims(vol[t], axis=0))
                    x_array_lungmask.append(np.expand_dims(mask[t], axis=0))
        
        count = count+1

    x_array          = (np.array(x_array)+1024.0)/1024.0
    x_array_lungmask = np.array(x_array_lungmask)
    
    return x_array, x_array_lungmask

def get_prediction(model, valx):
    output_array   = []
    batch_size     = 1
    
    model.eval()
    
    for ik in range(len(valx)//batch_size):
        x = valx[ik*batch_size:(ik+1)*batch_size, :, :, :]
        x = torch.tensor(x, device=device).float()

        output = model.forward(x)
        output = torch.sigmoid(output)
        output = output.data.cpu().numpy()
        #output[output > 0.5]= 1
        #output[output < 0.5]= 0
        
        for k in range(output.shape[0]):
            output_array.append(output[k, 0])
    
    output_array = np.array(output_array)
    output_array = np.expand_dims(output_array, 1)
    
    return output_array

def get_predictions(models, valx):
    output_array   = []
    batch_size     = 1
    
    for i in range(5):
        models[i].eval()
    
    for ik in range(len(valx)//batch_size):
        x = valx[ik*batch_size:(ik+1)*batch_size, :, :, :]
        x = torch.tensor(x, device=device).float()
        
        outputs = []
        for k in range(5):
            output = models[k].forward(x)
            output = torch.sigmoid(output)
            output = output.data.cpu().numpy()
            outputs.append(output)
        
        output_sum = np.zeros(outputs[0].shape, dtype='float16')
        for k in range(5):
            output_sum = output_sum+outputs[k]
        output_sum = output_sum/5.0
        
        for k in range(output.shape[0]):
            output_array.append(output_sum[k, 0])
    
    output_array = np.array(output_array)
    output_array = np.expand_dims(output_array, 1)
    
    return output_array

def get_filtered(valx, valy):
    valxf = []
    valyf = []
    
    for i in range(valx.shape[0]):
        if np.count_nonzero(valy[i]) > 0:
            valxf.append(valx[i])
            valyf.append(valy[i])
    return np.array(valxf), np.array(valyf)

def evaluate_result(model, valx, valy):
    model.eval()
    
    val_dice       = []
    batch_size     = 4
    for ik in range(len(valx)//batch_size):
        x = valx[ik*batch_size:(ik+1)*batch_size, :, :, :]
        y = valy[ik*batch_size:(ik+1)*batch_size, :, :, :]

        x = torch.tensor(x, device=device).float()

        output = model.forward(x)

        output = torch.sigmoid(output)        
        output = output.data.cpu().numpy()

        output[output < 0.5] = 0
        output[output > 0.5] = 1
        
        for pk in range(output.shape[0]):
            dt = dice(y[pk, 0, :, :], output[pk, 0, :, :])
            val_dice.append(dt)
    return val_dice

def train_model(model, batch_size, optimizer, criterion, trainx, trainy):
    loss_array = []
    
    model.train()
    
    for i in range(len(trainx)//batch_size):
        x = trainx[i*batch_size:(i+1)*batch_size, :, :, :]
        y = trainy[i*batch_size:(i+1)*batch_size, :, :, :]
        
        x = torch.tensor(x, device=device).float()
        y = torch.tensor(y, device=device).float()
        
        optimizer.zero_grad()
        output = model.forward(x)        
        loss   = criterion(output , y)
        loss.backward()
        
        loss_array.append(loss.item())
        optimizer.step()
    
    loss_array = np.mean(loss_array)
    return loss_array

def prepare_batch(batch_size, k_means, trainx_l, trainy_l, h):
    a = []
    b = []
    
    for i in range(int(batch_size/2)):
        idx = random.randint(0, trainx_l.shape[0]-1)
        c   = k_means.predict(np.reshape(trainx_l[idx].astype('float32'), [1, 512*512]))[0]
        
        a.append(trainx_l[idx])
        b.append(trainy_l[idx])
        
        idx = random.randint(0, len(h[c])-1)
        t1  = np.expand_dims(np.load(h[c][idx]), 0)
        t2  = np.expand_dims(np.load(h[c][idx].replace('-x', '-y')), 0)
        
        a.append(t1)
        b.append(t2)
   
    a1 = np.array(a).astype('float16')
    b1 = np.array(b).astype('float16')
   
    return a1, b1

def store_cluster_slices(model_teacher, k_means, version):
    epoch_array = np.arange(79)
    all_labels  = []
    step_size   = 10 
    count       = 0
    
    for epoch in epoch_array:
        temp_index               = epoch%(int(len(unlabelled_ids)/step_size))
        trainx, trainx_lungmask  = read_training_data_unlabelled(unlabelled_ids[temp_index*step_size:temp_index*step_size+step_size])
        trainy                   = get_prediction(model_teacher, trainx)
        
        #trainy = np.load('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/PREDICTION-NUMPY/'+str(epoch)+'.npy')
        trainy = np.reshape(trainy, [trainy.shape[0], 512*512])
        #print(epoch, trainy.shape, trainx.shape)
        
        l1     = k_means.predict(trainy)
        
        for jt, t in enumerate(l1):
            temp  = np.reshape(trainy[jt], [512, 512]).astype('float16')
            np.save('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/CLUSTER-NUMPY-'+str(version)+'/'+str(t)+'-'+str(count)+'-y.npy', temp)
            
            temp  = np.reshape(trainx[jt], [512, 512]).astype('float16')
            np.save('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/CLUSTER-NUMPY-'+str(version)+'/'+str(t)+'-'+str(count)+'-x.npy', temp)
            
            count = count+1
    
    return

def prepare_hash(version):
    all_cluster_files = glob.glob('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/CLUSTER-NUMPY-'+str(version)+'/*.npy')
    print('Version ', version, 'File name counts ', len(all_cluster_files))
    filename_hash = {}
    for i in range(50):
        filename_hash[i] = []

    for t in all_cluster_files:
        filename_hash[int(t.split('/')[-1].split('-')[0])].append(t)
    
    return filename_hash

def get_all_covid_lesions(valx, valy, lesion_size):
    lesion_shapes_x = []
    lesion_shapes_y = []
    
    for i in range(valy.shape[0]):
        tx           = valx[i, 0]
        blobs        = valy[i, 0]
        blobs_labels = skimage.measure.label(blobs, background=0)
        propsa       = skimage.measure.regionprops(blobs_labels)
        
        for k in range(len(propsa)):
            temp = (blobs_labels == propsa[k].label).astype('uint8')
            
            temp_size = np.count_nonzero(temp.flatten().astype('uint8'))
            if temp_size < lesion_size and temp_size > 5:
                slice_x, slice_y = ndimage.find_objects(temp == 1)[0]
                
                roi_y = 1-temp[slice_x, slice_y]
                roi_x = tx[slice_x, slice_y]*temp[slice_x, slice_y]
                
                lesion_shapes_x.append(roi_x)
                lesion_shapes_y.append(roi_y)
                
                lesion_shapes_x.append(roi_x.T)
                lesion_shapes_y.append(roi_y.T)
                
                lesion_shapes_x.append(np.rot90(roi_x, 180))
                lesion_shapes_y.append(np.rot90(roi_y, 180))
    
    return lesion_shapes_x, lesion_shapes_y

def get_augmented_slice(batch_size, read_ids, lesion_shapes_x, lesion_shapes_y):
    x_array          = []
    x_array_lungmask = []
    
    index   = random.randint(0, len(read_ids)-1)
    #print(read_ids[index])
    
    p       = read_ids[index].split('_')[0]
    types   = 'CT-1'#read_ids[index].split('_')[1]
    count   = 0
    
    name     = basepath+'studies/'+types+'/'
    maskname = name+'study_'+p+'_mask.nii.gz'
    volname  = name+'study_'+p+'.nii.gz'
    
    segmentation_mask = basepath+'masks/'
    segmentation_mask = segmentation_mask+'study_'+p+'_mask.nii.gz'
    
    mask     = sitk.GetArrayFromImage(sitk.ReadImage(maskname))
    vol      = (sitk.GetArrayFromImage(sitk.ReadImage(volname))+1024.0)/1024.0
    segmentation_mask = sitk.GetArrayFromImage(sitk.ReadImage(segmentation_mask))
    
    mask[mask > 0] = 1
    count          = 0
    
    while(count < batch_size):
        t     = np.random.randint(0, mask.shape[0]-1)
        temp  = np.count_nonzero(mask[t].flatten())
        
        # Check if lung region is present
        if temp > 0:
            st  = vol[t]
            i,j = np.nonzero(mask[t])
            
            index = random.randint(0, len(i)-1)
            
            i = i[index]
            j = j[index]
            
            lesion_index = random.randint(0, len(lesion_shapes_x)-1)
            
            lesion_x     = lesion_shapes_x[lesion_index]
            lesion_y     = lesion_shapes_y[lesion_index]
            
            sx     = int(lesion_x.shape[0]/2)
            sy     = int(lesion_x.shape[1]/2)
            
            if st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy].shape == lesion_x.shape:
                st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  =  lesion_y*st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]
                st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  =  lesion_x + st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]

                m1 = segmentation_mask[t]#np.zeros(st.shape)
                m1[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  += 1-lesion_y
                m1         = m1*mask[t]
                m1[m1 > 0] = 1

                x_array.append(np.expand_dims(st,          axis=0))
                x_array_lungmask.append(np.expand_dims(m1, axis=0))

                count = count+1

    x_array          = np.array(x_array)
    x_array_lungmask = np.array(x_array_lungmask)
    
    return x_array, x_array_lungmask

def get_multiple_augmented_slice(batch_size, read_ids, lesion_shapes_x, lesion_shapes_y):
    x_array          = []
    x_array_lungmask = []
    
    index   = random.randint(0, len(read_ids)-1)
    #print(read_ids[index])
    
    p       = read_ids[index].split('_')[0]
    types   = 'CT-1'#read_ids[index].split('_')[1]
    count   = 0
    
    name     = basepath+'studies/'+types+'/'
    maskname = name+'study_'+p+'_mask.nii.gz'
    volname  = name+'study_'+p+'.nii.gz'
    
    segmentation_mask = basepath+'masks/'
    segmentation_mask = segmentation_mask+'study_'+p+'_mask.nii.gz'
    
    mask     = sitk.GetArrayFromImage(sitk.ReadImage(maskname))
    vol      = (sitk.GetArrayFromImage(sitk.ReadImage(volname))+1024.0)/1024.0
    segmentation_mask = sitk.GetArrayFromImage(sitk.ReadImage(segmentation_mask))
    
    mask[mask > 0] = 1
    count          = 0
    
    while(count < batch_size):
        t     = np.random.randint(0, mask.shape[0]-1)
        temp  = np.count_nonzero(mask[t].flatten())
        
        # Check if lung region is present
        if temp > 0:
            st  = vol[t]
            #segmen
            ipl, jpl = np.nonzero(mask[t])
            
            lesion_count = random.randint(0, 5)
            temp_count   = 0
            
            while(temp_count < lesion_count):
                index = random.randint(0, len(ipl)-1)

                i = ipl[index]
                j = jpl[index]

                lesion_index = random.randint(0, len(lesion_shapes_x)-1)

                lesion_x     = lesion_shapes_x[lesion_index]
                lesion_y     = lesion_shapes_y[lesion_index]

                sx     = int(lesion_x.shape[0]/2)
                sy     = int(lesion_x.shape[1]/2)

                if st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy].shape == lesion_x.shape:
                    st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  =  lesion_y*st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]
                    st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  =  lesion_x + st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]

                    m1 = segmentation_mask[t]#np.zeros(st.shape)
                    m1[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  += 1-lesion_y
                    m1         = m1*mask[t]
                    m1[m1 > 0] = 1
                    segmentation_mask[t] = m1
                    temp_count           = temp_count + 1
            
            x_array.append(np.expand_dims(st,          axis=0))
            x_array_lungmask.append(np.expand_dims(m1, axis=0))
            
            count = count+1

    x_array          = np.array(x_array)
    x_array_lungmask = np.array(x_array_lungmask)
    
    return x_array, x_array_lungmask

def plot_figure_slope(model_save_name):
    N = 2
    a = val_dice_array1#np.convolve(val_dice_array1, np.ones((N,))/N, mode='valid')
    b = train_dice_array1#np.convolve(train_dice_array1, np.ones((N,))/N, mode='valid')
    c = test_dice_array1#np.convolve(test_dice_array1, np.ones((N,))/N, mode='valid')
    
    temp  = 0
    slope = 0
    #np.abs(np.abs(b[i]-b[i-1])-np.abs(a[i]-a[i-1])) < 0.1 and
    for i in range(1, len(a)):
        if b[i] >= b[i-1] and a[i] >= a[i-1]:
            temp  = i#np.argmax(a)
            slope = b[i]-b[i-1]-(a[i]-a[i-1])
            #print(i, slope, np.abs(b[i]-b[i-1]), np.abs(a[i]-a[i-1]), b[i], b[i-1])
    
    import matplotlib.pyplot as plt
    plt.plot(a)
    plt.plot(b)
    plt.plot(c)
    plt.ylabel('some numbers')
    plt.annotate('Index '+str(temp), xy=(0.75, 0.25), xycoords='axes fraction')
    plt.annotate('Train '+str(round(b[temp], 3)), xy=(0.75, 0.20), xycoords='axes fraction')
    plt.annotate('Val   '+str(round(a[temp], 3)), xy=(0.75, 0.15), xycoords='axes fraction')
    plt.annotate('Test  '+str(round(c[temp], 3)), xy=(0.75, 0.10), xycoords='axes fraction')
    plt.annotate('Slope '+str(round(slope, 3)),   xy=(0.75, 0.05), xycoords='axes fraction')
    #plt.text(6, 0, )
    #plt.text(6, 0.1, 'Val   '+str(round(a[temp], 3)))
    #plt.text(6, 0.2, 'Train '+str(round(b[temp], 3)))
    #plt.text(6, 0.3, 'Test  '+str(round(c[temp], 3)))
    
    plt.savefig(model_save_name+".png")
    
    plt.close()
    plt.clf()
    
    return

def sort_data(trainx1, trainy1):
    # Sort the data
    X = trainx1
    Y = trainy1
    r = [t for t in sorted(zip(Y,X), key=lambda pair: np.sum(pair[0].flatten()))]
    
    trainx = []
    trainy = []
    
    for i in range(len(X)):
        trainy.append(r[i][0])
        trainx.append(r[i][1])
    
    trainx = np.array(trainx)
    trainy = np.array(trainy)
    
    return trainx, trainy

def plot_figure(model_save_name):
    a = list(val_dice_array)#np.convolve(val_dice_array1, np.ones((N,))/N, mode='valid')
    b = list(train_dice_array)#np.convolve(train_dice_array1, np.ones((N,))/N, mode='valid')
    c = list(test_dice_array)#np.convolve(test_dice_array1, np.ones((N,))/N, mode='valid')
    
    #a.insert(0, 0)
    #b.insert(0, 0)
    #c.insert(0, 0)
#     temp  = 0
#     slope = 0
#     #np.abs(np.abs(b[i]-b[i-1])-np.abs(a[i]-a[i-1])) < 0.1 and
#     for i in range(1, len(a)):
#         if b[i] >= b[i-1] and a[i] >= a[i-1]:
#             temp  = i#np.argmax(a)
#             slope = b[i]-b[i-1]-(a[i]-a[i-1])
#             #print(i, slope, np.abs(b[i]-b[i-1]), np.abs(a[i]-a[i-1]), b[i], b[i-1])
    
    # Take arg max for semi model
    temp = np.argmax(a)
    
    import matplotlib.pyplot as plt
    plt.plot(a)
    plt.plot(b)
    plt.plot(c)
    plt.ylabel('some numbers')
    plt.annotate('Index '+str(temp), xy=(0.75, 0.25), xycoords='axes fraction')
    plt.annotate('Train '+str(round(b[temp], 3)), xy=(0.75, 0.20), xycoords='axes fraction')
    plt.annotate('Val   '+str(round(a[temp], 3)), xy=(0.75, 0.15), xycoords='axes fraction')
    plt.annotate('Test  '+str(round(c[temp], 3)), xy=(0.75, 0.10), xycoords='axes fraction')
    #plt.annotate('Slope '+str(round(slope, 3)),   xy=(0.75, 0.05), xycoords='axes fraction')
    #plt.text(6, 0, )
    #plt.text(6, 0.1, 'Val   '+str(round(a[temp], 3)))
    #plt.text(6, 0.2, 'Train '+str(round(b[temp], 3)))
    #plt.text(6, 0.3, 'Test  '+str(round(c[temp], 3)))
    
    plt.savefig(model_save_name+".png")
    
    plt.close()
    plt.clf()
    
    return
device         = torch.device("cuda:0")


train_ids      = np.load(basepath+'TRAIN.npy')
val_ids        = np.load(basepath+'VALIDATION.npy')
test_ids       = np.load(basepath+'TEST.npy')
unlabelled_ids = np.load(basepath+'NOTLABELLED.npy')
#nocovid_ids    = np.load(basepath+'NOCOVID.npy')


unlabelled_ids     = unlabelled_ids
train_ids          = train_ids#[:4]
val_ids            = val_ids
test_ids           = test_ids

trainx_l, trainy_l = read_training_data(train_ids)
valx, valy         = read_training_data(val_ids)
testx, testy       = read_training_data(test_ids)

print(trainx_l.shape, valx.shape, testx.shape)

def init_normal(m):
    if type(m) == nn.Conv2d:
        nn.init.kaiming_normal_(m.weight)
    if type(m) == nn.Linear:
        #nn.init.kaiming_normal_(m.weight)
        nn.init.xavier_uniform_(m.weight)
            
prev_max        = -1000
model_save_name = "tmi-compare-lstm"

attn_decoder1 = AttnDecoderRNN(256, 256, dropout_p=0.05)
attn_decoder1.cuda()

model_student   = UNetDoubleSmallGroupNormdifferent(1, 1)
model_student.cuda()

optimizer_student  = optim.Adam(model_student.parameters(), lr=0.001)
criterion          = nn.BCEWithLogitsLoss(torch.ones([1]).cuda())

optimizer_attn_w  = optim.Adam(attn_decoder1.parameters(), lr=0.001, weight_decay=1e-5)


val_dice_array   = []
train_dice_array = []
test_dice_array  = []

#trainx, trainy   = sort_data(trainx_l, trainy_l)
total_epochs = 100




teacher_dice_array = []
for epoch in range(total_epochs):
    if epoch%10 ==1:
        print(epoch)
    
    index  = np.random.permutation(trainx.shape[0])
    trainx = trainx[index]
    trainy = trainy[index]
    
    train_loss    = train_model(model_student, 4, optimizer_student, criterion, trainx, trainy)
    
    val_dice      = evaluate_result(model_student, valx, valy)
    student_dice1 = evaluate_result(model_student, trainx, trainy)
    student_dice2 = evaluate_result(model_student, testx, testy)
    
    
    train_dice_array.append(np.mean(student_dice1))
    val_dice_array.append(np.mean(val_dice))
    test_dice_array.append(np.mean(student_dice2))
    
    model_save_name = "tmi-compare-lstm-mosmed"
    
    #if np.mean(val_dice) > prev_max:
    print("Step %d  Train Dice %f Val Dice %.5f, Test Dice %.5f" % (epoch, np.mean(student_dice1), np.mean(val_dice), np.mean(student_dice2)))
    prev_max     = np.mean(val_dice)
    torch.save(model_student.state_dict(), basepath_models+model_save_name+'-'+str(epoch)+".pt")

    #np.save(model_save_name+'_train.npy',      train_dice_array)
    #np.save(model_save_name+'_validation.npy', val_dice_array)

In [ ]:
# [STAR] For training different models for comparison on COVID-19 dataset using LSTM Model

import skimage
import torch.optim as optim
from tqdm import tqdm
import random
from skimage.transform import rotate, AffineTransform, warp
import scipy
from scipy.stats import entropy
import numpy as np

import torch.optim as optim
from tqdm import tqdm
import random
from skimage.transform import rotate, AffineTransform, warp
from scipy.stats import entropy


basepath         = '/home/yu-hao/SEMISUNET/Dataset/'
basepath_models  = '/home/yu-hao/SEMISUNET/Dataset/models/'

def read_training_data(read_ids):
    x_array = []
    y_array = []
    
    for p in read_ids:
        name = basepath+'masks/'
        name = name+'study_'+p+'_mask.nii.gz'
        
        mask = sitk.GetArrayFromImage(sitk.ReadImage(name))
        vol  = sitk.GetArrayFromImage(sitk.ReadImage(name.replace('_mask.nii.gz', '.nii.gz').replace('masks', 'studies/CT-1')))
        
        for t in range(mask.shape[0]):
            temp  = np.count_nonzero(mask[t].flatten())
            if temp > 0:
                x_array.append(np.expand_dims(vol[t], axis=0))
                y_array.append(np.expand_dims(mask[t], axis=0))

    x_array = (np.array(x_array)+1024.0)/1024.0
    y_array = np.array(y_array)
    
    return x_array, y_array

def dice(im1, im2):
    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)
    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)
    return 2. * intersection.sum() / (im1.sum() + im2.sum()+0.00001)

def dice_loss(pred, target, smooth = 1.):
    pred = F.sigmoid(pred)
    
    pred   = pred.contiguous()
    target = target.contiguous()    
    intersection = (pred * target).sum(dim=2).sum(dim=2)
    loss = (1 - ((2. * intersection + smooth) / (pred.sum(dim=2).sum(dim=2) + target.sum(dim=2).sum(dim=2) + smooth)))
    return loss.mean()

def read_training_data_unlabelled(read_ids):
    x_array          = []
    x_array_lungmask = []
    
    names   = [x.split('_')[0] for x in read_ids]
    types   = [x.split('_')[1] for x in read_ids]
    count   = 0
    
    for p in names:
        name     = basepath+'studies/'+types[count]+'/'
        maskname = name+'study_'+p+'_mask.nii.gz'
        volname  = name+'study_'+p+'.nii.gz'
        
        mask = sitk.GetArrayFromImage(sitk.ReadImage(maskname))
        vol  = sitk.GetArrayFromImage(sitk.ReadImage(volname))
        mask[mask > 0] = 1
        
        for t in range(mask.shape[0]):
            if True:#t % 1 == 0:
                temp  = np.count_nonzero(mask[t].flatten())
                if temp > 0: # Check if lung region is present
                    x_array.append(np.expand_dims(vol[t], axis=0))
                    x_array_lungmask.append(np.expand_dims(mask[t], axis=0))
        
        count = count+1

    x_array          = (np.array(x_array)+1024.0)/1024.0
    x_array_lungmask = np.array(x_array_lungmask)
    
    return x_array, x_array_lungmask

def get_prediction(model, valx):
    output_array   = []
    batch_size     = 4
    
    model.eval()
    
    for ik in range(len(valx)//batch_size):
        x = valx[ik*batch_size:(ik+1)*batch_size, :, :, :]
        x = torch.tensor(x, device=device).float()

        output = model.forward(x)
        output = torch.sigmoid(output)
        output = output.data.cpu().numpy()
        #output[output > 0.5]= 1
        #output[output < 0.5]= 0
        
        for k in range(output.shape[0]):
            output_array.append(output[k, 0])
    
    output_array = np.array(output_array)
    output_array = np.expand_dims(output_array, 1)
    
    return output_array

def get_predictions(models, valx):
    output_array   = []
    batch_size     = 1
    
    for i in range(5):
        models[i].eval()
    
    for ik in range(len(valx)//batch_size):
        x = valx[ik*batch_size:(ik+1)*batch_size, :, :, :]
        x = torch.tensor(x, device=device).float()
        
        outputs = []
        for k in range(5):
            output = models[k].forward(x)
            output = torch.sigmoid(output)
            output = output.data.cpu().numpy()
            outputs.append(output)
        
        output_sum = np.zeros(outputs[0].shape, dtype='float16')
        for k in range(5):
            output_sum = output_sum+outputs[k]
        output_sum = output_sum/5.0
        
        for k in range(output.shape[0]):
            output_array.append(output_sum[k, 0])
    
    output_array = np.array(output_array)
    output_array = np.expand_dims(output_array, 1)
    
    return output_array

def get_filtered(valx, valy):
    valxf = []
    valyf = []
    
    for i in range(valx.shape[0]):
        if np.count_nonzero(valy[i]) > 0:
            valxf.append(valx[i])
            valyf.append(valy[i])
    return np.array(valxf), np.array(valyf)

def evaluate_result(model, valx, valy):
    model.eval()
    
    val_dice       = []
    batch_size     = 4
    for ik in range(len(valx)//batch_size):
        x = valx[ik*batch_size:(ik+1)*batch_size, :, :, :]
        y = valy[ik*batch_size:(ik+1)*batch_size, :, :, :]

        x = torch.tensor(x, device=device).float()

        output = model.forward(x)

        output = torch.sigmoid(output)        
        output = output.data.cpu().numpy()

        output[output < 0.5] = 0
        output[output > 0.5] = 1
        
        for pk in range(output.shape[0]):
            dt = dice(y[pk, 0, :, :], output[pk, 0, :, :])
            val_dice.append(dt)
    return val_dice

def evaluate_result_new(pred, valy):
    val_dice       = []
    batch_size     = 4
    
    for ik in range(len(valx)//batch_size):
        output = pred[ik*batch_size:(ik+1)*batch_size, :, :, :]
        y      = valy[ik*batch_size:(ik+1)*batch_size, :, :, :]
        
        output[output < 0.5] = 0
        output[output > 0.5] = 1
        
        for pk in range(output.shape[0]):
            t1 = scipy.ndimage.zoom(output[0, 0].astype('uint8'), 0.6875, order=0)
            t2 = scipy.ndimage.zoom(y[0, 0].astype('uint8'),      0.6875, order=0)
            #print(t1.shape, t2.shape)
            dt = dice(y[pk, 0, :, :], output[pk, 0, :, :])
            val_dice.append(dt)
    
    return val_dice

def train_model(model, batch_size, optimizer, criterion, trainx, trainy, augment=False):
    loss_array = []
    
    idx    = np.random.permutation(trainx.shape[0])
    trainx = trainx[idx]
    trainy = trainy[idx]
    
    model.train()
    
    for i in range(len(trainx)//batch_size):
        x = trainx[i*batch_size:(i+1)*batch_size, :, :, :]
        y = trainy[i*batch_size:(i+1)*batch_size, :, :, :]
        
        if augment:
            for k in range(x.shape[0]):
                rotv = random.randint(0, 3)
                x[k, 0, :, :] = np.rot90(x[k, 0, :, :], rotv)
                y[k, 0, :, :] = np.rot90(y[k, 0, :, :], rotv)
        
        x = torch.tensor(x, device=device).float()
        y = torch.tensor(y, device=device).float()
        
        optimizer.zero_grad()
        output = model.forward(x)        
        loss   = criterion(output , y)
        loss.backward()
        
        loss_array.append(loss.item())
        optimizer.step()
    
    loss_array = np.mean(loss_array)
    return loss_array

def prepare_batch(batch_size, k_means, trainx_l, trainy_l, h):
    a = []
    b = []
    
    for i in range(int(batch_size/2)):
        idx = random.randint(0, trainx_l.shape[0]-1)
        c   = k_means.predict(np.reshape(trainx_l[idx].astype('float32'), [1, 512*512]))[0]
        
        a.append(trainx_l[idx])
        b.append(trainy_l[idx])
        
        idx = random.randint(0, len(h[c])-1)
        t1  = np.expand_dims(np.load(h[c][idx]), 0)
        t2  = np.expand_dims(np.load(h[c][idx].replace('-x', '-y')), 0)
        
        a.append(t1)
        b.append(t2)
   
    a1 = np.array(a).astype('float16')
    b1 = np.array(b).astype('float16')
   
    return a1, b1

def store_cluster_slices(model_teacher, k_means, version):
    epoch_array = np.arange(79)
    all_labels  = []
    step_size   = 10 
    count       = 0
    
    for epoch in epoch_array:
        temp_index               = epoch%(int(len(unlabelled_ids)/step_size))
        trainx, trainx_lungmask  = read_training_data_unlabelled(unlabelled_ids[temp_index*step_size:temp_index*step_size+step_size])
        trainy                   = get_prediction(model_teacher, trainx)
        
        #trainy = np.load('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/PREDICTION-NUMPY/'+str(epoch)+'.npy')
        trainy = np.reshape(trainy, [trainy.shape[0], 512*512])
        #print(epoch, trainy.shape, trainx.shape)
        
        l1     = k_means.predict(trainy)
        
        for jt, t in enumerate(l1):
            temp  = np.reshape(trainy[jt], [512, 512]).astype('float16')
            np.save('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/CLUSTER-NUMPY-'+str(version)+'/'+str(t)+'-'+str(count)+'-y.npy', temp)
            
            temp  = np.reshape(trainx[jt], [512, 512]).astype('float16')
            np.save('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/CLUSTER-NUMPY-'+str(version)+'/'+str(t)+'-'+str(count)+'-x.npy', temp)
            
            count = count+1
    
    return

def prepare_hash(version):
    all_cluster_files = glob.glob('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/CLUSTER-NUMPY-'+str(version)+'/*.npy')
    print('Version ', version, 'File name counts ', len(all_cluster_files))
    filename_hash = {}
    for i in range(50):
        filename_hash[i] = []

    for t in all_cluster_files:
        filename_hash[int(t.split('/')[-1].split('-')[0])].append(t)
    
    return filename_hash

def get_all_covid_lesions(valx, valy, lesion_size):
    lesion_shapes_x = []
    lesion_shapes_y = []
    
    for i in range(valy.shape[0]):
        tx           = valx[i, 0]
        blobs        = valy[i, 0]
        blobs_labels = skimage.measure.label(blobs, background=0)
        propsa       = skimage.measure.regionprops(blobs_labels)
        
        for k in range(len(propsa)):
            temp = (blobs_labels == propsa[k].label).astype('uint8')
            
            temp_size = np.count_nonzero(temp.flatten().astype('uint8'))
            if temp_size < lesion_size and temp_size > 5:
                slice_x, slice_y = ndimage.find_objects(temp == 1)[0]
                
                roi_y = 1-temp[slice_x, slice_y]
                roi_x = tx[slice_x, slice_y]*temp[slice_x, slice_y]
                
                lesion_shapes_x.append(roi_x)
                lesion_shapes_y.append(roi_y)
                
                lesion_shapes_x.append(roi_x.T)
                lesion_shapes_y.append(roi_y.T)
                
                lesion_shapes_x.append(np.rot90(roi_x, 180))
                lesion_shapes_y.append(np.rot90(roi_y, 180))
    
    return lesion_shapes_x, lesion_shapes_y

def get_augmented_slice(batch_size, read_ids, lesion_shapes_x, lesion_shapes_y):
    x_array          = []
    x_array_lungmask = []
    
    index   = random.randint(0, len(read_ids)-1)
    #print(read_ids[index])
    
    p       = read_ids[index].split('_')[0]
    types   = 'CT-1'#read_ids[index].split('_')[1]
    count   = 0
    
    name     = basepath+'studies/'+types+'/'
    maskname = name+'study_'+p+'_mask.nii.gz'
    volname  = name+'study_'+p+'.nii.gz'
    
    segmentation_mask = basepath+'masks/'
    segmentation_mask = segmentation_mask+'study_'+p+'_mask.nii.gz'
    
    mask     = sitk.GetArrayFromImage(sitk.ReadImage(maskname))
    vol      = (sitk.GetArrayFromImage(sitk.ReadImage(volname))+1024.0)/1024.0
    segmentation_mask = sitk.GetArrayFromImage(sitk.ReadImage(segmentation_mask))
    
    mask[mask > 0] = 1
    count          = 0
    
    while(count < batch_size):
        t     = np.random.randint(0, mask.shape[0]-1)
        temp  = np.count_nonzero(mask[t].flatten())
        
        # Check if lung region is present
        if temp > 0:
            st  = vol[t]
            i,j = np.nonzero(mask[t])
            
            index = random.randint(0, len(i)-1)
            
            i = i[index]
            j = j[index]
            
            lesion_index = random.randint(0, len(lesion_shapes_x)-1)
            
            lesion_x     = lesion_shapes_x[lesion_index]
            lesion_y     = lesion_shapes_y[lesion_index]
            
            sx     = int(lesion_x.shape[0]/2)
            sy     = int(lesion_x.shape[1]/2)
            
            if st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy].shape == lesion_x.shape:
                st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  =  lesion_y*st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]
                st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  =  lesion_x + st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]

                m1 = segmentation_mask[t]#np.zeros(st.shape)
                m1[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  += 1-lesion_y
                m1         = m1*mask[t]
                m1[m1 > 0] = 1

                x_array.append(np.expand_dims(st,          axis=0))
                x_array_lungmask.append(np.expand_dims(m1, axis=0))

                count = count+1

    x_array          = np.array(x_array)
    x_array_lungmask = np.array(x_array_lungmask)
    
    return x_array, x_array_lungmask

def get_multiple_augmented_slice(batch_size, read_ids, lesion_shapes_x, lesion_shapes_y):
    x_array          = []
    x_array_lungmask = []
    
    index   = random.randint(0, len(read_ids)-1)
    #print(read_ids[index])
    
    p       = read_ids[index].split('_')[0]
    types   = 'CT-1'#read_ids[index].split('_')[1]
    count   = 0
    
    name     = basepath+'studies/'+types+'/'
    maskname = name+'study_'+p+'_mask.nii.gz'
    volname  = name+'study_'+p+'.nii.gz'
    
    segmentation_mask = basepath+'masks/'
    segmentation_mask = segmentation_mask+'study_'+p+'_mask.nii.gz'
    
    mask     = sitk.GetArrayFromImage(sitk.ReadImage(maskname))
    vol      = (sitk.GetArrayFromImage(sitk.ReadImage(volname))+1024.0)/1024.0
    segmentation_mask = sitk.GetArrayFromImage(sitk.ReadImage(segmentation_mask))
    
    mask[mask > 0] = 1
    count          = 0
    
    while(count < batch_size):
        t     = np.random.randint(0, mask.shape[0]-1)
        temp  = np.count_nonzero(mask[t].flatten())
        
        # Check if lung region is present
        if temp > 0:
            st  = vol[t]
            #segmen
            ipl, jpl = np.nonzero(mask[t])
            
            lesion_count = random.randint(0, 5)
            temp_count   = 0
            
            while(temp_count < lesion_count):
                index = random.randint(0, len(ipl)-1)

                i = ipl[index]
                j = jpl[index]

                lesion_index = random.randint(0, len(lesion_shapes_x)-1)

                lesion_x     = lesion_shapes_x[lesion_index]
                lesion_y     = lesion_shapes_y[lesion_index]

                sx     = int(lesion_x.shape[0]/2)
                sy     = int(lesion_x.shape[1]/2)

                if st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy].shape == lesion_x.shape:
                    st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  =  lesion_y*st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]
                    st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  =  lesion_x + st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]

                    m1 = segmentation_mask[t]#np.zeros(st.shape)
                    m1[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  += 1-lesion_y
                    m1         = m1*mask[t]
                    m1[m1 > 0] = 1
                    segmentation_mask[t] = m1
                    temp_count           = temp_count + 1
            
            x_array.append(np.expand_dims(st,          axis=0))
            x_array_lungmask.append(np.expand_dims(m1, axis=0))
            
            count = count+1

    x_array          = np.array(x_array)
    x_array_lungmask = np.array(x_array_lungmask)
    
    return x_array, x_array_lungmask

def plot_figure_slope(model_save_name):
    N = 2
    a = val_dice_array1#np.convolve(val_dice_array1, np.ones((N,))/N, mode='valid')
    b = train_dice_array1#np.convolve(train_dice_array1, np.ones((N,))/N, mode='valid')
    c = test_dice_array1#np.convolve(test_dice_array1, np.ones((N,))/N, mode='valid')
    
    temp  = 0
    slope = 0
    #np.abs(np.abs(b[i]-b[i-1])-np.abs(a[i]-a[i-1])) < 0.1 and
    for i in range(1, len(a)):
        if b[i] >= b[i-1] and a[i] >= a[i-1]:
            temp  = i#np.argmax(a)
            slope = b[i]-b[i-1]-(a[i]-a[i-1])
            #print(i, slope, np.abs(b[i]-b[i-1]), np.abs(a[i]-a[i-1]), b[i], b[i-1])
    
    import matplotlib.pyplot as plt
    plt.plot(a)
    plt.plot(b)
    plt.plot(c)
    plt.ylabel('some numbers')
    plt.annotate('Index '+str(temp), xy=(0.75, 0.25), xycoords='axes fraction')
    plt.annotate('Train '+str(round(b[temp], 3)), xy=(0.75, 0.20), xycoords='axes fraction')
    plt.annotate('Val   '+str(round(a[temp], 3)), xy=(0.75, 0.15), xycoords='axes fraction')
    plt.annotate('Test  '+str(round(c[temp], 3)), xy=(0.75, 0.10), xycoords='axes fraction')
    plt.annotate('Slope '+str(round(slope, 3)),   xy=(0.75, 0.05), xycoords='axes fraction')
    #plt.text(6, 0, )
    #plt.text(6, 0.1, 'Val   '+str(round(a[temp], 3)))
    #plt.text(6, 0.2, 'Train '+str(round(b[temp], 3)))
    #plt.text(6, 0.3, 'Test  '+str(round(c[temp], 3)))
    
    plt.savefig(model_save_name+".png")
    
    plt.close()
    plt.clf()
    
    return

def sort_data(trainx1, trainy1):
    # Sort the data
    X = trainx1
    Y = trainy1
    r = [t for t in sorted(zip(Y,X), key=lambda pair: np.sum(pair[0].flatten()))]
    
    trainx = []
    trainy = []
    
    for i in range(len(X)):
        trainy.append(r[i][0])
        trainx.append(r[i][1])
    
    trainx = np.array(trainx)
    trainy = np.array(trainy)
    
    return trainx, trainy

def plot_figure(model_save_name):
    a = list(val_dice_array)#np.convolve(val_dice_array1, np.ones((N,))/N, mode='valid')
    b = list(train_dice_array)#np.convolve(train_dice_array1, np.ones((N,))/N, mode='valid')
    c = list(test_dice_array)#np.convolve(test_dice_array1, np.ones((N,))/N, mode='valid')
    
    #a.insert(0, 0)
    #b.insert(0, 0)
    #c.insert(0, 0)
#     temp  = 0
#     slope = 0
#     #np.abs(np.abs(b[i]-b[i-1])-np.abs(a[i]-a[i-1])) < 0.1 and
#     for i in range(1, len(a)):
#         if b[i] >= b[i-1] and a[i] >= a[i-1]:
#             temp  = i#np.argmax(a)
#             slope = b[i]-b[i-1]-(a[i]-a[i-1])
#             #print(i, slope, np.abs(b[i]-b[i-1]), np.abs(a[i]-a[i-1]), b[i], b[i-1])
    
    # Take arg max for semi model
    temp = np.argmax(a)
    
    import matplotlib.pyplot as plt
    plt.plot(a)
    plt.plot(b)
    plt.plot(c)
    plt.ylabel('some numbers')
    plt.annotate('Index '+str(temp), xy=(0.75, 0.25), xycoords='axes fraction')
    plt.annotate('Train '+str(round(b[temp], 3)), xy=(0.75, 0.20), xycoords='axes fraction')
    plt.annotate('Val   '+str(round(a[temp], 3)), xy=(0.75, 0.15), xycoords='axes fraction')
    plt.annotate('Test  '+str(round(c[temp], 3)), xy=(0.75, 0.10), xycoords='axes fraction')
    #plt.annotate('Slope '+str(round(slope, 3)),   xy=(0.75, 0.05), xycoords='axes fraction')
    #plt.text(6, 0, )
    #plt.text(6, 0.1, 'Val   '+str(round(a[temp], 3)))
    #plt.text(6, 0.2, 'Train '+str(round(b[temp], 3)))
    #plt.text(6, 0.3, 'Test  '+str(round(c[temp], 3)))
    
    plt.savefig(model_save_name+".png")
    
    plt.close()
    plt.clf()
    
    return

def train_model1(model, batch_size, optimizer, criterion, trainx, trainy, augment=False):
    loss_array = []
   
    model.train()
    #print(len(trainx)//batch_size)
   
    for i in range(len(trainx)//batch_size):
        x = trainx[i*batch_size:(i+1)*batch_size, 0, :, :]
        y = trainy[i*batch_size:(i+1)*batch_size, 0, :, :]
                
        if augment:
            for k in range(x.shape[0]):
                rotv = random.randint(0, 3)
                x[k, 0, :, :] = np.rot90(x[k, 0, :, :], rotv)
                y[k, 0, :, :] = np.rot90(y[k, 0, :, :], rotv)
       
        #x2 = model.forward(x)        
        #print(x2.shape)
        
#         lstm = nn.LSTM(512*512,512*512,batchfirst=True)
#         hidden = (torch.randn(1, 512, 512), torch.randn(1, 512, 512))
#         outlstm = lstm(x, hidden)
#         n = np.asarray(outlstm)
  
        print(i, x.shape[0])
        
        if(x.shape[0]!= 4):
            break
    
        x = np.expand_dims(x, 1)
        y = np.expand_dims(y, 1)
        y = torch.tensor(y, device=device).float()
        x = torch.tensor(x, device=device).float()
        
       
        optimizer.zero_grad()
        optimizer_attn_w.zero_grad()
        
        output = model.forward(x)
        #print(i,attn_weights[1])
        
        loss   = criterion(output , y)
        loss.backward()
       
        loss_array.append(loss.item())
        
       # torch.nn.utils.clip_grad_norm(attn_decoder1.parameters(),0.7)
        
        optimizer.step()
        optimizer_attn_w.step()
   
    loss_array = np.mean(loss_array)
    return loss_array

def train_model2(model, batch_size, optimizer, criterion, trainx, trainy, augment=False):
    #batch_size = 4
    loss_array = []
   
    model.train()
    #print(len(trainx)//batch_size)
   
    for i in range(len(trainx)//batch_size):
        x = trainx[i*batch_size:(i+1)*batch_size, 0, :, :]
        y = trainy[i*batch_size:(i+1)*batch_size, 0, :, :]
        
                
        if augment:
            for k in range(x.shape[0]):
                rotv = random.randint(0, 3)
                x[k, 0, :, :] = np.rot90(x[k, 0, :, :], rotv)
                y[k, 0, :, :] = np.rot90(y[k, 0, :, :], rotv)
       
  
        if(x.shape[0]!=4):
            break
            
    
        x = np.expand_dims(x, 1)
        y = np.expand_dims(y, 1)

        y = torch.tensor(y, device=device).float()
        x = torch.tensor(x, device=device).float()
        
        optimizer.zero_grad()
        optimizer_attn_w.zero_grad()
        
        output = model.forward(x)
        #print(i,attn_weights[1])
        
        loss   = criterion(output , y)
        loss.backward()
       
        loss_array.append(loss.item())
        
       # torch.nn.utils.clip_grad_norm(attn_decoder1.parameters(),0.7)
        
        optimizer.step()
        optimizer_attn_w.step()
   
    loss_array = np.mean(loss_array)
    return loss_array

device         = torch.device("cuda:0")


# train_ids      = np.load(basepath+'TRAIN.npy')
# val_ids        = np.load(basepath+'VALIDATION.npy')
# test_ids       = np.load(basepath+'TEST.npy')
# unlabelled_ids = np.load(basepath+'NOTLABELLED.npy')
# nocovid_ids    = np.load(basepath+'NOCOVID.npy')


# unlabelled_ids     = unlabelled_ids
# train_ids          = train_ids[:4]
# val_ids            = val_ids
# test_ids           = test_ids


covid19 = False
mosmed  = True

# For COVID-19 dataset
if covid19:
    trainx_l = np.load(basepath+'train_x.npy')/255.0
    trainy_l = np.load(basepath+'train_y.npy')
    trainy_l[trainy_l > 0] = 1

    train_size    = 45
    valx          = trainx_l[train_size:]
    valy          = trainy_l[train_size:]

    trainx_l = trainx_l[:train_size]
    trainy_l = trainy_l[:train_size]

    testx = np.load(basepath+'test_x.npy')/255.0
    testy = np.load(basepath+'test_y.npy')
    testy[testy > 0] = 1

    trainx_l1 = np.zeros([trainx_l.shape[0], 1, 512, 512], dtype='float16')
    valx1     = np.zeros([valx.shape[0], 1, 512, 512],     dtype='float16')
    testx1    = np.zeros([testx.shape[0], 1, 512, 512],    dtype='float16')

    trainy_l1 = np.zeros([trainy_l.shape[0], 1, 512, 512], dtype='float16')
    valy1     = np.zeros([valy.shape[0], 1, 512, 512],     dtype='float16')
    testy1    = np.zeros([testy.shape[0], 1, 512, 512],    dtype='float16')
    
    for i in range(trainx_l.shape[0]):
        trainx_l1[i, 0] = scipy.ndimage.zoom(trainx_l[i], 2, order=3)
        trainy_l1[i, 0] = scipy.ndimage.zoom(trainy_l[i], 2, order=0)

    for i in range(valx.shape[0]):
        valx1[i, 0] = scipy.ndimage.zoom(valx[i], 2, order=3)
        valy1[i, 0] = scipy.ndimage.zoom(valy[i], 2, order=0)

    for i in range(testx.shape[0]):
        testx1[i, 0] = scipy.ndimage.zoom(testx[i], 2, order=3)
        testy1[i, 0] = scipy.ndimage.zoom(testy[i], 2, order=0)
    
    trainx_l = trainx_l1
    trainy_l = trainy_l1
    valx     = valx1
    valy     = valy1
    testx    = testx1
    testy    = testy1

# For Mosmed Dataset
if mosmed:
    basepath         = '/media/yu-hao/WindowsData/COVID_MOSCOW/COVID_MOSCOW/COVID19_1110/'
    basepath_models  = '/media/yu-hao/WindowsData/COVID_MOSCOW/COVID_MOSCOW/COVID19_1110/models/single_models/'

    train_ids      = np.load(basepath+'TRAIN.npy')
    val_ids        = np.load(basepath+'VALIDATION.npy')
    test_ids       = np.load(basepath+'TEST.npy')
    
    train_ids          = train_ids
    val_ids            = val_ids
    test_ids           = test_ids
    
    trainx_l, trainy_l = read_training_data(train_ids)
    valx, valy         = read_training_data(val_ids)
    testx, testy       = read_training_data(test_ids)

    
print(trainx_l.shape, valx.shape, testx.shape)


def init_normal(m):
    if type(m) == nn.Conv2d:
        nn.init.kaiming_normal_(m.weight)
    if type(m) == nn.Linear:
        #nn.init.kaiming_normal_(m.weight)
        nn.init.xavier_uniform_(m.weight)

attn_decoder1 = AttnDecoderRNN(256, 256, dropout_p=0.5)
attn_decoder1.cuda()

prev_max        = -1000
model_student   = UNetDoubleSmallGroupNormdifferent(1, 1)
model_student.cuda()

optimizer_student  = optim.Adam(model_student.parameters(), lr=0.001)
criterion          = nn.BCEWithLogitsLoss(torch.ones([1]).cuda())

optimizer_attn_w  = optim.Adam(attn_decoder1.parameters(), lr=0.001, weight_decay=1e-2)

# Good setting
# 0.5, 0.001, 0.001

val_dice_array   = []
train_dice_array = []
test_dice_array  = []

trainx, trainy   = sort_data(trainx_l, trainy_l)
total_epochs = 200


# trainx = np.expand_dims(trainx, axis=1)
# trainy = np.expand_dims(trainy, axis=1)

# valx   = np.expand_dims(valx, axis=1)
# valy   = np.expand_dims(valy, axis=1)

# testx  = np.expand_dims(testx, axis=1)
# testy  = np.expand_dims(testy, axis=1)

teacher_dice_array = []
test_dice_array    = []


for epoch in range(total_epochs):
    if epoch%10 ==1:
        print(epoch)
    
    index  = np.random.permutation(trainx.shape[0])
    trainx = trainx[index]
    trainy = trainy[index]
    
    #train_model1(model, optimizer, criterion, trainx, trainy, augment=False)
    train_loss    = train_model2(model_student, 4, optimizer_student, criterion, trainx, trainy, False)
    #train_loss    = train_model(model_student, 2, optimizer_student, criterion, trainx, trainy, False)
    
    pred      = get_prediction(model_student, valx)
    val_dice1 = evaluate_result_new(pred, valy)
    #print(pred.shape, len(val_dice1), valy.shape)
    
    pred          = get_prediction(model_student, testx)
    student_dice2 = evaluate_result_new(pred, testy)
    #print(pred.shape, len(student_dice2), testy.shape)
    
    #val_dice      = evaluate_result(model_student, valx,   valy)
    student_dice1 = evaluate_result(model_student, trainx, trainy)
    #student_dice2 = evaluate_result(model_student, testx,  testy)
    
    
    train_dice_array.append(np.mean(student_dice1))
    val_dice_array.append(np.mean(val_dice1))
    test_dice_array.append(np.mean(student_dice2))

    #model_save_name = "ipmi-attentionlstm-covid19"
    model_save_name = "tmi-compare-lstm"
    
    #if np.mean(val_dice1) > prev_max:
    print("Step %d  Val Dice %.5f, Train Dice %f, Test Dice %f" % (epoch, np.mean(val_dice1), np.mean(student_dice1), np.mean(student_dice2)))
    prev_max     = np.mean(val_dice1)
    torch.save(model_student.state_dict(), basepath_models+model_save_name+'-studentmodel.pt')
    torch.save(attn_decoder1.state_dict(), basepath_models+model_save_name+'-attention.pt')
    #attn_decoder1

(480, 1, 512, 512) (179, 1, 512, 512) (126, 1, 512, 512)
Step 0  Val Dice 0.00000, Train Dice 0.000000, Test Dice 0.000000
1
Step 1  Val Dice 0.03130, Train Dice 0.055555, Test Dice 0.076240
Step 2  Val Dice 0.17153, Train Dice 0.190977, Test Dice 0.280173
Step 3  Val Dice 0.21350, Train Dice 0.253893, Test Dice 0.323412
Step 4  Val Dice 0.41975, Train Dice 0.368689, Test Dice 0.430583
Step 5  Val Dice 0.51642, Train Dice 0.509652, Test Dice 0.552892
Step 6  Val Dice 0.45213, Train Dice 0.413167, Test Dice 0.457101
Step 7  Val Dice 0.51544, Train Dice 0.500628, Test Dice 0.542900
Step 8  Val Dice 0.42377, Train Dice 0.431474, Test Dice 0.474605
Step 9  Val Dice 0.47723, Train Dice 0.514814, Test Dice 0.559402
Step 10  Val Dice 0.52624, Train Dice 0.562126, Test Dice 0.598172
11
Step 11  Val Dice 0.52085, Train Dice 0.515840, Test Dice 0.552679
Step 12  Val Dice 0.32870, Train Dice 0.343167, Test Dice 0.372356
Step 13  Val Dice 0.47750, Train Dice 0.481559, Test Dice 0.525449
Step 14  V

In [ ]:
# [STAR] Double Attention Model

class UNetDoubleSmallGroupNormdifferent(nn.Module):
    def __init__(self, n_channels, n_classes,bilinear=True):
        
        super(UNetDoubleSmallGroupNormdifferent, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 16)
        self.down1 = Down(16, 32)
        self.downnew =Down(16,16)
        self.down2 = Down(32, 64)
        self.down3 = Down(64, 128)
        factor = 2 if bilinear else 1
        self.down4 = Down(128, 256 // factor)
        
        self.upsam = nn.Upsample(scale_factor=2, mode='nearest')

        
        self.down5 = Down(128,256)
        
        self.ups3 = nn.ConvTranspose2d(1 , 1, kernel_size=2, stride=2)
        
        self.ups4  = nn.ConvTranspose2d(256 , 256 // 2, kernel_size=2, stride=2)
        
        
        self.up1 = Up(256, 128 // factor, bilinear)
        self.up2 = Up(128, 64 // factor, bilinear)
        self.up3 = Up(64, 32 // factor, bilinear)
        self.up4 = Up(32, 16, bilinear)
        self.outc = OutConv(16, n_classes)
        #self.out_sigmoid = nn.Sigmoid()
        self.out_softmax = nn.LogSoftmax(dim=1)
        
        self.lsgn1 = nn.GroupNorm(64,128)
        
        self.lsgn2 = nn.GroupNorm(64,1024)
        
        self.lsgnp1 = nn.GroupNorm(128,256)
        
        self.lsgnp2 = nn.GroupNorm(64,256)
        
        
        self.gn1 = nn.GroupNorm(8, 16)
        self.gn2 = nn.GroupNorm(16, 32)
        self.gn3 = nn.GroupNorm(32, 64)
        self.gn4 = nn.GroupNorm(64, 128)
        self.gn5 = nn.GroupNorm(32, 64)
        self.gn6 = nn.GroupNorm(16, 32)
        self.gn7 = nn.GroupNorm(8, 16)
        self.gn8 = nn.GroupNorm(4,8)
   
    def forward(self, x):
        #x = self.upsam()
        
        x1 = self.inc(x)
        #x1 = self.gn1(x1)
       
        x2 = self.down1(x1)
        #x2 = self.gn2(x2)
       
        x3 = self.down2(x2)
        #x3 = self.gn3(x3)
       
        x4 = self.down3(x3)
        #x4 = self.gn4(x4)
       
        x5 = self.down4(x4)
        
        #x5 = torch.squeeze(x5)
        #x5 = self.down5(x5)
        #x5 = self.down6(x5)
        
        #print('x5:',x5.shape)
        
        ############### first attention layer for size 128,32,32 ---part 1 ######################
        xlst = x5.reshape([4,128,1024])
        

        lstm = nn.LSTM(1024,1024,batch_first= True,bidirectional=True,num_layers=1).cuda()
                
        #print('xlst',xlst.shape)    
        
        xlst = self.lsgn1(xlst)
        
        ylst = lstm(xlst)
        
        #print(hidden)
        
        f = np.asarray(ylst)
        
        #print(f.shape)
        
        h  = torch.cuda.FloatTensor(ylst[0])
        h = torch.squeeze(h)
        
        encoder_o = f[0]
        
        a = np.zeros((4,128,1024))

        a = torch.from_numpy(a)
        a.cuda()
        
        for i in range(4):
    
            oo,b = attn_decoder1.forward(xlst,h[i],encoder_o[i])
            oo = self.lsgn2(oo)
            a[i] = oo
        
            
        a = a.unsqueeze(0)
        a = a.reshape([4,128,32,32])
        
        
        
        x5 = a  
        x5 = x5.cuda()
        
        
        x5 = x5.type(torch.cuda.FloatTensor)
        x5 = self.lsgn1(x5)
        ############### second attention layer for size 256,16,16 ---part 2 ######################
        x5 = self.down5(x5)
        xlst = x5.reshape([4,256,256])
        lstm = nn.LSTM(256,256,batch_first= True,bidirectional=True,num_layers=1).cuda()
        
        xlst = self.lsgnp1(xlst)
        
        ylst = lstm(xlst)
        
        
        #print(hidden)
        
        f = np.asarray(ylst)
    
        h  = torch.cuda.FloatTensor(ylst[0])
        h = torch.squeeze(h)
        
        encoder_o = f[0]
        
        a = np.zeros((4,256,256))

        a = torch.from_numpy(a)
        a.cuda()
        
        for i in range(4):
    
            oo,b = attn_decoder2.forward(xlst,h[i],encoder_o[i])
            oo = self.lsgnp2(oo)
            a[i] = oo
        
            
        a = a.unsqueeze(0)
        a = a.reshape([4,256,16,16])
        
        
        
        x5 = a  
        x5 = x5.cuda()
        
        
        x5 = x5.type(torch.cuda.FloatTensor)
        x5 = self.lsgnp2(x5)
        
        x5 = self.ups4(x5)
        
        #x5 = self.ups4(x5)
    
        x = self.up1(x5, x4)
        #x = self.gn5(x)
        
        x = self.up2(x, x3)
        #x = self.gn6(x)
       
        x = self.up3(x, x2)
        #x = self.gn7(x)
       
        x = self.up4(x, x1)
        #x = self.gn7(x)

        #x = self.downnew(x)
        
        #out    = self.out_softmax(logits)
        
        logits = self.outc(x)
        
        return logits

class AttnDecoderRNN_2(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=256, bilinear=True):
        super(AttnDecoderRNN_2, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        self.bilinear = bilinear
        self.n_classes = 1

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size*2, self.max_length)
        
        self.attn_24 = nn.Linear(self.hidden_size*4, self.hidden_size*2)
        
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        
        self.attn_combine_bilstm = nn.Linear(self.hidden_size * 3, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)
       # self.hidden = nn.Parameter(torch.randn(4,256,256).cuda()),nn.Parameter(torch.randn(4,256,256).cuda())
       
        self.lsgn_a = nn.GroupNorm(128,256)
    
        self.down5 = Down(128,256)
        
        factor = 2 if bilinear else 1
                
        self.ups4 = nn.ConvTranspose2d(256 , 256 // 2, kernel_size=2, stride=2)
        self.upsconv4 = DoubleConv(256,128)

        self.lstm = nn.LSTM(256,256,batch_first=False,bidirectional=True,num_layers=1).cuda()
    
    def forward(self, input,hidden,encoder_outputs):
        
        h = torch.unsqueeze(hidden,0)
        
        embedded = input
        
        embedded = self.dropout(embedded)
        
        embedded =self.lsgn_a(embedded)

        hidden_bilstm = h[0]
        
        
        hidden_bilinn =  hidden_bilstm
        
        hidden_bilinn = self.attn(hidden_bilinn)
    
        hidden_bilinn = self.lsgn_a(hidden_bilinn)

        attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden_bilinn), 1)), dim=1)
        
        
        
        attn_weights  = self.lsgn_a(attn_weights)
    
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),encoder_outputs.unsqueeze(0))
        
   #     print('attn_applied: encoder outputs',attn_applied[0].shape,encoder_outputs[0].shape)


        output = torch.cat((embedded[0], attn_applied[0]), 1)
  #      print('The output shape is : ',output.shape)
        
        output = self.attn_combine_bilstm(output).unsqueeze(0)
 #      print('The output shape after is : ',output.shape)
        
    
        hidden_bi = hidden_bilinn.unsqueeze(0)
        
        output = F.relu(output)
        
        output = self.lsgn_a(output)
        #print("output and hidden before lstm ",output.shape,hidden_bi.shape)

        output, hidden = self.gru(output, hidden_bi)
        
        output = F.log_softmax(self.out(output[0]), dim=1)
        output = self.lsgn_a(output)
        
       #output = self.lsgn_a(output)
    
        return output,hidden


    def initHidden(self):
        return torch.randn(4, 256, self.hidden_size, device=device)

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=128, bilinear=True):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        self.bilinear = bilinear
        self.n_classes = 1

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(2048, 1024)
        
        self.attn2 = nn.Linear(1024,128)
        
        self.attn_24 = nn.Linear(self.hidden_size*4, self.hidden_size*2)
        
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        
        self.attn_combine_bilstm = nn.Linear(3072, 1024)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(1024, 1024)
        self.out = nn.Linear(1024, 1024)
       # self.hidden = nn.Parameter(torch.randn(4,256,256).cuda()),nn.Parameter(torch.randn(4,256,256).cuda())
       
        self.lsgn_a = nn.GroupNorm(512,1024)
        
        self.lsgn_in = nn.GroupNorm(64,128)
    
        self.down5 = Down(128,256)
        
        factor = 2 if bilinear else 1
                
        self.ups4 = nn.ConvTranspose2d(256 , 256 // 2, kernel_size=2, stride=2)
        self.upsconv4 = DoubleConv(256,128)

        self.lstm = nn.LSTM(256,256,batch_first=False,bidirectional=True,num_layers=1).cuda()
    
    def forward(self, input,hidden,encoder_outputs):
        
        h = torch.unsqueeze(hidden,0)
        
        embedded = input
        
        embedded = self.dropout(embedded)
        
        embedded = self.lsgn_in(embedded)

        hidden_bilstm = h[0]
        
        
        hidden_bilinn =  hidden_bilstm
        
        
        
        hidden_bilinn = self.attn(hidden_bilinn)
        
        #print('hidden bilinn shape:',hidden_bilinn.shape)
    
        hidden_bilinn = self.lsgn_a(hidden_bilinn)
        #print(hidden_bilinn.shape)
        

        attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden_bilinn), 1)), dim=1)
        
        
        
        attn_weights  = self.lsgn_a(attn_weights)
        
        attn_weights  = self.attn2(attn_weights)
        
        attn_weights = self.lsgn_in(attn_weights)
        
        #print(attn_weights.unsqueeze(0).shape,encoder_outputs.unsqueeze(0).shape)
    
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),encoder_outputs.unsqueeze(0))
        
        #print('attn_applied: encoder outputs',attn_applied[0].shape,encoder_outputs[0].shape)

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        #print('The output shape is : ',output.shape)
        
        output = self.attn_combine_bilstm(output).unsqueeze(0)
        
        #print('The output shape after is : ',output.shape)
        
    
        hidden_bi = hidden_bilinn.unsqueeze(0)
        
        output = F.relu(output)
        
        output = self.lsgn_in(output)
        
        #print("output and hidden before lstm ",output.shape,hidden_bi.shape)

        output, hidden = self.gru(output, hidden_bi)
        
        output = F.log_softmax(self.out(output[0]), dim=1)
        output = self.lsgn_a(output)
        

    
        return output,hidden


    def initHidden(self):
        return torch.randn(4, 256, self.hidden_size, device=device)


trainx_l = np.load(basepath+'train_x.npy')/255.0
trainy_l = np.load(basepath+'train_y.npy')
trainy_l[trainy_l > 0] = 1

train_size    = 45
valx          = trainx_l[train_size:]
valy          = trainy_l[train_size:]

trainx_l = trainx_l[:train_size]
trainy_l = trainy_l[:train_size]

testx = np.load(basepath+'test_x.npy')/255.0
testy = np.load(basepath+'test_y.npy')
testy[testy > 0] = 1


trainx_l1 = np.zeros([trainx_l.shape[0], 1, 512, 512], dtype='float16')
valx1     = np.zeros([valx.shape[0], 1, 512, 512],     dtype='float16')
testx1    = np.zeros([testx.shape[0], 1, 512, 512],    dtype='float16')

trainy_l1 = np.zeros([trainy_l.shape[0], 1, 512, 512], dtype='float16')
valy1     = np.zeros([valy.shape[0], 1, 512, 512],     dtype='float16')
testy1    = np.zeros([testy.shape[0], 1, 512, 512],    dtype='float16')


for i in range(trainx_l.shape[0]):
    trainx_l1[i, 0] = scipy.ndimage.zoom(trainx_l[i], 2, order=3)
    trainy_l1[i, 0] = scipy.ndimage.zoom(trainy_l[i], 2, order=0)

for i in range(valx.shape[0]):
    valx1[i, 0] = scipy.ndimage.zoom(valx[i], 2, order=3)
    valy1[i, 0] = scipy.ndimage.zoom(valy[i], 2, order=0)

for i in range(testx.shape[0]):
    testx1[i, 0] = scipy.ndimage.zoom(testx[i], 2, order=3)
    testy1[i, 0] = scipy.ndimage.zoom(testy[i], 2, order=0)


trainx_l = trainx_l1
trainy_l = trainy_l1
valx     = valx1
valy     = valy1
testx    = testx1
testy    = testy1

print(trainx_l.shape, valx.shape, testx.shape)

def init_normal(m):
    if type(m) == nn.Conv2d:
        nn.init.kaiming_normal_(m.weight)
    if type(m) == nn.Linear:
        #nn.init.kaiming_normal_(m.weight)
        nn.init.xavier_uniform_(m.weight)


attn_decoder1 = AttnDecoderRNN(256, 256,dropout_p=0.85)
attn_decoder1.cuda()

attn_decoder2 = AttnDecoderRNN_2(256, 256,dropout_p=0.85)
attn_decoder2.cuda()

model_student = UNetDoubleSmallGroupNormdifferent(1,1)
model_student.cuda()


optimizer_student    = optim.Adam(model_student.parameters(), lr=0.0001,weight_decay=1e-5)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


optimizer_attn_w    = optim.Adam(attn_decoder1.parameters(), lr=0.0001,weight_decay=1e-5)
optimizer_attn_w_2  = optim.Adam(attn_decoder2.parameters(), lr=0.0001,weight_decay=1e-5)


criterion          = nn.BCEWithLogitsLoss(torch.ones([1]).cuda())


val_loss_array   = []
train_loss_array = []


def train_model(model, optimizer, criterion, trainx, trainy, augment=False):
    batch_size = 4
    loss_array = []
   
    model.train()

   
    for i in range(len(trainx)//batch_size):
        x = trainx[i*batch_size:(i+1)*batch_size, 0, :, :]
        y = trainy[i*batch_size:(i+1)*batch_size, 0, :, :]
        
                
        if augment:
            for k in range(x.shape[0]):
                rotv = random.randint(0, 3)
                x[k, 0, :, :] = np.rot90(x[k, 0, :, :], rotv)
                y[k, 0, :, :] = np.rot90(y[k, 0, :, :], rotv)
              
  
        if(x.shape[0]!=4):
            break
            
        x = np.expand_dims(x, 1)
        
        y = np.expand_dims(y, 1)

        y = torch.tensor(y, device=device).float()
        
        x = torch.tensor(x, device=device).float()
        
       
        optimizer.zero_grad()
        optimizer_attn_w.zero_grad()
        optimizer_attn_w_2.zero_grad()
        
        output = model.forward(x)
        loss   = criterion(output , y)
        loss.backward()
           
        loss_array.append(loss.item())

    #    torch.nn.utils.clip_grad_norm(attn_decoder1.parameters(),0.7)
        
        optimizer.step()
        optimizer_attn_w.step()
        
        optimizer_attn_w_2.step()
   
    loss_array = np.mean(loss_array)
    return loss_array


batch_size = 4
teacher_dice_array = []
test_dice_array    = []
total_epochs       = 1000

for epoch in range(total_epochs):
    if epoch%10 ==1:
        print(epoch)
    
    index  = np.random.permutation(trainx.shape[0])
    trainx = trainx[index]
    trainy = trainy[index]
    
    #train_model1(model, optimizer, criterion, trainx, trainy, augment=False)
    train_loss    = train_model(model_student, optimizer_student, criterion, trainx, trainy, False)
    #train_loss    = train_model(model_student, 2, optimizer_student, criterion, trainx, trainy, False)
    
    pred      = get_prediction(model_student, valx)
    val_dice1 = evaluate_result_new(pred, valy)
    #print(pred.shape, len(val_dice1), valy.shape)
    
    pred          = get_prediction(model_student, testx)
    student_dice2 = evaluate_result_new(pred, testy)
    #print(pred.shape, len(student_dice2), testy.shape)
    
    #val_dice      = evaluate_result(model_student, valx,   valy)
    student_dice1 = evaluate_result(model_student, trainx, trainy)
    #student_dice2 = evaluate_result(model_student, testx,  testy)
    
    
    train_dice_array.append(np.mean(student_dice1))
    val_dice_array.append(np.mean(val_dice1))
    test_dice_array.append(np.mean(student_dice2))

    #model_save_name = "ipmi-attentionlstm-covid19"
    model_save_name = "tmi-compare-lstm"
    
    #if np.mean(val_dice1) > prev_max:
    print("Step %d  Val Dice %.5f, Train Dice %f, Test Dice %f" % (epoch, np.mean(val_dice1), np.mean(student_dice1), np.mean(student_dice2)))
    prev_max     = np.mean(val_dice1)
    torch.save(model_student.state_dict(), basepath_models+model_save_name+'-double-studentmodel.pt')
    torch.save(attn_decoder1.state_dict(), basepath_models+model_save_name+'-double-attention.pt')
    #attn_decoder1

In [ ]:
# [STAR] For training different models for comparison on COVID-19 dataset using SU-Net Model

import skimage
import torch.optim as optim
from tqdm import tqdm
import random
from skimage.transform import rotate, AffineTransform, warp
import scipy
from scipy.stats import entropy
import numpy as np

import torch.optim as optim
from tqdm import tqdm
import random
from skimage.transform import rotate, AffineTransform, warp
from scipy.stats import entropy


basepath         = '/home/yu-hao/SEMISUNET/Dataset/'
basepath_models  = '/home/yu-hao/SEMISUNET/Dataset/models/'


def read_training_data(read_ids):
    x_array = []
    y_array = []
    
    for p in read_ids:
        name = basepath+'masks/'
        name = name+'study_'+p+'_mask.nii.gz'
        
        mask = sitk.GetArrayFromImage(sitk.ReadImage(name))
        vol  = sitk.GetArrayFromImage(sitk.ReadImage(name.replace('_mask.nii.gz', '.nii.gz').replace('masks', 'studies/CT-1')))
        
        for t in range(mask.shape[0]):
            temp  = np.count_nonzero(mask[t].flatten())
            if temp > 0:
                x_array.append(np.expand_dims(vol[t], axis=0))
                y_array.append(np.expand_dims(mask[t], axis=0))

    x_array = (np.array(x_array)+1024.0)/1024.0
    y_array = np.array(y_array)
    
    return x_array, y_array

def dice(im1, im2):
    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)
    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)
    return 2. * intersection.sum() / (im1.sum() + im2.sum()+0.00001)

def dice_loss(pred, target, smooth = 1.):
    pred = F.sigmoid(pred)
    
    pred   = pred.contiguous()
    target = target.contiguous()    
    intersection = (pred * target).sum(dim=2).sum(dim=2)
    loss = (1 - ((2. * intersection + smooth) / (pred.sum(dim=2).sum(dim=2) + target.sum(dim=2).sum(dim=2) + smooth)))
    return loss.mean()

def read_training_data_unlabelled(read_ids):
    x_array          = []
    x_array_lungmask = []
    
    names   = [x.split('_')[0] for x in read_ids]
    types   = [x.split('_')[1] for x in read_ids]
    count   = 0
    
    for p in names:
        name     = basepath+'studies/'+types[count]+'/'
        maskname = name+'study_'+p+'_mask.nii.gz'
        volname  = name+'study_'+p+'.nii.gz'
        
        mask = sitk.GetArrayFromImage(sitk.ReadImage(maskname))
        vol  = sitk.GetArrayFromImage(sitk.ReadImage(volname))
        mask[mask > 0] = 1
        
        for t in range(mask.shape[0]):
            if True:#t % 1 == 0:
                temp  = np.count_nonzero(mask[t].flatten())
                if temp > 0: # Check if lung region is present
                    x_array.append(np.expand_dims(vol[t], axis=0))
                    x_array_lungmask.append(np.expand_dims(mask[t], axis=0))
        
        count = count+1

    x_array          = (np.array(x_array)+1024.0)/1024.0
    x_array_lungmask = np.array(x_array_lungmask)
    
    return x_array, x_array_lungmask

def get_prediction(model, valx):
    output_array   = []
    batch_size     = 1
    
    model.eval()
    
    for ik in range(len(valx)//batch_size):
        x = valx[ik*batch_size:(ik+1)*batch_size, :, :, :]
        x = torch.tensor(x, device=device).float()

        output = model.forward(x)
        output = torch.sigmoid(output)
        output = output.data.cpu().numpy()
        #output[output > 0.5]= 1
        #output[output < 0.5]= 0
        
        for k in range(output.shape[0]):
            output_array.append(output[k, 0])
    
    output_array = np.array(output_array)
    output_array = np.expand_dims(output_array, 1)
    
    return output_array

def get_predictions(models, valx):
    output_array   = []
    batch_size     = 1
    
    for i in range(5):
        models[i].eval()
    
    for ik in range(len(valx)//batch_size):
        x = valx[ik*batch_size:(ik+1)*batch_size, :, :, :]
        x = torch.tensor(x, device=device).float()
        
        outputs = []
        for k in range(5):
            output = models[k].forward(x)
            output = torch.sigmoid(output)
            output = output.data.cpu().numpy()
            outputs.append(output)
        
        output_sum = np.zeros(outputs[0].shape, dtype='float16')
        for k in range(5):
            output_sum = output_sum+outputs[k]
        output_sum = output_sum/5.0
        
        for k in range(output.shape[0]):
            output_array.append(output_sum[k, 0])
    
    output_array = np.array(output_array)
    output_array = np.expand_dims(output_array, 1)
    
    return output_array

def get_filtered(valx, valy):
    valxf = []
    valyf = []
    
    for i in range(valx.shape[0]):
        if np.count_nonzero(valy[i]) > 0:
            valxf.append(valx[i])
            valyf.append(valy[i])
    return np.array(valxf), np.array(valyf)

def evaluate_result(model, valx, valy):
    model.eval()
    
    val_dice       = []
    batch_size     = 1
    for ik in range(len(valx)//batch_size):
        x = valx[ik*batch_size:(ik+1)*batch_size, :, :, :]
        y = valy[ik*batch_size:(ik+1)*batch_size, :, :, :]

        x = torch.tensor(x, device=device).float()

        output = model.forward(x)

        output = torch.sigmoid(output)        
        output = output.data.cpu().numpy()

        output[output < 0.5] = 0
        output[output > 0.5] = 1
        
        for pk in range(output.shape[0]):
            dt = dice(y[pk, 0, :, :], output[pk, 0, :, :])
            val_dice.append(dt)
    return val_dice

def evaluate_result_new(pred, valy):
    val_dice       = []
    batch_size     = 1
    
    for ik in range(len(valx)//batch_size):
        output = pred[ik*batch_size:(ik+1)*batch_size, :, :, :]
        y      = valy[ik*batch_size:(ik+1)*batch_size, :, :, :]
        
        output[output < 0.5] = 0
        output[output > 0.5] = 1
        
        for pk in range(output.shape[0]):
            t1 = scipy.ndimage.zoom(output[0, 0].astype('uint8'), 0.6875, order=0)
            t2 = scipy.ndimage.zoom(y[0, 0].astype('uint8'),      0.6875, order=0)
            #print(t1.shape, t2.shape)
            dt = dice(y[pk, 0, :, :], output[pk, 0, :, :])
            val_dice.append(dt)
    
    return val_dice

def train_model(model, batch_size, optimizer, criterion, trainx, trainy, augment=False):
    loss_array = []
    
    idx    = np.random.permutation(trainx.shape[0])
    trainx = trainx[idx]
    trainy = trainy[idx]
    
    model.train()
    
    for i in range(len(trainx)//batch_size):
        x = trainx[i*batch_size:(i+1)*batch_size, :, :, :]
        y = trainy[i*batch_size:(i+1)*batch_size, :, :, :]
        
        if augment:
            for k in range(x.shape[0]):
                rotv = random.randint(0, 3)
                x[k, 0, :, :] = np.rot90(x[k, 0, :, :], rotv)
                y[k, 0, :, :] = np.rot90(y[k, 0, :, :], rotv)
        
        x = torch.tensor(x, device=device).float()
        y = torch.tensor(y, device=device).float()
        
        optimizer.zero_grad()
        output = model.forward(x)        
        loss   = criterion(output , y)
        loss.backward()
        
        loss_array.append(loss.item())
        optimizer.step()
    
    loss_array = np.mean(loss_array)
    return loss_array

def prepare_batch(batch_size, k_means, trainx_l, trainy_l, h):
    a = []
    b = []
    
    for i in range(int(batch_size/2)):
        idx = random.randint(0, trainx_l.shape[0]-1)
        c   = k_means.predict(np.reshape(trainx_l[idx].astype('float32'), [1, 512*512]))[0]
        
        a.append(trainx_l[idx])
        b.append(trainy_l[idx])
        
        idx = random.randint(0, len(h[c])-1)
        t1  = np.expand_dims(np.load(h[c][idx]), 0)
        t2  = np.expand_dims(np.load(h[c][idx].replace('-x', '-y')), 0)
        
        a.append(t1)
        b.append(t2)
   
    a1 = np.array(a).astype('float16')
    b1 = np.array(b).astype('float16')
   
    return a1, b1

def store_cluster_slices(model_teacher, k_means, version):
    epoch_array = np.arange(79)
    all_labels  = []
    step_size   = 10 
    count       = 0
    
    for epoch in epoch_array:
        temp_index               = epoch%(int(len(unlabelled_ids)/step_size))
        trainx, trainx_lungmask  = read_training_data_unlabelled(unlabelled_ids[temp_index*step_size:temp_index*step_size+step_size])
        trainy                   = get_prediction(model_teacher, trainx)
        
        #trainy = np.load('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/PREDICTION-NUMPY/'+str(epoch)+'.npy')
        trainy = np.reshape(trainy, [trainy.shape[0], 512*512])
        #print(epoch, trainy.shape, trainx.shape)
        
        l1     = k_means.predict(trainy)
        
        for jt, t in enumerate(l1):
            temp  = np.reshape(trainy[jt], [512, 512]).astype('float16')
            np.save('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/CLUSTER-NUMPY-'+str(version)+'/'+str(t)+'-'+str(count)+'-y.npy', temp)
            
            temp  = np.reshape(trainx[jt], [512, 512]).astype('float16')
            np.save('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/CLUSTER-NUMPY-'+str(version)+'/'+str(t)+'-'+str(count)+'-x.npy', temp)
            
            count = count+1
    
    return

def prepare_hash(version):
    all_cluster_files = glob.glob('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/CLUSTER-NUMPY-'+str(version)+'/*.npy')
    print('Version ', version, 'File name counts ', len(all_cluster_files))
    filename_hash = {}
    for i in range(50):
        filename_hash[i] = []

    for t in all_cluster_files:
        filename_hash[int(t.split('/')[-1].split('-')[0])].append(t)
    
    return filename_hash

def get_all_covid_lesions(valx, valy, lesion_size):
    lesion_shapes_x = []
    lesion_shapes_y = []
    
    for i in range(valy.shape[0]):
        tx           = valx[i, 0]
        blobs        = valy[i, 0]
        blobs_labels = skimage.measure.label(blobs, background=0)
        propsa       = skimage.measure.regionprops(blobs_labels)
        
        for k in range(len(propsa)):
            temp = (blobs_labels == propsa[k].label).astype('uint8')
            
            temp_size = np.count_nonzero(temp.flatten().astype('uint8'))
            if temp_size < lesion_size and temp_size > 5:
                slice_x, slice_y = ndimage.find_objects(temp == 1)[0]
                
                roi_y = 1-temp[slice_x, slice_y]
                roi_x = tx[slice_x, slice_y]*temp[slice_x, slice_y]
                
                lesion_shapes_x.append(roi_x)
                lesion_shapes_y.append(roi_y)
                
                lesion_shapes_x.append(roi_x.T)
                lesion_shapes_y.append(roi_y.T)
                
                lesion_shapes_x.append(np.rot90(roi_x, 180))
                lesion_shapes_y.append(np.rot90(roi_y, 180))
    
    return lesion_shapes_x, lesion_shapes_y

def get_augmented_slice(batch_size, read_ids, lesion_shapes_x, lesion_shapes_y):
    x_array          = []
    x_array_lungmask = []
    
    index   = random.randint(0, len(read_ids)-1)
    #print(read_ids[index])
    
    p       = read_ids[index].split('_')[0]
    types   = 'CT-1'#read_ids[index].split('_')[1]
    count   = 0
    
    name     = basepath+'studies/'+types+'/'
    maskname = name+'study_'+p+'_mask.nii.gz'
    volname  = name+'study_'+p+'.nii.gz'
    
    segmentation_mask = basepath+'masks/'
    segmentation_mask = segmentation_mask+'study_'+p+'_mask.nii.gz'
    
    mask     = sitk.GetArrayFromImage(sitk.ReadImage(maskname))
    vol      = (sitk.GetArrayFromImage(sitk.ReadImage(volname))+1024.0)/1024.0
    segmentation_mask = sitk.GetArrayFromImage(sitk.ReadImage(segmentation_mask))
    
    mask[mask > 0] = 1
    count          = 0
    
    while(count < batch_size):
        t     = np.random.randint(0, mask.shape[0]-1)
        temp  = np.count_nonzero(mask[t].flatten())
        
        # Check if lung region is present
        if temp > 0:
            st  = vol[t]
            i,j = np.nonzero(mask[t])
            
            index = random.randint(0, len(i)-1)
            
            i = i[index]
            j = j[index]
            
            lesion_index = random.randint(0, len(lesion_shapes_x)-1)
            
            lesion_x     = lesion_shapes_x[lesion_index]
            lesion_y     = lesion_shapes_y[lesion_index]
            
            sx     = int(lesion_x.shape[0]/2)
            sy     = int(lesion_x.shape[1]/2)
            
            if st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy].shape == lesion_x.shape:
                st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  =  lesion_y*st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]
                st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  =  lesion_x + st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]

                m1 = segmentation_mask[t]#np.zeros(st.shape)
                m1[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  += 1-lesion_y
                m1         = m1*mask[t]
                m1[m1 > 0] = 1

                x_array.append(np.expand_dims(st,          axis=0))
                x_array_lungmask.append(np.expand_dims(m1, axis=0))

                count = count+1

    x_array          = np.array(x_array)
    x_array_lungmask = np.array(x_array_lungmask)
    
    return x_array, x_array_lungmask

def get_multiple_augmented_slice(batch_size, read_ids, lesion_shapes_x, lesion_shapes_y):
    x_array          = []
    x_array_lungmask = []
    
    index   = random.randint(0, len(read_ids)-1)
    #print(read_ids[index])
    
    p       = read_ids[index].split('_')[0]
    types   = 'CT-1'#read_ids[index].split('_')[1]
    count   = 0
    
    name     = basepath+'studies/'+types+'/'
    maskname = name+'study_'+p+'_mask.nii.gz'
    volname  = name+'study_'+p+'.nii.gz'
    
    segmentation_mask = basepath+'masks/'
    segmentation_mask = segmentation_mask+'study_'+p+'_mask.nii.gz'
    
    mask     = sitk.GetArrayFromImage(sitk.ReadImage(maskname))
    vol      = (sitk.GetArrayFromImage(sitk.ReadImage(volname))+1024.0)/1024.0
    segmentation_mask = sitk.GetArrayFromImage(sitk.ReadImage(segmentation_mask))
    
    mask[mask > 0] = 1
    count          = 0
    
    while(count < batch_size):
        t     = np.random.randint(0, mask.shape[0]-1)
        temp  = np.count_nonzero(mask[t].flatten())
        
        # Check if lung region is present
        if temp > 0:
            st  = vol[t]
            #segmen
            ipl, jpl = np.nonzero(mask[t])
            
            lesion_count = random.randint(0, 5)
            temp_count   = 0
            
            while(temp_count < lesion_count):
                index = random.randint(0, len(ipl)-1)

                i = ipl[index]
                j = jpl[index]

                lesion_index = random.randint(0, len(lesion_shapes_x)-1)

                lesion_x     = lesion_shapes_x[lesion_index]
                lesion_y     = lesion_shapes_y[lesion_index]

                sx     = int(lesion_x.shape[0]/2)
                sy     = int(lesion_x.shape[1]/2)

                if st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy].shape == lesion_x.shape:
                    st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  =  lesion_y*st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]
                    st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  =  lesion_x + st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]

                    m1 = segmentation_mask[t]#np.zeros(st.shape)
                    m1[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  += 1-lesion_y
                    m1         = m1*mask[t]
                    m1[m1 > 0] = 1
                    segmentation_mask[t] = m1
                    temp_count           = temp_count + 1
            
            x_array.append(np.expand_dims(st,          axis=0))
            x_array_lungmask.append(np.expand_dims(m1, axis=0))
            
            count = count+1

    x_array          = np.array(x_array)
    x_array_lungmask = np.array(x_array_lungmask)
    
    return x_array, x_array_lungmask

def plot_figure_slope(model_save_name):
    N = 2
    a = val_dice_array1#np.convolve(val_dice_array1, np.ones((N,))/N, mode='valid')
    b = train_dice_array1#np.convolve(train_dice_array1, np.ones((N,))/N, mode='valid')
    c = test_dice_array1#np.convolve(test_dice_array1, np.ones((N,))/N, mode='valid')
    
    temp  = 0
    slope = 0
    #np.abs(np.abs(b[i]-b[i-1])-np.abs(a[i]-a[i-1])) < 0.1 and
    for i in range(1, len(a)):
        if b[i] >= b[i-1] and a[i] >= a[i-1]:
            temp  = i#np.argmax(a)
            slope = b[i]-b[i-1]-(a[i]-a[i-1])
            #print(i, slope, np.abs(b[i]-b[i-1]), np.abs(a[i]-a[i-1]), b[i], b[i-1])
    
    import matplotlib.pyplot as plt
    plt.plot(a)
    plt.plot(b)
    plt.plot(c)
    plt.ylabel('some numbers')
    plt.annotate('Index '+str(temp), xy=(0.75, 0.25), xycoords='axes fraction')
    plt.annotate('Train '+str(round(b[temp], 3)), xy=(0.75, 0.20), xycoords='axes fraction')
    plt.annotate('Val   '+str(round(a[temp], 3)), xy=(0.75, 0.15), xycoords='axes fraction')
    plt.annotate('Test  '+str(round(c[temp], 3)), xy=(0.75, 0.10), xycoords='axes fraction')
    plt.annotate('Slope '+str(round(slope, 3)),   xy=(0.75, 0.05), xycoords='axes fraction')
    #plt.text(6, 0, )
    #plt.text(6, 0.1, 'Val   '+str(round(a[temp], 3)))
    #plt.text(6, 0.2, 'Train '+str(round(b[temp], 3)))
    #plt.text(6, 0.3, 'Test  '+str(round(c[temp], 3)))
    
    plt.savefig(model_save_name+".png")
    
    plt.close()
    plt.clf()
    
    return

def sort_data(trainx1, trainy1):
    # Sort the data
    X = trainx1
    Y = trainy1
    r = [t for t in sorted(zip(Y,X), key=lambda pair: np.sum(pair[0].flatten()))]
    
    trainx = []
    trainy = []
    
    for i in range(len(X)):
        trainy.append(r[i][0])
        trainx.append(r[i][1])
    
    trainx = np.array(trainx)
    trainy = np.array(trainy)
    
    return trainx, trainy

def plot_figure(model_save_name):
    a = list(val_dice_array)#np.convolve(val_dice_array1, np.ones((N,))/N, mode='valid')
    b = list(train_dice_array)#np.convolve(train_dice_array1, np.ones((N,))/N, mode='valid')
    c = list(test_dice_array)#np.convolve(test_dice_array1, np.ones((N,))/N, mode='valid')
    
    #a.insert(0, 0)
    #b.insert(0, 0)
    #c.insert(0, 0)
#     temp  = 0
#     slope = 0
#     #np.abs(np.abs(b[i]-b[i-1])-np.abs(a[i]-a[i-1])) < 0.1 and
#     for i in range(1, len(a)):
#         if b[i] >= b[i-1] and a[i] >= a[i-1]:
#             temp  = i#np.argmax(a)
#             slope = b[i]-b[i-1]-(a[i]-a[i-1])
#             #print(i, slope, np.abs(b[i]-b[i-1]), np.abs(a[i]-a[i-1]), b[i], b[i-1])
    
    # Take arg max for semi model
    temp = np.argmax(a)
    
    import matplotlib.pyplot as plt
    plt.plot(a)
    plt.plot(b)
    plt.plot(c)
    plt.ylabel('some numbers')
    plt.annotate('Index '+str(temp), xy=(0.75, 0.25), xycoords='axes fraction')
    plt.annotate('Train '+str(round(b[temp], 3)), xy=(0.75, 0.20), xycoords='axes fraction')
    plt.annotate('Val   '+str(round(a[temp], 3)), xy=(0.75, 0.15), xycoords='axes fraction')
    plt.annotate('Test  '+str(round(c[temp], 3)), xy=(0.75, 0.10), xycoords='axes fraction')
    #plt.annotate('Slope '+str(round(slope, 3)),   xy=(0.75, 0.05), xycoords='axes fraction')
    #plt.text(6, 0, )
    #plt.text(6, 0.1, 'Val   '+str(round(a[temp], 3)))
    #plt.text(6, 0.2, 'Train '+str(round(b[temp], 3)))
    #plt.text(6, 0.3, 'Test  '+str(round(c[temp], 3)))
    
    plt.savefig(model_save_name+".png")
    
    plt.close()
    plt.clf()
    
    return

def train_model1(model, batch_size, optimizer, criterion, trainx, trainy, augment=False):
    loss_array = []
   
    model.train()
    #print(len(trainx)//batch_size)
   
    for i in range(len(trainx)//batch_size):
        x = trainx[i*batch_size:(i+1)*batch_size, 0, :, :]
        y = trainy[i*batch_size:(i+1)*batch_size, 0, :, :]
                
        if augment:
            for k in range(x.shape[0]):
                rotv = random.randint(0, 3)
                x[k, 0, :, :] = np.rot90(x[k, 0, :, :], rotv)
                y[k, 0, :, :] = np.rot90(y[k, 0, :, :], rotv)
       
        #x2 = model.forward(x)        
        #print(x2.shape)
        
#         lstm = nn.LSTM(512*512,512*512,batchfirst=True)
#         hidden = (torch.randn(1, 512, 512), torch.randn(1, 512, 512))
#         outlstm = lstm(x, hidden)
#         n = np.asarray(outlstm)
  
        print(i, x.shape[0])
        
        if(x.shape[0]!= 4):
            break
    
        x = np.expand_dims(x, 1)
        y = np.expand_dims(y, 1)
        y = torch.tensor(y, device=device).float()
        x = torch.tensor(x, device=device).float()
        
       
        optimizer.zero_grad()
        optimizer_attn_w.zero_grad()
        
        output = model.forward(x)
        #print(i,attn_weights[1])
        
        loss   = criterion(output , y)
        loss.backward()
       
        loss_array.append(loss.item())
        
       # torch.nn.utils.clip_grad_norm(attn_decoder1.parameters(),0.7)
        
        optimizer.step()
        optimizer_attn_w.step()
   
    loss_array = np.mean(loss_array)
    return loss_array

def train_model2(model, batch_size, optimizer, criterion, trainx, trainy, augment=False):
    #batch_size = 4
    loss_array = []
   
    model.train()
    #print(len(trainx)//batch_size)
   
    for i in range(len(trainx)//batch_size):
        x = trainx[i*batch_size:(i+1)*batch_size, 0, :, :]
        y = trainy[i*batch_size:(i+1)*batch_size, 0, :, :]
        
                
        if augment:
            for k in range(x.shape[0]):
                rotv = random.randint(0, 3)
                x[k, 0, :, :] = np.rot90(x[k, 0, :, :], rotv)
                y[k, 0, :, :] = np.rot90(y[k, 0, :, :], rotv)
       
  
        if(x.shape[0]!=4):
            break
            
    
        x = np.expand_dims(x, 1)
        

        y = np.expand_dims(y, 1)

        y = torch.tensor(y, device=device).float()
        
        x = torch.tensor(x, device=device).float()
        
       
        optimizer.zero_grad()
        optimizer_attn_w.zero_grad()
        
        output = model.forward(x)
        #print(i,attn_weights[1])
        
        loss   = criterion(output , y)
        loss.backward()
       
        loss_array.append(loss.item())
        
       # torch.nn.utils.clip_grad_norm(attn_decoder1.parameters(),0.7)
        
        optimizer.step()
        optimizer_attn_w.step()
   
    loss_array = np.mean(loss_array)
    return loss_array

device         = torch.device("cuda:0")


# train_ids      = np.load(basepath+'TRAIN.npy')
# val_ids        = np.load(basepath+'VALIDATION.npy')
# test_ids       = np.load(basepath+'TEST.npy')
# unlabelled_ids = np.load(basepath+'NOTLABELLED.npy')
# nocovid_ids    = np.load(basepath+'NOCOVID.npy')


# unlabelled_ids     = unlabelled_ids
# train_ids          = train_ids[:4]
# val_ids            = val_ids
# test_ids           = test_ids

trainx_l = np.load(basepath+'train_x.npy')/255.0
trainy_l = np.load(basepath+'train_y.npy')
trainy_l[trainy_l > 0] = 1

#index    = np.random.permutation(trainx_l.shape[0])
#trainx_l = trainx_l[index]
#trainy_l = trainy_l[index]

train_size    = 45
valx = trainx_l[train_size:]
valy = trainy_l[train_size:]

trainx_l = trainx_l[:train_size]
trainy_l = trainy_l[:train_size]

testx = np.load(basepath+'test_x.npy')/255.0
testy = np.load(basepath+'test_y.npy')
testy[testy > 0] = 1


trainx_l1 = np.zeros([trainx_l.shape[0], 1, 512, 512], dtype='float16')
valx1     = np.zeros([valx.shape[0], 1, 512, 512],     dtype='float16')
testx1    = np.zeros([testx.shape[0], 1, 512, 512],    dtype='float16')

trainy_l1 = np.zeros([trainy_l.shape[0], 1, 512, 512], dtype='float16')
valy1     = np.zeros([valy.shape[0], 1, 512, 512],     dtype='float16')
testy1    = np.zeros([testy.shape[0], 1, 512, 512],    dtype='float16')


for i in range(trainx_l.shape[0]):
    trainx_l1[i, 0] = scipy.ndimage.zoom(trainx_l[i], 2, order=3)
    trainy_l1[i, 0] = scipy.ndimage.zoom(trainy_l[i], 2, order=0)

for i in range(valx.shape[0]):
    valx1[i, 0] = scipy.ndimage.zoom(valx[i], 2, order=3)
    valy1[i, 0] = scipy.ndimage.zoom(valy[i], 2, order=0)

for i in range(testx.shape[0]):
    testx1[i, 0] = scipy.ndimage.zoom(testx[i], 2, order=3)
    testy1[i, 0] = scipy.ndimage.zoom(testy[i], 2, order=0)


trainx_l = trainx_l1
trainy_l = trainy_l1
valx = valx1
valy = valy1
testx = testx1
testy = testy1

print(trainx_l.shape, valx.shape, testx.shape)

def init_normal(m):
    if type(m) == nn.Conv2d:
        nn.init.kaiming_normal_(m.weight)
    if type(m) == nn.Linear:
        #nn.init.kaiming_normal_(m.weight)
        nn.init.xavier_uniform_(m.weight)

prev_max        = -1000
model_student   = SUNet(1, 1)
model_student.cuda()

optimizer_student  = optim.Adam(model_student.parameters(), lr=0.0001)
criterion          = nn.BCEWithLogitsLoss(torch.ones([1]).cuda())

val_dice_array   = []
train_dice_array = []
test_dice_array  = []

trainx, trainy   = trainx_l, trainy_l
trainx, trainy   = sort_data(trainx_l, trainy_l)
total_epochs = 30


# trainx = np.expand_dims(trainx, axis=1)
# trainy = np.expand_dims(trainy, axis=1)

# valx   = np.expand_dims(valx, axis=1)
# valy   = np.expand_dims(valy, axis=1)

# testx  = np.expand_dims(testx, axis=1)
# testy  = np.expand_dims(testy, axis=1)

teacher_dice_array = []
test_dice_array    = []


for epoch in range(total_epochs):
    if epoch%10 ==1:
        print(epoch)
    #train_model1(model, optimizer, criterion, trainx, trainy, augment=False)
    train_loss    = train_model(model_student, 2, optimizer_student, criterion, trainx, trainy, False)
    #train_loss    = train_model(model_student, 2, optimizer_student, criterion, trainx, trainy, False)
    
    pred      = get_prediction(model_student, valx)
    val_dice1 = evaluate_result_new(pred, valy)
    print(pred.shape, len(val_dice1), valy.shape)
    
    pred          = get_prediction(model_student, testx)
    student_dice2 = evaluate_result_new(pred, testy)
    print(pred.shape, len(student_dice2), testy.shape)
    
    #val_dice      = evaluate_result(model_student, valx,   valy)
    student_dice1 = evaluate_result(model_student, trainx, trainy)
    #student_dice2 = evaluate_result(model_student, testx,  testy)
    
    
    train_dice_array.append(np.mean(student_dice1))
    val_dice_array.append(np.mean(val_dice1))
    test_dice_array.append(np.mean(student_dice2))

    model_save_name = "ipmi-sunet-covid19"
    
    if np.mean(val_dice1) > prev_max:
        print("Step %d  Val Dice %.5f, Train Dice %f, Test Dice %f" % (epoch, np.mean(val_dice1), np.mean(student_dice1), np.mean(student_dice2)))
        prev_max     = np.mean(val_dice1)
        torch.save(model_student.state_dict(), basepath_models+model_save_name+'-6.pt')

    #np.save(model_save_name+'_train.npy',      train_dice_array)
    #np.save(model_save_name+'_validation.npy', val_dice_array)

In [ ]:
# [STAR] Semi-supervised training SU-Net Model

import skimage
import torch.optim as optim
from tqdm import tqdm
import random
from skimage.transform import rotate, AffineTransform, warp
import scipy
from scipy.stats import entropy
import numpy as np

import torch.optim as optim
from tqdm import tqdm
import random
from skimage.transform import rotate, AffineTransform, warp
from scipy.stats import entropy


basepath         = '/media/pranjal/2d33dff3-95f7-4dc0-9842-a9b18bcf1bf9/pranjal/COVID19/COVID-SemiSeg/Dataset/'
basepath_models  = '/media/pranjal/2d33dff3-95f7-4dc0-9842-a9b18bcf1bf9/pranjal/COVID19/COVID-SemiSeg/Dataset/models/'


def read_training_data(read_ids):
    x_array = []
    y_array = []
    
    for p in read_ids:
        name = basepath+'masks/'
        name = name+'study_'+p+'_mask.nii.gz'
        
        mask = sitk.GetArrayFromImage(sitk.ReadImage(name))
        vol  = sitk.GetArrayFromImage(sitk.ReadImage(name.replace('_mask.nii.gz', '.nii.gz').replace('masks', 'studies/CT-1')))
        
        for t in range(mask.shape[0]):
            temp  = np.count_nonzero(mask[t].flatten())
            if temp > 0:
                x_array.append(np.expand_dims(vol[t], axis=0))
                y_array.append(np.expand_dims(mask[t], axis=0))

    x_array = (np.array(x_array)+1024.0)/1024.0
    y_array = np.array(y_array)
    
    return x_array, y_array

def dice(im1, im2):
    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)
    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)
    return 2. * intersection.sum() / (im1.sum() + im2.sum()+0.00001)

def dice_loss(pred, target, smooth = 1.):
    pred = F.sigmoid(pred)
    
    pred   = pred.contiguous()
    target = target.contiguous()    
    intersection = (pred * target).sum(dim=2).sum(dim=2)
    loss = (1 - ((2. * intersection + smooth) / (pred.sum(dim=2).sum(dim=2) + target.sum(dim=2).sum(dim=2) + smooth)))
    return loss.mean()

def read_training_data_unlabelled(read_ids):
    x_array          = []
    x_array_lungmask = []
    
    names   = [x.split('_')[0] for x in read_ids]
    types   = [x.split('_')[1] for x in read_ids]
    count   = 0
    
    for p in names:
        name     = basepath+'studies/'+types[count]+'/'
        maskname = name+'study_'+p+'_mask.nii.gz'
        volname  = name+'study_'+p+'.nii.gz'
        
        mask = sitk.GetArrayFromImage(sitk.ReadImage(maskname))
        vol  = sitk.GetArrayFromImage(sitk.ReadImage(volname))
        mask[mask > 0] = 1
        
        for t in range(mask.shape[0]):
            if True:#t % 1 == 0:
                temp  = np.count_nonzero(mask[t].flatten())
                if temp > 0: # Check if lung region is present
                    x_array.append(np.expand_dims(vol[t], axis=0))
                    x_array_lungmask.append(np.expand_dims(mask[t], axis=0))
        
        count = count+1

    x_array          = (np.array(x_array)+1024.0)/1024.0
    x_array_lungmask = np.array(x_array_lungmask)
    
    return x_array, x_array_lungmask

def get_prediction(model, datax):
    output_array   = []
    batch_size     = 1
    
    model.eval()
    
    for ik in range(len(datax)//batch_size):
        x = datax[ik*batch_size:(ik+1)*batch_size, :, :, :]
        x = torch.tensor(x, device=device).float()

        output = model.forward(x)
        output = torch.sigmoid(output)
        output = output.data.cpu().numpy()
        #output[output > 0.5]= 1
        #output[output < 0.5]= 0
        
        for k in range(output.shape[0]):
            output_array.append(output[k, 0])
    
    output_array = np.array(output_array)
    output_array = np.expand_dims(output_array, 1)
    
    return output_array

def get_predictions(models, valx):
    output_array   = []
    batch_size     = 1
    
    for i in range(5):
        models[i].eval()
    
    for ik in range(len(valx)//batch_size):
        x = valx[ik*batch_size:(ik+1)*batch_size, :, :, :]
        x = torch.tensor(x, device=device).float()
        
        outputs = []
        for k in range(5):
            output = models[k].forward(x)
            output = torch.sigmoid(output)
            output = output.data.cpu().numpy()
            outputs.append(output)
        
        output_sum = np.zeros(outputs[0].shape, dtype='float16')
        for k in range(5):
            output_sum = output_sum+outputs[k]
        output_sum = output_sum/5.0
        
        for k in range(output.shape[0]):
            output_array.append(output_sum[k, 0])
    
    output_array = np.array(output_array)
    output_array = np.expand_dims(output_array, 1)
    
    return output_array

def get_filtered(valx, valy):
    valxf = []
    valyf = []
    
    for i in range(valx.shape[0]):
        if np.count_nonzero(valy[i]) > 0:
            valxf.append(valx[i])
            valyf.append(valy[i])
    return np.array(valxf), np.array(valyf)

def evaluate_result(model, valx, valy):
    model.eval()
    
    val_dice       = []
    batch_size     = 1
    for ik in range(len(valx)//batch_size):
        x = valx[ik*batch_size:(ik+1)*batch_size, :, :, :]
        y = valy[ik*batch_size:(ik+1)*batch_size, :, :, :]

        x = torch.tensor(x, device=device).float()

        output = model.forward(x)

        output = torch.sigmoid(output)        
        output = output.data.cpu().numpy()

        output[output < 0.5] = 0
        output[output > 0.5] = 1
        
        for pk in range(output.shape[0]):
            dt = dice(y[pk, 0, :, :], output[pk, 0, :, :])
            val_dice.append(dt)
    return val_dice

def train_model(model, batch_size, optimizer, criterion, trainx, trainy, augment=False):
    loss_array = []
    
    idx    = np.random.permutation(trainx.shape[0])
    trainx = trainx[idx]
    trainy = trainy[idx]
    
    model.train()
    
    for i in range(len(trainx)//batch_size):
        x = trainx[i*batch_size:(i+1)*batch_size, :, :, :]
        y = trainy[i*batch_size:(i+1)*batch_size, :, :, :]
        
        if augment:
            for k in range(x.shape[0]):
                rotv = random.randint(0, 3)
                x[k, 0, :, :] = np.rot90(x[k, 0, :, :], rotv)
                y[k, 0, :, :] = np.rot90(y[k, 0, :, :], rotv)
        
        x = torch.tensor(x, device=device).float()
        y = torch.tensor(y, device=device).float()
        
        optimizer.zero_grad()
        output = model.forward(x)        
        loss   = criterion(output , y)
        loss.backward()
        
        loss_array.append(loss.item())
        optimizer.step()
    
    loss_array = np.mean(loss_array)
    return loss_array

def prepare_batch(batch_size, k_means, trainx_l, trainy_l, h):
    a = []
    b = []
    
    for i in range(int(batch_size/2)):
        idx = random.randint(0, trainx_l.shape[0]-1)
        c   = k_means.predict(np.reshape(trainx_l[idx].astype('float32'), [1, 512*512]))[0]
        
        a.append(trainx_l[idx])
        b.append(trainy_l[idx])
        
        idx = random.randint(0, len(h[c])-1)
        t1  = np.expand_dims(np.load(h[c][idx]), 0)
        t2  = np.expand_dims(np.load(h[c][idx].replace('-x', '-y')), 0)
        
        a.append(t1)
        b.append(t2)
   
    a1 = np.array(a).astype('float16')
    b1 = np.array(b).astype('float16')
   
    return a1, b1

def store_cluster_slices(model_teacher, k_means, version):
    epoch_array = np.arange(79)
    all_labels  = []
    step_size   = 10 
    count       = 0
    
    for epoch in epoch_array:
        temp_index               = epoch%(int(len(unlabelled_ids)/step_size))
        trainx, trainx_lungmask  = read_training_data_unlabelled(unlabelled_ids[temp_index*step_size:temp_index*step_size+step_size])
        trainy                   = get_prediction(model_teacher, trainx)
        
        #trainy = np.load('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/PREDICTION-NUMPY/'+str(epoch)+'.npy')
        trainy = np.reshape(trainy, [trainy.shape[0], 512*512])
        #print(epoch, trainy.shape, trainx.shape)
        
        l1     = k_means.predict(trainy)
        
        for jt, t in enumerate(l1):
            temp  = np.reshape(trainy[jt], [512, 512]).astype('float16')
            np.save('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/CLUSTER-NUMPY-'+str(version)+'/'+str(t)+'-'+str(count)+'-y.npy', temp)
            
            temp  = np.reshape(trainx[jt], [512, 512]).astype('float16')
            np.save('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/CLUSTER-NUMPY-'+str(version)+'/'+str(t)+'-'+str(count)+'-x.npy', temp)
            
            count = count+1
    
    return

def prepare_hash(version):
    all_cluster_files = glob.glob('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/CLUSTER-NUMPY-'+str(version)+'/*.npy')
    print('Version ', version, 'File name counts ', len(all_cluster_files))
    filename_hash = {}
    for i in range(50):
        filename_hash[i] = []

    for t in all_cluster_files:
        filename_hash[int(t.split('/')[-1].split('-')[0])].append(t)
    
    return filename_hash

def get_all_covid_lesions(valx, valy, lesion_size):
    lesion_shapes_x = []
    lesion_shapes_y = []
    
    for i in range(valy.shape[0]):
        tx           = valx[i, 0]
        blobs        = valy[i, 0]
        blobs_labels = skimage.measure.label(blobs, background=0)
        propsa       = skimage.measure.regionprops(blobs_labels)
        
        for k in range(len(propsa)):
            temp = (blobs_labels == propsa[k].label).astype('uint8')
            
            temp_size = np.count_nonzero(temp.flatten().astype('uint8'))
            if temp_size < lesion_size and temp_size > 5:
                slice_x, slice_y = ndimage.find_objects(temp == 1)[0]
                
                roi_y = 1-temp[slice_x, slice_y]
                roi_x = tx[slice_x, slice_y]*temp[slice_x, slice_y]
                
                lesion_shapes_x.append(roi_x)
                lesion_shapes_y.append(roi_y)
                
                lesion_shapes_x.append(roi_x.T)
                lesion_shapes_y.append(roi_y.T)
                
                lesion_shapes_x.append(np.rot90(roi_x, 180))
                lesion_shapes_y.append(np.rot90(roi_y, 180))
    
    return lesion_shapes_x, lesion_shapes_y

def get_augmented_slice(batch_size, read_ids, lesion_shapes_x, lesion_shapes_y):
    x_array          = []
    x_array_lungmask = []
    
    index   = random.randint(0, len(read_ids)-1)
    #print(read_ids[index])
    
    p       = read_ids[index].split('_')[0]
    types   = 'CT-1'#read_ids[index].split('_')[1]
    count   = 0
    
    name     = basepath+'studies/'+types+'/'
    maskname = name+'study_'+p+'_mask.nii.gz'
    volname  = name+'study_'+p+'.nii.gz'
    
    segmentation_mask = basepath+'masks/'
    segmentation_mask = segmentation_mask+'study_'+p+'_mask.nii.gz'
    
    mask     = sitk.GetArrayFromImage(sitk.ReadImage(maskname))
    vol      = (sitk.GetArrayFromImage(sitk.ReadImage(volname))+1024.0)/1024.0
    segmentation_mask = sitk.GetArrayFromImage(sitk.ReadImage(segmentation_mask))
    
    mask[mask > 0] = 1
    count          = 0
    
    while(count < batch_size):
        t     = np.random.randint(0, mask.shape[0]-1)
        temp  = np.count_nonzero(mask[t].flatten())
        
        # Check if lung region is present
        if temp > 0:
            st  = vol[t]
            i,j = np.nonzero(mask[t])
            
            index = random.randint(0, len(i)-1)
            
            i = i[index]
            j = j[index]
            
            lesion_index = random.randint(0, len(lesion_shapes_x)-1)
            
            lesion_x     = lesion_shapes_x[lesion_index]
            lesion_y     = lesion_shapes_y[lesion_index]
            
            sx     = int(lesion_x.shape[0]/2)
            sy     = int(lesion_x.shape[1]/2)
            
            if st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy].shape == lesion_x.shape:
                st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  =  lesion_y*st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]
                st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  =  lesion_x + st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]

                m1 = segmentation_mask[t]#np.zeros(st.shape)
                m1[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  += 1-lesion_y
                m1         = m1*mask[t]
                m1[m1 > 0] = 1

                x_array.append(np.expand_dims(st,          axis=0))
                x_array_lungmask.append(np.expand_dims(m1, axis=0))

                count = count+1

    x_array          = np.array(x_array)
    x_array_lungmask = np.array(x_array_lungmask)
    
    return x_array, x_array_lungmask

def get_multiple_augmented_slice(batch_size, read_ids, lesion_shapes_x, lesion_shapes_y):
    x_array          = []
    x_array_lungmask = []
    
    index   = random.randint(0, len(read_ids)-1)
    #print(read_ids[index])
    
    p       = read_ids[index].split('_')[0]
    types   = 'CT-1'#read_ids[index].split('_')[1]
    count   = 0
    
    name     = basepath+'studies/'+types+'/'
    maskname = name+'study_'+p+'_mask.nii.gz'
    volname  = name+'study_'+p+'.nii.gz'
    
    segmentation_mask = basepath+'masks/'
    segmentation_mask = segmentation_mask+'study_'+p+'_mask.nii.gz'
    
    mask     = sitk.GetArrayFromImage(sitk.ReadImage(maskname))
    vol      = (sitk.GetArrayFromImage(sitk.ReadImage(volname))+1024.0)/1024.0
    segmentation_mask = sitk.GetArrayFromImage(sitk.ReadImage(segmentation_mask))
    
    mask[mask > 0] = 1
    count          = 0
    
    while(count < batch_size):
        t     = np.random.randint(0, mask.shape[0]-1)
        temp  = np.count_nonzero(mask[t].flatten())
        
        # Check if lung region is present
        if temp > 0:
            st  = vol[t]
            #segmen
            ipl, jpl = np.nonzero(mask[t])
            
            lesion_count = random.randint(0, 5)
            temp_count   = 0
            
            while(temp_count < lesion_count):
                index = random.randint(0, len(ipl)-1)

                i = ipl[index]
                j = jpl[index]

                lesion_index = random.randint(0, len(lesion_shapes_x)-1)

                lesion_x     = lesion_shapes_x[lesion_index]
                lesion_y     = lesion_shapes_y[lesion_index]

                sx     = int(lesion_x.shape[0]/2)
                sy     = int(lesion_x.shape[1]/2)

                if st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy].shape == lesion_x.shape:
                    st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  =  lesion_y*st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]
                    st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  =  lesion_x + st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]

                    m1 = segmentation_mask[t]#np.zeros(st.shape)
                    m1[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  += 1-lesion_y
                    m1         = m1*mask[t]
                    m1[m1 > 0] = 1
                    segmentation_mask[t] = m1
                    temp_count           = temp_count + 1
            
            x_array.append(np.expand_dims(st,          axis=0))
            x_array_lungmask.append(np.expand_dims(m1, axis=0))
            
            count = count+1

    x_array          = np.array(x_array)
    x_array_lungmask = np.array(x_array_lungmask)
    
    return x_array, x_array_lungmask

def plot_figure_slope(model_save_name):
    N = 2
    a = val_dice_array1#np.convolve(val_dice_array1, np.ones((N,))/N, mode='valid')
    b = train_dice_array1#np.convolve(train_dice_array1, np.ones((N,))/N, mode='valid')
    c = test_dice_array1#np.convolve(test_dice_array1, np.ones((N,))/N, mode='valid')
    
    temp  = 0
    slope = 0
    #np.abs(np.abs(b[i]-b[i-1])-np.abs(a[i]-a[i-1])) < 0.1 and
    for i in range(1, len(a)):
        if b[i] >= b[i-1] and a[i] >= a[i-1]:
            temp  = i#np.argmax(a)
            slope = b[i]-b[i-1]-(a[i]-a[i-1])
            #print(i, slope, np.abs(b[i]-b[i-1]), np.abs(a[i]-a[i-1]), b[i], b[i-1])
    
    import matplotlib.pyplot as plt
    plt.plot(a)
    plt.plot(b)
    plt.plot(c)
    plt.ylabel('some numbers')
    plt.annotate('Index '+str(temp), xy=(0.75, 0.25), xycoords='axes fraction')
    plt.annotate('Train '+str(round(b[temp], 3)), xy=(0.75, 0.20), xycoords='axes fraction')
    plt.annotate('Val   '+str(round(a[temp], 3)), xy=(0.75, 0.15), xycoords='axes fraction')
    plt.annotate('Test  '+str(round(c[temp], 3)), xy=(0.75, 0.10), xycoords='axes fraction')
    plt.annotate('Slope '+str(round(slope, 3)),   xy=(0.75, 0.05), xycoords='axes fraction')
    #plt.text(6, 0, )
    #plt.text(6, 0.1, 'Val   '+str(round(a[temp], 3)))
    #plt.text(6, 0.2, 'Train '+str(round(b[temp], 3)))
    #plt.text(6, 0.3, 'Test  '+str(round(c[temp], 3)))
    
    plt.savefig(model_save_name+".png")
    
    plt.close()
    plt.clf()
    
    return

def sort_data(trainx1, trainy1):
    # Sort the data
    X = trainx1
    Y = trainy1
    r = [t for t in sorted(zip(Y,X), key=lambda pair: np.sum(pair[0].flatten()))]
    
    trainx = []
    trainy = []
    
    for i in range(len(X)):
        trainy.append(r[i][0])
        trainx.append(r[i][1])
    
    trainx = np.array(trainx)
    trainy = np.array(trainy)
    
    return trainx, trainy

def plot_figure(model_save_name):
    a = list(val_dice_array)#np.convolve(val_dice_array1, np.ones((N,))/N, mode='valid')
    b = list(train_dice_array)#np.convolve(train_dice_array1, np.ones((N,))/N, mode='valid')
    c = list(test_dice_array)#np.convolve(test_dice_array1, np.ones((N,))/N, mode='valid')
    
    #a.insert(0, 0)
    #b.insert(0, 0)
    #c.insert(0, 0)
#     temp  = 0
#     slope = 0
#     #np.abs(np.abs(b[i]-b[i-1])-np.abs(a[i]-a[i-1])) < 0.1 and
#     for i in range(1, len(a)):
#         if b[i] >= b[i-1] and a[i] >= a[i-1]:
#             temp  = i#np.argmax(a)
#             slope = b[i]-b[i-1]-(a[i]-a[i-1])
#             #print(i, slope, np.abs(b[i]-b[i-1]), np.abs(a[i]-a[i-1]), b[i], b[i-1])
    
    # Take arg max for semi model
    temp = np.argmax(a)
    
    import matplotlib.pyplot as plt
    plt.plot(a)
    plt.plot(b)
    plt.plot(c)
    plt.ylabel('some numbers')
    plt.annotate('Index '+str(temp), xy=(0.75, 0.25), xycoords='axes fraction')
    plt.annotate('Train '+str(round(b[temp], 3)), xy=(0.75, 0.20), xycoords='axes fraction')
    plt.annotate('Val   '+str(round(a[temp], 3)), xy=(0.75, 0.15), xycoords='axes fraction')
    plt.annotate('Test  '+str(round(c[temp], 3)), xy=(0.75, 0.10), xycoords='axes fraction')
    #plt.annotate('Slope '+str(round(slope, 3)),   xy=(0.75, 0.05), xycoords='axes fraction')
    #plt.text(6, 0, )
    #plt.text(6, 0.1, 'Val   '+str(round(a[temp], 3)))
    #plt.text(6, 0.2, 'Train '+str(round(b[temp], 3)))
    #plt.text(6, 0.3, 'Test  '+str(round(c[temp], 3)))
    
    plt.savefig(model_save_name+".png")
    
    plt.close()
    plt.clf()
    
    return

def train_model1(model, batch_size, optimizer, criterion, trainx, trainy, augment=False):
    loss_array = []
   
    model.train()
    #print(len(trainx)//batch_size)
   
    for i in range(len(trainx)//batch_size):
        x = trainx[i*batch_size:(i+1)*batch_size, 0, :, :]
        y = trainy[i*batch_size:(i+1)*batch_size, 0, :, :]
                
        if augment:
            for k in range(x.shape[0]):
                rotv = random.randint(0, 3)
                x[k, 0, :, :] = np.rot90(x[k, 0, :, :], rotv)
                y[k, 0, :, :] = np.rot90(y[k, 0, :, :], rotv)
       
        #x2 = model.forward(x)        
        #print(x2.shape)
        
#         lstm = nn.LSTM(512*512,512*512,batchfirst=True)
#         hidden = (torch.randn(1, 512, 512), torch.randn(1, 512, 512))
#         outlstm = lstm(x, hidden)
#         n = np.asarray(outlstm)
  
        print(i, x.shape[0])
        
        if(x.shape[0]!= 4):
            break
    
        x = np.expand_dims(x, 1)
        y = np.expand_dims(y, 1)
        y = torch.tensor(y, device=device).float()
        x = torch.tensor(x, device=device).float()
        
       
        optimizer.zero_grad()
        optimizer_attn_w.zero_grad()
        
        output = model.forward(x)
        #print(i,attn_weights[1])
        
        loss   = criterion(output , y)
        loss.backward()
       
        loss_array.append(loss.item())
        
       # torch.nn.utils.clip_grad_norm(attn_decoder1.parameters(),0.7)
        
        optimizer.step()
        optimizer_attn_w.step()
   
    loss_array = np.mean(loss_array)
    return loss_array

def train_model2(model, batch_size, optimizer, criterion, trainx, trainy, augment=False):
    #batch_size = 4
    loss_array = []
   
    model.train()
    #print(len(trainx)//batch_size)
   
    for i in range(len(trainx)//batch_size):
        x = trainx[i*batch_size:(i+1)*batch_size, 0, :, :]
        y = trainy[i*batch_size:(i+1)*batch_size, 0, :, :]
        
                
        if augment:
            for k in range(x.shape[0]):
                rotv = random.randint(0, 3)
                x[k, 0, :, :] = np.rot90(x[k, 0, :, :], rotv)
                y[k, 0, :, :] = np.rot90(y[k, 0, :, :], rotv)
       
  
        if(x.shape[0]!=4):
            break
            
    
        x = np.expand_dims(x, 1)
        

        y = np.expand_dims(y, 1)

        y = torch.tensor(y, device=device).float()
        
        x = torch.tensor(x, device=device).float()
        
       
        optimizer.zero_grad()
        optimizer_attn_w.zero_grad()
        
        output = model.forward(x)
        #print(i,attn_weights[1])
        
        loss   = criterion(output , y)
        loss.backward()
       
        loss_array.append(loss.item())
        
       # torch.nn.utils.clip_grad_norm(attn_decoder1.parameters(),0.7)
        
        optimizer.step()
        optimizer_attn_w.step()
   
    loss_array = np.mean(loss_array)
    return loss_array

device         = torch.device("cuda:0")


# train_ids      = np.load(basepath+'TRAIN.npy')
# val_ids        = np.load(basepath+'VALIDATION.npy')
# test_ids       = np.load(basepath+'TEST.npy')
# unlabelled_ids = np.load(basepath+'NOTLABELLED.npy')
# nocovid_ids    = np.load(basepath+'NOCOVID.npy')


# unlabelled_ids     = unlabelled_ids
# train_ids          = train_ids[:4]
# val_ids            = val_ids
# test_ids           = test_ids

trainx_l = np.load(basepath+'train_x.npy')/255.0
trainy_l = np.load(basepath+'train_y.npy')
trainy_l[trainy_l > 0] = 1

train_size    = 45
valx = trainx_l[train_size:]
valy = trainy_l[train_size:]

trainx_l = trainx_l[:train_size]
trainy_l = trainy_l[:train_size]

testx = np.load(basepath+'test_x.npy')/255.0
testy = np.load(basepath+'test_y.npy')
testy[testy > 0] = 1

unlabelledx_l = np.load(basepath+'unlabelled_x.npy')/255.0


trainx_l1 = np.zeros([trainx_l.shape[0], 1, 512, 512], dtype='float16')
valx1     = np.zeros([valx.shape[0], 1, 512, 512],     dtype='float16')
testx1    = np.zeros([testx.shape[0], 1, 512, 512],    dtype='float16')
unlabelledx1    = np.zeros([unlabelledx_l.shape[0], 1, 512, 512],    dtype='float16')

trainy_l1 = np.zeros([trainy_l.shape[0], 1, 512, 512], dtype='float16')
valy1     = np.zeros([valy.shape[0], 1, 512, 512],     dtype='float16')
testy1    = np.zeros([testy.shape[0], 1, 512, 512],    dtype='float16')

for i in range(trainx_l.shape[0]):
    trainx_l1[i, 0] = scipy.ndimage.zoom(trainx_l[i], 2, order=3)
    trainy_l1[i, 0] = scipy.ndimage.zoom(trainy_l[i], 2, order=0)

for i in range(valx.shape[0]):
    valx1[i, 0] = scipy.ndimage.zoom(valx[i], 2, order=3)
    valy1[i, 0] = scipy.ndimage.zoom(valy[i], 2, order=0)

for i in range(testx.shape[0]):
    testx1[i, 0] = scipy.ndimage.zoom(testx[i], 2, order=3)
    testy1[i, 0] = scipy.ndimage.zoom(testy[i], 2, order=0)

for i in range(unlabelledx1.shape[0]):
    unlabelledx1[i, 0] = scipy.ndimage.zoom(unlabelledx_l[i], 2, order=3)
    #testy1[i, 0] = scipy.ndimage.zoom(unlabelledy1[i], 2, order=0)

model_student   = SUNet(1, 1)
model_student.cuda()
p1         = torch.load(basepath_models+"tmi-compare-sunet-covid19-30.pt")
model_student.load_state_dict(p1)


unlabelledy1 = get_prediction(model_student, unlabelledx1)

trainx_l = trainx_l1
trainy_l = trainy_l1
valx = valx1
valy = valy1
testx = testx1
testy = testy1

print(trainx_l.shape, valx.shape, testx.shape, unlabelledy1.shape)

def init_normal(m):
    if type(m) == nn.Conv2d:
        nn.init.kaiming_normal_(m.weight)
    if type(m) == nn.Linear:
        #nn.init.kaiming_normal_(m.weight)
        nn.init.xavier_uniform_(m.weight)

prev_max        = -1000
model_student   = SUNet(1, 1)
model_student.cuda()
p1         = torch.load(basepath_models+"tmi-compare-sunet-covid19-30.pt")
model_student.load_state_dict(p1)

optimizer_student  = optim.Adam(model_student.parameters(), lr=0.0001)
criterion          = nn.BCEWithLogitsLoss(torch.ones([1]).cuda())
#criterion          = nn.MSELoss()

val_dice_array   = []
train_dice_array = []
test_dice_array  = []

#trainy[trainy > 0.5] = 1
#trainy[trainy < 0.5] = 0
total_epochs     = 1000
#trainx, trainy   = sort_data(trainx, trainy)

trainx = trainx#[800:]
trainy = trainy#[800:]
# trainx = np.expand_dims(trainx, axis=1)
# trainy = np.expand_dims(trainy, axis=1)

# valx   = np.expand_dims(valx, axis=1)
# valy   = np.expand_dims(valy, axis=1)

# testx  = np.expand_dims(testx, axis=1)
# testy  = np.expand_dims(testy, axis=1)

teacher_dice_array = []
test_dice_array    = []

index        = np.random.permutation(np.arange(len(unlabelledx1)))
unlabelledx1 = unlabelledx1[index]
unlabelledy1 = unlabelledy1[index]

for epoch in range(total_epochs):
    if epoch%10 ==1:
        print(epoch)
    
    # Re-generate pseudo labels again
    if epoch%5 == 0:
        step_size        = len(unlabelledx1)//20-1
        inputx           = unlabelledx1#[0:(1+epoch%20)*step_size]
        unlabelledy1     = get_prediction(model_student, inputx)
        #unlabelledy1[unlabelledy1 < 0.5] = 0
        #unlabelledy1[unlabelledy1 > 0.5] = 1
        trainx_l1        = inputx#np.concatenate([trainx_l, inputx], axis=0)
        trainy_l1        = unlabelledy1#np.concatenate([trainy_l, unlabelledy1], axis=0)
        trainx, trainy   = sort_data(trainx_l1, trainy_l1)
    
    #train_model1(model, optimizer, criterion, trainx, trainy, augment=False)
    train_loss    = train_model(model_student, 4, optimizer_student, criterion, trainx, trainy, False)
    #train_loss    = train_model(model_student, 2, optimizer_student, criterion, trainx, trainy, False)
    
    val_dice      = evaluate_result(model_student, valx,   valy)
    student_dice1 = evaluate_result(model_student, trainx, trainy)
    student_dice2 = evaluate_result(model_student, testx,  testy)
    
    
    train_dice_array.append(np.mean(student_dice1))
    val_dice_array.append(np.mean(val_dice))
    test_dice_array.append(np.mean(student_dice2))
    
    
    model_save_name = "tmi-compare-sunet-covid19-semi"
    
    #if np.mean(val_dice) > prev_max:
    print("Step %d  Val Dice %.5f, Train Dice %f, Test Dice %f" % (epoch, np.mean(val_dice), np.mean(student_dice1), np.mean(student_dice2)))
    prev_max     = np.mean(val_dice)
    torch.save(model_student.state_dict(), basepath_models+model_save_name+'-'+str(epoch)+".pt")

    #np.save(model_save_name+'_train.npy',      train_dice_array)
    #np.save(model_save_name+'_validation.npy', val_dice_array)

In [ ]:
# [STAR] Semi-supervised training LSTM Model

import skimage
import torch.optim as optim
from tqdm import tqdm
import random
from skimage.transform import rotate, AffineTransform, warp
import scipy
from scipy.stats import entropy
import numpy as np

import torch.optim as optim
from tqdm import tqdm
import random
from skimage.transform import rotate, AffineTransform, warp
from scipy.stats import entropy


basepath         = '/home/yu-hao/SEMISUNET/Dataset/'
basepath_models  = '/home/yu-hao/SEMISUNET/Dataset/models/'

def read_training_data(read_ids):
    x_array = []
    y_array = []
    
    for p in read_ids:
        name = basepath+'masks/'
        name = name+'study_'+p+'_mask.nii.gz'
        
        mask = sitk.GetArrayFromImage(sitk.ReadImage(name))
        vol  = sitk.GetArrayFromImage(sitk.ReadImage(name.replace('_mask.nii.gz', '.nii.gz').replace('masks', 'studies/CT-1')))
        
        for t in range(mask.shape[0]):
            temp  = np.count_nonzero(mask[t].flatten())
            if temp > 0:
                x_array.append(np.expand_dims(vol[t], axis=0))
                y_array.append(np.expand_dims(mask[t], axis=0))

    x_array = (np.array(x_array)+1024.0)/1024.0
    y_array = np.array(y_array)
    
    return x_array, y_array

def dice(im1, im2):
    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)
    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)
    return 2. * intersection.sum() / (im1.sum() + im2.sum()+0.00001)

def dice_loss(pred, target, smooth = 1.):
    pred = F.sigmoid(pred)
    
    pred   = pred.contiguous()
    target = target.contiguous()    
    intersection = (pred * target).sum(dim=2).sum(dim=2)
    loss = (1 - ((2. * intersection + smooth) / (pred.sum(dim=2).sum(dim=2) + target.sum(dim=2).sum(dim=2) + smooth)))
    return loss.mean()

def read_training_data_unlabelled(read_ids):
    x_array          = []
    x_array_lungmask = []
    
    names   = [x.split('_')[0] for x in read_ids]
    types   = [x.split('_')[1] for x in read_ids]
    count   = 0
    
    for p in names:
        name     = basepath+'studies/'+types[count]+'/'
        maskname = name+'study_'+p+'_mask.nii.gz'
        volname  = name+'study_'+p+'.nii.gz'
        
        mask = sitk.GetArrayFromImage(sitk.ReadImage(maskname))
        vol  = sitk.GetArrayFromImage(sitk.ReadImage(volname))
        mask[mask > 0] = 1
        
        for t in range(mask.shape[0]):
            if True:#t % 1 == 0:
                temp  = np.count_nonzero(mask[t].flatten())
                if temp > 0: # Check if lung region is present
                    x_array.append(np.expand_dims(vol[t], axis=0))
                    x_array_lungmask.append(np.expand_dims(mask[t], axis=0))
        
        count = count+1

    x_array          = (np.array(x_array)+1024.0)/1024.0
    x_array_lungmask = np.array(x_array_lungmask)
    
    return x_array, x_array_lungmask

def get_prediction(model, datax):
    output_array   = []
    batch_size     = 4
    
    model.eval()
    
    for ik in range(len(datax)//batch_size):
        x = datax[ik*batch_size:(ik+1)*batch_size, :, :, :]
        x = torch.tensor(x, device=device).float()

        output = model.forward(x)
        output = torch.sigmoid(output)
        output = output.data.cpu().numpy()
        #output[output > 0.5]= 1
        #output[output < 0.5]= 0
        
        for k in range(output.shape[0]):
            output_array.append(output[k, 0])
    
    output_array = np.array(output_array)
    output_array = np.expand_dims(output_array, 1)
    
    return output_array

def get_predictions(models, valx):
    output_array   = []
    batch_size     = 1
    
    for i in range(5):
        models[i].eval()
    
    for ik in range(len(valx)//batch_size):
        x = valx[ik*batch_size:(ik+1)*batch_size, :, :, :]
        x = torch.tensor(x, device=device).float()
        
        outputs = []
        for k in range(5):
            output = models[k].forward(x)
            output = torch.sigmoid(output)
            output = output.data.cpu().numpy()
            outputs.append(output)
        
        output_sum = np.zeros(outputs[0].shape, dtype='float16')
        for k in range(5):
            output_sum = output_sum+outputs[k]
        output_sum = output_sum/5.0
        
        for k in range(output.shape[0]):
            output_array.append(output_sum[k, 0])
    
    output_array = np.array(output_array)
    output_array = np.expand_dims(output_array, 1)
    
    return output_array

def get_filtered(valx, valy):
    valxf = []
    valyf = []
    
    for i in range(valx.shape[0]):
        if np.count_nonzero(valy[i]) > 0:
            valxf.append(valx[i])
            valyf.append(valy[i])
    return np.array(valxf), np.array(valyf)

def evaluate_result(model, valx, valy):
    model.eval()
    
    val_dice       = []
    batch_size     = 4
    for ik in range(len(valx)//batch_size):
        x = valx[ik*batch_size:(ik+1)*batch_size, :, :, :]
        y = valy[ik*batch_size:(ik+1)*batch_size, :, :, :]

        x = torch.tensor(x, device=device).float()

        output = model.forward(x)

        output = torch.sigmoid(output)        
        output = output.data.cpu().numpy()

        output[output < 0.5] = 0
        output[output > 0.5] = 1
        
        for pk in range(output.shape[0]):
            dt = dice(y[pk, 0, :, :], output[pk, 0, :, :])
            val_dice.append(dt)
    return val_dice

def evaluate_result_new(pred, valy):
    val_dice       = []
    batch_size     = 4
    
    for ik in range(len(valx)//batch_size):
        output = pred[ik*batch_size:(ik+1)*batch_size, :, :, :]
        y      = valy[ik*batch_size:(ik+1)*batch_size, :, :, :]
        
        output[output < 0.5] = 0
        output[output > 0.5] = 1
        
        for pk in range(output.shape[0]):
            t1 = scipy.ndimage.zoom(output[0, 0].astype('uint8'), 0.6875, order=0)
            t2 = scipy.ndimage.zoom(y[0, 0].astype('uint8'),      0.6875, order=0)
            #print(t1.shape, t2.shape)
            dt = dice(y[pk, 0, :, :], output[pk, 0, :, :])
            val_dice.append(dt)
    
    return val_dice

def train_model(model, batch_size, optimizer, criterion, trainx, trainy, augment=False):
    loss_array = []
    
    idx    = np.random.permutation(trainx.shape[0])
    trainx = trainx[idx]
    trainy = trainy[idx]
    
    model.train()
    
    for i in range(len(trainx)//batch_size):
        x = trainx[i*batch_size:(i+1)*batch_size, :, :, :]
        y = trainy[i*batch_size:(i+1)*batch_size, :, :, :]
        
        if augment:
            for k in range(x.shape[0]):
                rotv = random.randint(0, 3)
                x[k, 0, :, :] = np.rot90(x[k, 0, :, :], rotv)
                y[k, 0, :, :] = np.rot90(y[k, 0, :, :], rotv)
        
        x = torch.tensor(x, device=device).float()
        y = torch.tensor(y, device=device).float()
        
        optimizer.zero_grad()
        output = model.forward(x)        
        loss   = criterion(output , y)
        loss.backward()
        
        loss_array.append(loss.item())
        optimizer.step()
    
    loss_array = np.mean(loss_array)
    return loss_array

def prepare_batch(batch_size, k_means, trainx_l, trainy_l, h):
    a = []
    b = []
    
    for i in range(int(batch_size/2)):
        idx = random.randint(0, trainx_l.shape[0]-1)
        c   = k_means.predict(np.reshape(trainx_l[idx].astype('float32'), [1, 512*512]))[0]
        
        a.append(trainx_l[idx])
        b.append(trainy_l[idx])
        
        idx = random.randint(0, len(h[c])-1)
        t1  = np.expand_dims(np.load(h[c][idx]), 0)
        t2  = np.expand_dims(np.load(h[c][idx].replace('-x', '-y')), 0)
        
        a.append(t1)
        b.append(t2)
   
    a1 = np.array(a).astype('float16')
    b1 = np.array(b).astype('float16')
   
    return a1, b1

def store_cluster_slices(model_teacher, k_means, version):
    epoch_array = np.arange(79)
    all_labels  = []
    step_size   = 10 
    count       = 0
    
    for epoch in epoch_array:
        temp_index               = epoch%(int(len(unlabelled_ids)/step_size))
        trainx, trainx_lungmask  = read_training_data_unlabelled(unlabelled_ids[temp_index*step_size:temp_index*step_size+step_size])
        trainy                   = get_prediction(model_teacher, trainx)
        
        #trainy = np.load('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/PREDICTION-NUMPY/'+str(epoch)+'.npy')
        trainy = np.reshape(trainy, [trainy.shape[0], 512*512])
        #print(epoch, trainy.shape, trainx.shape)
        
        l1     = k_means.predict(trainy)
        
        for jt, t in enumerate(l1):
            temp  = np.reshape(trainy[jt], [512, 512]).astype('float16')
            np.save('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/CLUSTER-NUMPY-'+str(version)+'/'+str(t)+'-'+str(count)+'-y.npy', temp)
            
            temp  = np.reshape(trainx[jt], [512, 512]).astype('float16')
            np.save('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/CLUSTER-NUMPY-'+str(version)+'/'+str(t)+'-'+str(count)+'-x.npy', temp)
            
            count = count+1
    
    return

def prepare_hash(version):
    all_cluster_files = glob.glob('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/CLUSTER-NUMPY-'+str(version)+'/*.npy')
    print('Version ', version, 'File name counts ', len(all_cluster_files))
    filename_hash = {}
    for i in range(50):
        filename_hash[i] = []

    for t in all_cluster_files:
        filename_hash[int(t.split('/')[-1].split('-')[0])].append(t)
    
    return filename_hash

def get_all_covid_lesions(valx, valy, lesion_size):
    lesion_shapes_x = []
    lesion_shapes_y = []
    
    for i in range(valy.shape[0]):
        tx           = valx[i, 0]
        blobs        = valy[i, 0]
        blobs_labels = skimage.measure.label(blobs, background=0)
        propsa       = skimage.measure.regionprops(blobs_labels)
        
        for k in range(len(propsa)):
            temp = (blobs_labels == propsa[k].label).astype('uint8')
            
            temp_size = np.count_nonzero(temp.flatten().astype('uint8'))
            if temp_size < lesion_size and temp_size > 5:
                slice_x, slice_y = ndimage.find_objects(temp == 1)[0]
                
                roi_y = 1-temp[slice_x, slice_y]
                roi_x = tx[slice_x, slice_y]*temp[slice_x, slice_y]
                
                lesion_shapes_x.append(roi_x)
                lesion_shapes_y.append(roi_y)
                
                lesion_shapes_x.append(roi_x.T)
                lesion_shapes_y.append(roi_y.T)
                
                lesion_shapes_x.append(np.rot90(roi_x, 180))
                lesion_shapes_y.append(np.rot90(roi_y, 180))
    
    return lesion_shapes_x, lesion_shapes_y

def get_augmented_slice(batch_size, read_ids, lesion_shapes_x, lesion_shapes_y):
    x_array          = []
    x_array_lungmask = []
    
    index   = random.randint(0, len(read_ids)-1)
    #print(read_ids[index])
    
    p       = read_ids[index].split('_')[0]
    types   = 'CT-1'#read_ids[index].split('_')[1]
    count   = 0
    
    name     = basepath+'studies/'+types+'/'
    maskname = name+'study_'+p+'_mask.nii.gz'
    volname  = name+'study_'+p+'.nii.gz'
    
    segmentation_mask = basepath+'masks/'
    segmentation_mask = segmentation_mask+'study_'+p+'_mask.nii.gz'
    
    mask     = sitk.GetArrayFromImage(sitk.ReadImage(maskname))
    vol      = (sitk.GetArrayFromImage(sitk.ReadImage(volname))+1024.0)/1024.0
    segmentation_mask = sitk.GetArrayFromImage(sitk.ReadImage(segmentation_mask))
    
    mask[mask > 0] = 1
    count          = 0
    
    while(count < batch_size):
        t     = np.random.randint(0, mask.shape[0]-1)
        temp  = np.count_nonzero(mask[t].flatten())
        
        # Check if lung region is present
        if temp > 0:
            st  = vol[t]
            i,j = np.nonzero(mask[t])
            
            index = random.randint(0, len(i)-1)
            
            i = i[index]
            j = j[index]
            
            lesion_index = random.randint(0, len(lesion_shapes_x)-1)
            
            lesion_x     = lesion_shapes_x[lesion_index]
            lesion_y     = lesion_shapes_y[lesion_index]
            
            sx     = int(lesion_x.shape[0]/2)
            sy     = int(lesion_x.shape[1]/2)
            
            if st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy].shape == lesion_x.shape:
                st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  =  lesion_y*st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]
                st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  =  lesion_x + st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]

                m1 = segmentation_mask[t]#np.zeros(st.shape)
                m1[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  += 1-lesion_y
                m1         = m1*mask[t]
                m1[m1 > 0] = 1

                x_array.append(np.expand_dims(st,          axis=0))
                x_array_lungmask.append(np.expand_dims(m1, axis=0))

                count = count+1

    x_array          = np.array(x_array)
    x_array_lungmask = np.array(x_array_lungmask)
    
    return x_array, x_array_lungmask

def get_multiple_augmented_slice(batch_size, read_ids, lesion_shapes_x, lesion_shapes_y):
    x_array          = []
    x_array_lungmask = []
    
    index   = random.randint(0, len(read_ids)-1)
    #print(read_ids[index])
    
    p       = read_ids[index].split('_')[0]
    types   = 'CT-1'#read_ids[index].split('_')[1]
    count   = 0
    
    name     = basepath+'studies/'+types+'/'
    maskname = name+'study_'+p+'_mask.nii.gz'
    volname  = name+'study_'+p+'.nii.gz'
    
    segmentation_mask = basepath+'masks/'
    segmentation_mask = segmentation_mask+'study_'+p+'_mask.nii.gz'
    
    mask     = sitk.GetArrayFromImage(sitk.ReadImage(maskname))
    vol      = (sitk.GetArrayFromImage(sitk.ReadImage(volname))+1024.0)/1024.0
    segmentation_mask = sitk.GetArrayFromImage(sitk.ReadImage(segmentation_mask))
    
    mask[mask > 0] = 1
    count          = 0
    
    while(count < batch_size):
        t     = np.random.randint(0, mask.shape[0]-1)
        temp  = np.count_nonzero(mask[t].flatten())
        
        # Check if lung region is present
        if temp > 0:
            st  = vol[t]
            #segmen
            ipl, jpl = np.nonzero(mask[t])
            
            lesion_count = random.randint(0, 5)
            temp_count   = 0
            
            while(temp_count < lesion_count):
                index = random.randint(0, len(ipl)-1)

                i = ipl[index]
                j = jpl[index]

                lesion_index = random.randint(0, len(lesion_shapes_x)-1)

                lesion_x     = lesion_shapes_x[lesion_index]
                lesion_y     = lesion_shapes_y[lesion_index]

                sx     = int(lesion_x.shape[0]/2)
                sy     = int(lesion_x.shape[1]/2)

                if st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy].shape == lesion_x.shape:
                    st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  =  lesion_y*st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]
                    st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  =  lesion_x + st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]

                    m1 = segmentation_mask[t]#np.zeros(st.shape)
                    m1[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  += 1-lesion_y
                    m1         = m1*mask[t]
                    m1[m1 > 0] = 1
                    segmentation_mask[t] = m1
                    temp_count           = temp_count + 1
            
            x_array.append(np.expand_dims(st,          axis=0))
            x_array_lungmask.append(np.expand_dims(m1, axis=0))
            
            count = count+1

    x_array          = np.array(x_array)
    x_array_lungmask = np.array(x_array_lungmask)
    
    return x_array, x_array_lungmask

def plot_figure_slope(model_save_name):
    N = 2
    a = val_dice_array1#np.convolve(val_dice_array1, np.ones((N,))/N, mode='valid')
    b = train_dice_array1#np.convolve(train_dice_array1, np.ones((N,))/N, mode='valid')
    c = test_dice_array1#np.convolve(test_dice_array1, np.ones((N,))/N, mode='valid')
    
    temp  = 0
    slope = 0
    #np.abs(np.abs(b[i]-b[i-1])-np.abs(a[i]-a[i-1])) < 0.1 and
    for i in range(1, len(a)):
        if b[i] >= b[i-1] and a[i] >= a[i-1]:
            temp  = i#np.argmax(a)
            slope = b[i]-b[i-1]-(a[i]-a[i-1])
            #print(i, slope, np.abs(b[i]-b[i-1]), np.abs(a[i]-a[i-1]), b[i], b[i-1])
    
    import matplotlib.pyplot as plt
    plt.plot(a)
    plt.plot(b)
    plt.plot(c)
    plt.ylabel('some numbers')
    plt.annotate('Index '+str(temp), xy=(0.75, 0.25), xycoords='axes fraction')
    plt.annotate('Train '+str(round(b[temp], 3)), xy=(0.75, 0.20), xycoords='axes fraction')
    plt.annotate('Val   '+str(round(a[temp], 3)), xy=(0.75, 0.15), xycoords='axes fraction')
    plt.annotate('Test  '+str(round(c[temp], 3)), xy=(0.75, 0.10), xycoords='axes fraction')
    plt.annotate('Slope '+str(round(slope, 3)),   xy=(0.75, 0.05), xycoords='axes fraction')
    #plt.text(6, 0, )
    #plt.text(6, 0.1, 'Val   '+str(round(a[temp], 3)))
    #plt.text(6, 0.2, 'Train '+str(round(b[temp], 3)))
    #plt.text(6, 0.3, 'Test  '+str(round(c[temp], 3)))
    
    plt.savefig(model_save_name+".png")
    
    plt.close()
    plt.clf()
    
    return

def sort_data(trainx1, trainy1):
    # Sort the data
    X = trainx1
    Y = trainy1
    r = [t for t in sorted(zip(Y,X), key=lambda pair: np.sum(pair[0].flatten()))]
    
    trainx = []
    trainy = []
    
    for i in range(len(X)):
        trainy.append(r[i][0])
        trainx.append(r[i][1])
    
    trainx = np.array(trainx)
    trainy = np.array(trainy)
    
    return trainx, trainy

def plot_figure(model_save_name):
    a = list(val_dice_array)#np.convolve(val_dice_array1, np.ones((N,))/N, mode='valid')
    b = list(train_dice_array)#np.convolve(train_dice_array1, np.ones((N,))/N, mode='valid')
    c = list(test_dice_array)#np.convolve(test_dice_array1, np.ones((N,))/N, mode='valid')
    
    #a.insert(0, 0)
    #b.insert(0, 0)
    #c.insert(0, 0)
#     temp  = 0
#     slope = 0
#     #np.abs(np.abs(b[i]-b[i-1])-np.abs(a[i]-a[i-1])) < 0.1 and
#     for i in range(1, len(a)):
#         if b[i] >= b[i-1] and a[i] >= a[i-1]:
#             temp  = i#np.argmax(a)
#             slope = b[i]-b[i-1]-(a[i]-a[i-1])
#             #print(i, slope, np.abs(b[i]-b[i-1]), np.abs(a[i]-a[i-1]), b[i], b[i-1])
    
    # Take arg max for semi model
    temp = np.argmax(a)
    
    import matplotlib.pyplot as plt
    plt.plot(a)
    plt.plot(b)
    plt.plot(c)
    plt.ylabel('some numbers')
    plt.annotate('Index '+str(temp), xy=(0.75, 0.25), xycoords='axes fraction')
    plt.annotate('Train '+str(round(b[temp], 3)), xy=(0.75, 0.20), xycoords='axes fraction')
    plt.annotate('Val   '+str(round(a[temp], 3)), xy=(0.75, 0.15), xycoords='axes fraction')
    plt.annotate('Test  '+str(round(c[temp], 3)), xy=(0.75, 0.10), xycoords='axes fraction')
    #plt.annotate('Slope '+str(round(slope, 3)),   xy=(0.75, 0.05), xycoords='axes fraction')
    #plt.text(6, 0, )
    #plt.text(6, 0.1, 'Val   '+str(round(a[temp], 3)))
    #plt.text(6, 0.2, 'Train '+str(round(b[temp], 3)))
    #plt.text(6, 0.3, 'Test  '+str(round(c[temp], 3)))
    
    plt.savefig(model_save_name+".png")
    
    plt.close()
    plt.clf()
    
    return

def train_model1(model, batch_size, optimizer, criterion, trainx, trainy, augment=False):
    loss_array = []
   
    model.train()
    #print(len(trainx)//batch_size)
   
    for i in range(len(trainx)//batch_size):
        x = trainx[i*batch_size:(i+1)*batch_size, 0, :, :]
        y = trainy[i*batch_size:(i+1)*batch_size, 0, :, :]
                
        if augment:
            for k in range(x.shape[0]):
                rotv = random.randint(0, 3)
                x[k, 0, :, :] = np.rot90(x[k, 0, :, :], rotv)
                y[k, 0, :, :] = np.rot90(y[k, 0, :, :], rotv)
       
        #x2 = model.forward(x)        
        #print(x2.shape)
        
#         lstm = nn.LSTM(512*512,512*512,batchfirst=True)
#         hidden = (torch.randn(1, 512, 512), torch.randn(1, 512, 512))
#         outlstm = lstm(x, hidden)
#         n = np.asarray(outlstm)
  
        print(i, x.shape[0])
        
        if(x.shape[0]!= 4):
            break
    
        x = np.expand_dims(x, 1)
        y = np.expand_dims(y, 1)
        y = torch.tensor(y, device=device).float()
        x = torch.tensor(x, device=device).float()
        
       
        optimizer.zero_grad()
        optimizer_attn_w.zero_grad()
        
        output = model.forward(x)
        #print(i,attn_weights[1])
        
        loss   = criterion(output , y)
        loss.backward()
       
        loss_array.append(loss.item())
        
       # torch.nn.utils.clip_grad_norm(attn_decoder1.parameters(),0.7)
        
        optimizer.step()
        optimizer_attn_w.step()
   
    loss_array = np.mean(loss_array)
    return loss_array

def train_model2(model, batch_size, optimizer, criterion, trainx, trainy, augment=False):
    #batch_size = 4
    loss_array = []
   
    model.train()
    #print(len(trainx)//batch_size)
   
    for i in range(len(trainx)//batch_size):
        x = trainx[i*batch_size:(i+1)*batch_size, 0, :, :]
        y = trainy[i*batch_size:(i+1)*batch_size, 0, :, :]
        
                
        if augment:
            for k in range(x.shape[0]):
                rotv = random.randint(0, 3)
                x[k, 0, :, :] = np.rot90(x[k, 0, :, :], rotv)
                y[k, 0, :, :] = np.rot90(y[k, 0, :, :], rotv)
       
  
        if(x.shape[0]!=4):
            break
            
    
        x = np.expand_dims(x, 1)
        

        y = np.expand_dims(y, 1)

        y = torch.tensor(y, device=device).float()
        
        x = torch.tensor(x, device=device).float()
        
       
        optimizer.zero_grad()
        optimizer_attn_w.zero_grad()
        
        output = model.forward(x)
        #print(i,attn_weights[1])
        
        loss   = criterion(output , y)
        loss.backward()
       
        loss_array.append(loss.item())
        
       # torch.nn.utils.clip_grad_norm(attn_decoder1.parameters(),0.7)
        
        optimizer.step()
        optimizer_attn_w.step()
   
    loss_array = np.mean(loss_array)
    return loss_array

device         = torch.device("cuda:0")


# train_ids      = np.load(basepath+'TRAIN.npy')
# val_ids        = np.load(basepath+'VALIDATION.npy')
# test_ids       = np.load(basepath+'TEST.npy')
# unlabelled_ids = np.load(basepath+'NOTLABELLED.npy')
# nocovid_ids    = np.load(basepath+'NOCOVID.npy')


# unlabelled_ids     = unlabelled_ids
# train_ids          = train_ids[:4]
# val_ids            = val_ids
# test_ids           = test_ids

trainx_l = np.load(basepath+'train_x.npy')/255.0
trainy_l = np.load(basepath+'train_y.npy')
trainy_l[trainy_l > 0] = 1

train_size    = 45
valx = trainx_l[train_size:]
valy = trainy_l[train_size:]

trainx_l = trainx_l[:train_size]
trainy_l = trainy_l[:train_size]

testx = np.load(basepath+'test_x.npy')/255.0
testy = np.load(basepath+'test_y.npy')
testy[testy > 0] = 1

unlabelledx_l = np.load(basepath+'unlabelled_x.npy')/255.0


trainx_l1 = np.zeros([trainx_l.shape[0], 1, 512, 512], dtype='float16')
valx1     = np.zeros([valx.shape[0], 1, 512, 512],     dtype='float16')
testx1    = np.zeros([testx.shape[0], 1, 512, 512],    dtype='float16')
unlabelledx1    = np.zeros([unlabelledx_l.shape[0], 1, 512, 512],    dtype='float16')

trainy_l1 = np.zeros([trainy_l.shape[0], 1, 512, 512], dtype='float16')
valy1     = np.zeros([valy.shape[0], 1, 512, 512],     dtype='float16')
testy1    = np.zeros([testy.shape[0], 1, 512, 512],    dtype='float16')

for i in range(trainx_l.shape[0]):
    trainx_l1[i, 0] = scipy.ndimage.zoom(trainx_l[i], 2, order=3)
    trainy_l1[i, 0] = scipy.ndimage.zoom(trainy_l[i], 2, order=0)

for i in range(valx.shape[0]):
    valx1[i, 0] = scipy.ndimage.zoom(valx[i], 2, order=3)
    valy1[i, 0] = scipy.ndimage.zoom(valy[i], 2, order=0)

for i in range(testx.shape[0]):
    testx1[i, 0] = scipy.ndimage.zoom(testx[i], 2, order=3)
    testy1[i, 0] = scipy.ndimage.zoom(testy[i], 2, order=0)

for i in range(unlabelledx1.shape[0]):
    unlabelledx1[i, 0] = scipy.ndimage.zoom(unlabelledx_l[i], 2, order=3)
    #testy1[i, 0] = scipy.ndimage.zoom(unlabelledy1[i], 2, order=0)

model_save_name = "tmi-compare-lstm"

attn_decoder1 = AttnDecoderRNN(256, 256, dropout_p=0.45)
attn_decoder1.cuda()
p1         = torch.load(basepath_models+model_save_name+'-attention.pt')
attn_decoder1.load_state_dict(p1)

prev_max        = -1000
model_teacher   = UNetDoubleSmallGroupNormdifferent(1, 1)
model_teacher.cuda()
p1         = torch.load(basepath_models+model_save_name+'-studentmodel.pt')
model_teacher.load_state_dict(p1)


model_student   = UNetDoubleSmallGroupNormdifferent(1, 1)
model_student.cuda()

#model_student   = SUNet(1, 1)
#model_student.cuda()
#p1         = torch.load(basepath_models+model_save_name+'-studentmodel.pt')
#model_student.load_state_dict(p1)


unlabelledy1 = get_prediction(model_student, unlabelledx1)

trainx_l = trainx_l1
trainy_l = trainy_l1
valx = valx1
valy = valy1
testx = testx1
testy = testy1

print(trainx_l.shape, valx.shape, testx.shape, unlabelledy1.shape)

def init_normal(m):
    if type(m) == nn.Conv2d:
        nn.init.kaiming_normal_(m.weight)
    if type(m) == nn.Linear:
        #nn.init.kaiming_normal_(m.weight)
        nn.init.xavier_uniform_(m.weight)

prev_max        = -1000
#model_student   = SUNet(1, 1)
#model_student.cuda()
#p1         = torch.load(basepath_models+"tmi-compare-sunet-covid19-30.pt")
#model_student.load_state_dict(p1)

optimizer_student  = optim.Adam(model_student.parameters(), lr=0.00001)
criterion          = nn.BCEWithLogitsLoss(torch.ones([1]).cuda())
#criterion          = nn.MSELoss()

optimizer_attn_w  = optim.Adam(attn_decoder1.parameters(), lr=0.00001, weight_decay=1e-5)

val_dice_array   = []
train_dice_array = []
test_dice_array  = []

#trainy[trainy > 0.5] = 1
#trainy[trainy < 0.5] = 0
total_epochs     = 1000
#trainx, trainy   = sort_data(trainx, trainy)

trainx = trainx#[800:]
trainy = trainy#[800:]
# trainx = np.expand_dims(trainx, axis=1)
# trainy = np.expand_dims(trainy, axis=1)

# valx   = np.expand_dims(valx, axis=1)
# valy   = np.expand_dims(valy, axis=1)

# testx  = np.expand_dims(testx, axis=1)
# testy  = np.expand_dims(testy, axis=1)

teacher_dice_array = []
test_dice_array    = []

index        = np.random.permutation(np.arange(len(unlabelledx1)))
unlabelledx1 = unlabelledx1[index]
unlabelledy1 = unlabelledy1[index]

pred          = get_prediction(model_teacher, testx)
student_dice2 = evaluate_result_new(pred, testy)

print('Student Dice in beginning is ', np.mean(student_dice2))
    
for epoch in range(total_epochs):
    if epoch%10 ==1:
        print(epoch)
    
    # Re-generate pseudo labels again
    #if epoch%5 == 0:
    step_size        = len(unlabelledx1)//20-1
    inputx           = unlabelledx1#[0:(1+epoch%20)*step_size]
    unlabelledy1     = get_prediction(model_teacher, inputx)
    unlabelledy1[unlabelledy1 < 0.5] = 0
    unlabelledy1[unlabelledy1 > 0.5] = 1
    trainx_l1        = inputx#np.concatenate([trainx_l, inputx], axis=0)
    trainy_l1        = unlabelledy1#np.concatenate([trainy_l, unlabelledy1], axis=0)
    index = np.random.permutation(np.arange(len(trainx_l1)))
    trainx_l1 = trainx_l1[index]
    trainy_l1 = trainy_l1[index]
    #trainx, trainy   = sort_data(trainx_l1, trainy_l1)
    
    #train_model1(model, optimizer, criterion, trainx, trainy, augment=False)
    train_loss    = train_model2(model_student, 4, optimizer_student, criterion, trainx, trainy, False)
    #train_loss    = train_model(model_student, 2, optimizer_student, criterion, trainx, trainy, False)
    
    val_dice      = evaluate_result(model_student, valx,   valy)
    student_dice1 = evaluate_result(model_student, trainx, trainy)
    
    pred          = get_prediction(model_student, testx)
    student_dice2 = evaluate_result_new(pred, testy)
    
    print('Student length is ', len(student_dice2))
    train_dice_array.append(np.mean(student_dice1))
    val_dice_array.append(np.mean(val_dice))
    test_dice_array.append(np.mean(student_dice2))
    
    
    model_save_name = "tmi-compare-lstm-semi"
    
    if np.mean(val_dice) > prev_max:
        print("Step %d  Val Dice %.5f, Train Dice %f, Test Dice %f" % (epoch, np.mean(val_dice), np.mean(student_dice1), np.mean(student_dice2)))
        prev_max     = np.mean(val_dice)
        torch.save(model_student.state_dict(), basepath_models+model_save_name+'-modelstudent.pt')
        torch.save(attn_decoder1.state_dict(), basepath_models+model_save_name+'-attention.pt')
    
    #np.save(model_save_name+'_train.npy',      train_dice_array)
    #np.save(model_save_name+'_validation.npy', val_dice_array)

In [ ]:
# [STAR] For training different models for comparison on COVID-19 dataset using SU-Net  Model

import skimage
import torch.optim as optim
from tqdm import tqdm
import random
from skimage.transform import rotate, AffineTransform, warp
import scipy
from scipy.stats import entropy
import numpy as np

import torch.optim as optim
from tqdm import tqdm
import random
from skimage.transform import rotate, AffineTransform, warp
from scipy.stats import entropy


basepath         = '/media/pranjal/2d33dff3-95f7-4dc0-9842-a9b18bcf1bf9/pranjal/COVID19/COVID-SemiSeg/Dataset/'
basepath_models  = '/media/pranjal/2d33dff3-95f7-4dc0-9842-a9b18bcf1bf9/pranjal/COVID19/COVID-SemiSeg/Dataset/models/'


def read_training_data(read_ids):
    x_array = []
    y_array = []
    
    for p in read_ids:
        name = basepath+'masks/'
        name = name+'study_'+p+'_mask.nii.gz'
        
        mask = sitk.GetArrayFromImage(sitk.ReadImage(name))
        vol  = sitk.GetArrayFromImage(sitk.ReadImage(name.replace('_mask.nii.gz', '.nii.gz').replace('masks', 'studies/CT-1')))
        
        for t in range(mask.shape[0]):
            temp  = np.count_nonzero(mask[t].flatten())
            if temp > 0:
                x_array.append(np.expand_dims(vol[t], axis=0))
                y_array.append(np.expand_dims(mask[t], axis=0))

    x_array = (np.array(x_array)+1024.0)/1024.0
    y_array = np.array(y_array)
    
    return x_array, y_array

def dice(im1, im2):
    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)
    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)
    return 2. * intersection.sum() / (im1.sum() + im2.sum()+0.00001)

def dice_loss(pred, target, smooth = 1.):
    pred = F.sigmoid(pred)
    
    pred   = pred.contiguous()
    target = target.contiguous()    
    intersection = (pred * target).sum(dim=2).sum(dim=2)
    loss = (1 - ((2. * intersection + smooth) / (pred.sum(dim=2).sum(dim=2) + target.sum(dim=2).sum(dim=2) + smooth)))
    return loss.mean()

def read_training_data_unlabelled(read_ids):
    x_array          = []
    x_array_lungmask = []
    
    names   = [x.split('_')[0] for x in read_ids]
    types   = [x.split('_')[1] for x in read_ids]
    count   = 0
    
    for p in names:
        name     = basepath+'studies/'+types[count]+'/'
        maskname = name+'study_'+p+'_mask.nii.gz'
        volname  = name+'study_'+p+'.nii.gz'
        
        mask = sitk.GetArrayFromImage(sitk.ReadImage(maskname))
        vol  = sitk.GetArrayFromImage(sitk.ReadImage(volname))
        mask[mask > 0] = 1
        
        for t in range(mask.shape[0]):
            if True:#t % 1 == 0:
                temp  = np.count_nonzero(mask[t].flatten())
                if temp > 0: # Check if lung region is present
                    x_array.append(np.expand_dims(vol[t], axis=0))
                    x_array_lungmask.append(np.expand_dims(mask[t], axis=0))
        
        count = count+1

    x_array          = (np.array(x_array)+1024.0)/1024.0
    x_array_lungmask = np.array(x_array_lungmask)
    
    return x_array, x_array_lungmask

def get_prediction(model, valx):
    output_array   = []
    batch_size     = 1
    
    model.eval()
    
    for ik in range(len(valx)//batch_size):
        x = valx[ik*batch_size:(ik+1)*batch_size, :, :, :]
        x = torch.tensor(x, device=device).float()

        output = model.forward(x)
        output = torch.sigmoid(output)
        output = output.data.cpu().numpy()
        #output[output > 0.5]= 1
        #output[output < 0.5]= 0
        
        for k in range(output.shape[0]):
            output_array.append(output[k, 0])
    
    output_array = np.array(output_array)
    output_array = np.expand_dims(output_array, 1)
    
    return output_array

def get_predictions(models, valx):
    output_array   = []
    batch_size     = 1
    
    for i in range(5):
        models[i].eval()
    
    for ik in range(len(valx)//batch_size):
        x = valx[ik*batch_size:(ik+1)*batch_size, :, :, :]
        x = torch.tensor(x, device=device).float()
        
        outputs = []
        for k in range(5):
            output = models[k].forward(x)
            output = torch.sigmoid(output)
            output = output.data.cpu().numpy()
            outputs.append(output)
        
        output_sum = np.zeros(outputs[0].shape, dtype='float16')
        for k in range(5):
            output_sum = output_sum+outputs[k]
        output_sum = output_sum/5.0
        
        for k in range(output.shape[0]):
            output_array.append(output_sum[k, 0])
    
    output_array = np.array(output_array)
    output_array = np.expand_dims(output_array, 1)
    
    return output_array

def get_filtered(valx, valy):
    valxf = []
    valyf = []
    
    for i in range(valx.shape[0]):
        if np.count_nonzero(valy[i]) > 0:
            valxf.append(valx[i])
            valyf.append(valy[i])
    return np.array(valxf), np.array(valyf)

def evaluate_result(model, valx, valy):
    model.eval()
    
    val_dice       = []
    batch_size     = 4
    for ik in range(len(valx)//batch_size):
        x = valx[ik*batch_size:(ik+1)*batch_size, :, :, :]
        y = valy[ik*batch_size:(ik+1)*batch_size, :, :, :]

        x = torch.tensor(x, device=device).float()

        output = model.forward(x)

        output = torch.sigmoid(output)        
        output = output.data.cpu().numpy()

        output[output < 0.5] = 0
        output[output > 0.5] = 1
        
        for pk in range(output.shape[0]):
            dt = dice(y[pk, 0, :, :], output[pk, 0, :, :])
            val_dice.append(dt)
    return val_dice

def train_model(model, batch_size, optimizer, criterion, trainx, trainy, augment=False):
    loss_array = []
    
    idx    = np.random.permutation(trainx.shape[0])
    trainx = trainx[idx]
    trainy = trainy[idx]
    
    model.train()
    
    for i in range(len(trainx)//batch_size):
        x = trainx[i*batch_size:(i+1)*batch_size, :, :, :]
        y = trainy[i*batch_size:(i+1)*batch_size, :, :, :]
        
        if augment:
            for k in range(x.shape[0]):
                rotv = random.randint(0, 3)
                x[k, 0, :, :] = np.rot90(x[k, 0, :, :], rotv)
                y[k, 0, :, :] = np.rot90(y[k, 0, :, :], rotv)
        
        x = torch.tensor(x, device=device).float()
        y = torch.tensor(y, device=device).float()
        
        optimizer.zero_grad()
        output = model.forward(x)        
        loss   = criterion(output , y)
        loss.backward()
        
        loss_array.append(loss.item())
        optimizer.step()
    
    loss_array = np.mean(loss_array)
    return loss_array

def prepare_batch(batch_size, k_means, trainx_l, trainy_l, h):
    a = []
    b = []
    
    for i in range(int(batch_size/2)):
        idx = random.randint(0, trainx_l.shape[0]-1)
        c   = k_means.predict(np.reshape(trainx_l[idx].astype('float32'), [1, 512*512]))[0]
        
        a.append(trainx_l[idx])
        b.append(trainy_l[idx])
        
        idx = random.randint(0, len(h[c])-1)
        t1  = np.expand_dims(np.load(h[c][idx]), 0)
        t2  = np.expand_dims(np.load(h[c][idx].replace('-x', '-y')), 0)
        
        a.append(t1)
        b.append(t2)
   
    a1 = np.array(a).astype('float16')
    b1 = np.array(b).astype('float16')
   
    return a1, b1

def store_cluster_slices(model_teacher, k_means, version):
    epoch_array = np.arange(79)
    all_labels  = []
    step_size   = 10 
    count       = 0
    
    for epoch in epoch_array:
        temp_index               = epoch%(int(len(unlabelled_ids)/step_size))
        trainx, trainx_lungmask  = read_training_data_unlabelled(unlabelled_ids[temp_index*step_size:temp_index*step_size+step_size])
        trainy                   = get_prediction(model_teacher, trainx)
        
        #trainy = np.load('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/PREDICTION-NUMPY/'+str(epoch)+'.npy')
        trainy = np.reshape(trainy, [trainy.shape[0], 512*512])
        #print(epoch, trainy.shape, trainx.shape)
        
        l1     = k_means.predict(trainy)
        
        for jt, t in enumerate(l1):
            temp  = np.reshape(trainy[jt], [512, 512]).astype('float16')
            np.save('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/CLUSTER-NUMPY-'+str(version)+'/'+str(t)+'-'+str(count)+'-y.npy', temp)
            
            temp  = np.reshape(trainx[jt], [512, 512]).astype('float16')
            np.save('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/CLUSTER-NUMPY-'+str(version)+'/'+str(t)+'-'+str(count)+'-x.npy', temp)
            
            count = count+1
    
    return

def prepare_hash(version):
    all_cluster_files = glob.glob('/media/pranjal/BackupPlus/SIEMENS/SIEMENS/CLUSTER-NUMPY-'+str(version)+'/*.npy')
    print('Version ', version, 'File name counts ', len(all_cluster_files))
    filename_hash = {}
    for i in range(50):
        filename_hash[i] = []

    for t in all_cluster_files:
        filename_hash[int(t.split('/')[-1].split('-')[0])].append(t)
    
    return filename_hash

def get_all_covid_lesions(valx, valy, lesion_size):
    lesion_shapes_x = []
    lesion_shapes_y = []
    
    for i in range(valy.shape[0]):
        tx           = valx[i, 0]
        blobs        = valy[i, 0]
        blobs_labels = skimage.measure.label(blobs, background=0)
        propsa       = skimage.measure.regionprops(blobs_labels)
        
        for k in range(len(propsa)):
            temp = (blobs_labels == propsa[k].label).astype('uint8')
            
            temp_size = np.count_nonzero(temp.flatten().astype('uint8'))
            if temp_size < lesion_size and temp_size > 5:
                slice_x, slice_y = ndimage.find_objects(temp == 1)[0]
                
                roi_y = 1-temp[slice_x, slice_y]
                roi_x = tx[slice_x, slice_y]*temp[slice_x, slice_y]
                
                lesion_shapes_x.append(roi_x)
                lesion_shapes_y.append(roi_y)
                
                lesion_shapes_x.append(roi_x.T)
                lesion_shapes_y.append(roi_y.T)
                
                lesion_shapes_x.append(np.rot90(roi_x, 180))
                lesion_shapes_y.append(np.rot90(roi_y, 180))
    
    return lesion_shapes_x, lesion_shapes_y

def get_augmented_slice(batch_size, read_ids, lesion_shapes_x, lesion_shapes_y):
    x_array          = []
    x_array_lungmask = []
    
    index   = random.randint(0, len(read_ids)-1)
    #print(read_ids[index])
    
    p       = read_ids[index].split('_')[0]
    types   = 'CT-1'#read_ids[index].split('_')[1]
    count   = 0
    
    name     = basepath+'studies/'+types+'/'
    maskname = name+'study_'+p+'_mask.nii.gz'
    volname  = name+'study_'+p+'.nii.gz'
    
    segmentation_mask = basepath+'masks/'
    segmentation_mask = segmentation_mask+'study_'+p+'_mask.nii.gz'
    
    mask     = sitk.GetArrayFromImage(sitk.ReadImage(maskname))
    vol      = (sitk.GetArrayFromImage(sitk.ReadImage(volname))+1024.0)/1024.0
    segmentation_mask = sitk.GetArrayFromImage(sitk.ReadImage(segmentation_mask))
    
    mask[mask > 0] = 1
    count          = 0
    
    while(count < batch_size):
        t     = np.random.randint(0, mask.shape[0]-1)
        temp  = np.count_nonzero(mask[t].flatten())
        
        # Check if lung region is present
        if temp > 0:
            st  = vol[t]
            i,j = np.nonzero(mask[t])
            
            index = random.randint(0, len(i)-1)
            
            i = i[index]
            j = j[index]
            
            lesion_index = random.randint(0, len(lesion_shapes_x)-1)
            
            lesion_x     = lesion_shapes_x[lesion_index]
            lesion_y     = lesion_shapes_y[lesion_index]
            
            sx     = int(lesion_x.shape[0]/2)
            sy     = int(lesion_x.shape[1]/2)
            
            if st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy].shape == lesion_x.shape:
                st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  =  lesion_y*st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]
                st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  =  lesion_x + st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]

                m1 = segmentation_mask[t]#np.zeros(st.shape)
                m1[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  += 1-lesion_y
                m1         = m1*mask[t]
                m1[m1 > 0] = 1

                x_array.append(np.expand_dims(st,          axis=0))
                x_array_lungmask.append(np.expand_dims(m1, axis=0))

                count = count+1

    x_array          = np.array(x_array)
    x_array_lungmask = np.array(x_array_lungmask)
    
    return x_array, x_array_lungmask

def get_multiple_augmented_slice(batch_size, read_ids, lesion_shapes_x, lesion_shapes_y):
    x_array          = []
    x_array_lungmask = []
    
    index   = random.randint(0, len(read_ids)-1)
    #print(read_ids[index])
    
    p       = read_ids[index].split('_')[0]
    types   = 'CT-1'#read_ids[index].split('_')[1]
    count   = 0
    
    name     = basepath+'studies/'+types+'/'
    maskname = name+'study_'+p+'_mask.nii.gz'
    volname  = name+'study_'+p+'.nii.gz'
    
    segmentation_mask = basepath+'masks/'
    segmentation_mask = segmentation_mask+'study_'+p+'_mask.nii.gz'
    
    mask     = sitk.GetArrayFromImage(sitk.ReadImage(maskname))
    vol      = (sitk.GetArrayFromImage(sitk.ReadImage(volname))+1024.0)/1024.0
    segmentation_mask = sitk.GetArrayFromImage(sitk.ReadImage(segmentation_mask))
    
    mask[mask > 0] = 1
    count          = 0
    
    while(count < batch_size):
        t     = np.random.randint(0, mask.shape[0]-1)
        temp  = np.count_nonzero(mask[t].flatten())
        
        # Check if lung region is present
        if temp > 0:
            st  = vol[t]
            #segmen
            ipl, jpl = np.nonzero(mask[t])
            
            lesion_count = random.randint(0, 5)
            temp_count   = 0
            
            while(temp_count < lesion_count):
                index = random.randint(0, len(ipl)-1)

                i = ipl[index]
                j = jpl[index]

                lesion_index = random.randint(0, len(lesion_shapes_x)-1)

                lesion_x     = lesion_shapes_x[lesion_index]
                lesion_y     = lesion_shapes_y[lesion_index]

                sx     = int(lesion_x.shape[0]/2)
                sy     = int(lesion_x.shape[1]/2)

                if st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy].shape == lesion_x.shape:
                    st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  =  lesion_y*st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]
                    st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  =  lesion_x + st[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]

                    m1 = segmentation_mask[t]#np.zeros(st.shape)
                    m1[i-sx:i+lesion_x.shape[0]-sx, j-sy:j+lesion_x.shape[1]-sy]  += 1-lesion_y
                    m1         = m1*mask[t]
                    m1[m1 > 0] = 1
                    segmentation_mask[t] = m1
                    temp_count           = temp_count + 1
            
            x_array.append(np.expand_dims(st,          axis=0))
            x_array_lungmask.append(np.expand_dims(m1, axis=0))
            
            count = count+1

    x_array          = np.array(x_array)
    x_array_lungmask = np.array(x_array_lungmask)
    
    return x_array, x_array_lungmask

def plot_figure_slope(model_save_name):
    N = 2
    a = val_dice_array1#np.convolve(val_dice_array1, np.ones((N,))/N, mode='valid')
    b = train_dice_array1#np.convolve(train_dice_array1, np.ones((N,))/N, mode='valid')
    c = test_dice_array1#np.convolve(test_dice_array1, np.ones((N,))/N, mode='valid')
    
    temp  = 0
    slope = 0
    #np.abs(np.abs(b[i]-b[i-1])-np.abs(a[i]-a[i-1])) < 0.1 and
    for i in range(1, len(a)):
        if b[i] >= b[i-1] and a[i] >= a[i-1]:
            temp  = i#np.argmax(a)
            slope = b[i]-b[i-1]-(a[i]-a[i-1])
            #print(i, slope, np.abs(b[i]-b[i-1]), np.abs(a[i]-a[i-1]), b[i], b[i-1])
    
    import matplotlib.pyplot as plt
    plt.plot(a)
    plt.plot(b)
    plt.plot(c)
    plt.ylabel('some numbers')
    plt.annotate('Index '+str(temp), xy=(0.75, 0.25), xycoords='axes fraction')
    plt.annotate('Train '+str(round(b[temp], 3)), xy=(0.75, 0.20), xycoords='axes fraction')
    plt.annotate('Val   '+str(round(a[temp], 3)), xy=(0.75, 0.15), xycoords='axes fraction')
    plt.annotate('Test  '+str(round(c[temp], 3)), xy=(0.75, 0.10), xycoords='axes fraction')
    plt.annotate('Slope '+str(round(slope, 3)),   xy=(0.75, 0.05), xycoords='axes fraction')
    #plt.text(6, 0, )
    #plt.text(6, 0.1, 'Val   '+str(round(a[temp], 3)))
    #plt.text(6, 0.2, 'Train '+str(round(b[temp], 3)))
    #plt.text(6, 0.3, 'Test  '+str(round(c[temp], 3)))
    
    plt.savefig(model_save_name+".png")
    
    plt.close()
    plt.clf()
    
    return

def sort_data(trainx1, trainy1):
    # Sort the data
    X = trainx1
    Y = trainy1
    r = [t for t in sorted(zip(Y,X), key=lambda pair: np.sum(pair[0].flatten()))]
    
    trainx = []
    trainy = []
    
    for i in range(len(X)):
        trainy.append(r[i][0])
        trainx.append(r[i][1])
    
    trainx = np.array(trainx)
    trainy = np.array(trainy)
    
    return trainx, trainy

def plot_figure(model_save_name):
    a = list(val_dice_array)#np.convolve(val_dice_array1, np.ones((N,))/N, mode='valid')
    b = list(train_dice_array)#np.convolve(train_dice_array1, np.ones((N,))/N, mode='valid')
    c = list(test_dice_array)#np.convolve(test_dice_array1, np.ones((N,))/N, mode='valid')
    
    #a.insert(0, 0)
    #b.insert(0, 0)
    #c.insert(0, 0)
#     temp  = 0
#     slope = 0
#     #np.abs(np.abs(b[i]-b[i-1])-np.abs(a[i]-a[i-1])) < 0.1 and
#     for i in range(1, len(a)):
#         if b[i] >= b[i-1] and a[i] >= a[i-1]:
#             temp  = i#np.argmax(a)
#             slope = b[i]-b[i-1]-(a[i]-a[i-1])
#             #print(i, slope, np.abs(b[i]-b[i-1]), np.abs(a[i]-a[i-1]), b[i], b[i-1])
    
    # Take arg max for semi model
    temp = np.argmax(a)
    
    import matplotlib.pyplot as plt
    plt.plot(a)
    plt.plot(b)
    plt.plot(c)
    plt.ylabel('some numbers')
    plt.annotate('Index '+str(temp), xy=(0.75, 0.25), xycoords='axes fraction')
    plt.annotate('Train '+str(round(b[temp], 3)), xy=(0.75, 0.20), xycoords='axes fraction')
    plt.annotate('Val   '+str(round(a[temp], 3)), xy=(0.75, 0.15), xycoords='axes fraction')
    plt.annotate('Test  '+str(round(c[temp], 3)), xy=(0.75, 0.10), xycoords='axes fraction')
    #plt.annotate('Slope '+str(round(slope, 3)),   xy=(0.75, 0.05), xycoords='axes fraction')
    #plt.text(6, 0, )
    #plt.text(6, 0.1, 'Val   '+str(round(a[temp], 3)))
    #plt.text(6, 0.2, 'Train '+str(round(b[temp], 3)))
    #plt.text(6, 0.3, 'Test  '+str(round(c[temp], 3)))
    
    plt.savefig(model_save_name+".png")
    
    plt.close()
    plt.clf()
    
    return

def train_model1(model, batch_size, optimizer, criterion, trainx, trainy, augment=False):
    loss_array = []
   
    model.train()
    #print(len(trainx)//batch_size)
   
    for i in range(len(trainx)//batch_size):
        x = trainx[i*batch_size:(i+1)*batch_size, 0, :, :]
        y = trainy[i*batch_size:(i+1)*batch_size, 0, :, :]
                
        if augment:
            for k in range(x.shape[0]):
                rotv = random.randint(0, 3)
                x[k, 0, :, :] = np.rot90(x[k, 0, :, :], rotv)
                y[k, 0, :, :] = np.rot90(y[k, 0, :, :], rotv)
       
        #x2 = model.forward(x)        
        #print(x2.shape)
        
#         lstm = nn.LSTM(512*512,512*512,batchfirst=True)
#         hidden = (torch.randn(1, 512, 512), torch.randn(1, 512, 512))
#         outlstm = lstm(x, hidden)
#         n = np.asarray(outlstm)
  
        print(i, x.shape[0])
        
        if(x.shape[0]!= 4):
            break
    
        x = np.expand_dims(x, 1)
        y = np.expand_dims(y, 1)
        y = torch.tensor(y, device=device).float()
        x = torch.tensor(x, device=device).float()
        
       
        optimizer.zero_grad()
        optimizer_attn_w.zero_grad()
        
        output = model.forward(x)
        #print(i,attn_weights[1])
        
        loss   = criterion(output , y)
        loss.backward()
       
        loss_array.append(loss.item())
        
       # torch.nn.utils.clip_grad_norm(attn_decoder1.parameters(),0.7)
        
        optimizer.step()
        optimizer_attn_w.step()
   
    loss_array = np.mean(loss_array)
    return loss_array

def train_model2(model, batch_size, optimizer, criterion, trainx, trainy, augment=False):
    #batch_size = 4
    loss_array = []
   
    model.train()
    #print(len(trainx)//batch_size)
   
    for i in range(len(trainx)//batch_size):
        x = trainx[i*batch_size:(i+1)*batch_size, 0, :, :]
        y = trainy[i*batch_size:(i+1)*batch_size, 0, :, :]
        
                
        if augment:
            for k in range(x.shape[0]):
                rotv = random.randint(0, 3)
                x[k, 0, :, :] = np.rot90(x[k, 0, :, :], rotv)
                y[k, 0, :, :] = np.rot90(y[k, 0, :, :], rotv)
       
  
        if(x.shape[0]!=4):
            break
            
    
        x = np.expand_dims(x, 1)
        

        y = np.expand_dims(y, 1)

        y = torch.tensor(y, device=device).float()
        
        x = torch.tensor(x, device=device).float()
        
       
        optimizer.zero_grad()
        optimizer_attn_w.zero_grad()
        
        output = model.forward(x)
        #print(i,attn_weights[1])
        
        loss   = criterion(output , y)
        loss.backward()
       
        loss_array.append(loss.item())
        
       # torch.nn.utils.clip_grad_norm(attn_decoder1.parameters(),0.7)
        
        optimizer.step()
        optimizer_attn_w.step()
   
    loss_array = np.mean(loss_array)
    return loss_array

device         = torch.device("cuda:0")


# train_ids      = np.load(basepath+'TRAIN.npy')
# val_ids        = np.load(basepath+'VALIDATION.npy')
# test_ids       = np.load(basepath+'TEST.npy')
# unlabelled_ids = np.load(basepath+'NOTLABELLED.npy')
# nocovid_ids    = np.load(basepath+'NOCOVID.npy')


# unlabelled_ids     = unlabelled_ids
# train_ids          = train_ids[:4]
# val_ids            = val_ids
# test_ids           = test_ids

trainx_l = np.load(basepath+'train_x.npy')/255.0
trainy_l = np.load(basepath+'train_y.npy')
trainy_l[trainy_l > 0] = 1

train_size    = 40
valx = trainx_l[train_size:]
valy = trainy_l[train_size:]

trainx_l = trainx_l[:train_size]
trainy_l = trainy_l[:train_size]

testx = np.load(basepath+'test_x.npy')/255.0
testy = np.load(basepath+'test_y.npy')
testy[testy > 0] = 1


trainx_l1 = np.zeros([trainx_l.shape[0], 1, 512, 512], dtype='float16')
valx1     = np.zeros([valx.shape[0], 1, 512, 512],     dtype='float16')
testx1    = np.zeros([testx.shape[0], 1, 512, 512],    dtype='float16')

trainy_l1 = np.zeros([trainy_l.shape[0], 1, 512, 512], dtype='float16')
valy1     = np.zeros([valy.shape[0], 1, 512, 512],     dtype='float16')
testy1    = np.zeros([testy.shape[0], 1, 512, 512],    dtype='float16')


for i in range(trainx_l.shape[0]):
    trainx_l1[i, 0] = scipy.ndimage.zoom(trainx_l[i], 2, order=3)
    trainy_l1[i, 0] = scipy.ndimage.zoom(trainy_l[i], 2, order=0)

for i in range(valx.shape[0]):
    valx1[i, 0] = scipy.ndimage.zoom(valx[i], 2, order=3)
    valy1[i, 0] = scipy.ndimage.zoom(valy[i], 2, order=0)

for i in range(testx.shape[0]):
    testx1[i, 0] = scipy.ndimage.zoom(testx[i], 2, order=3)
    testy1[i, 0] = scipy.ndimage.zoom(testy[i], 2, order=0)


trainx_l = trainx_l1
trainy_l = trainy_l1
valx = valx1
valy = valy1
testx = testx1
testy = testy1

print(trainx_l.shape, valx.shape, testx.shape)

def init_normal(m):
    if type(m) == nn.Conv2d:
        nn.init.kaiming_normal_(m.weight)
    if type(m) == nn.Linear:
        #nn.init.kaiming_normal_(m.weight)
        nn.init.xavier_uniform_(m.weight)

prev_max        = -1000
model_student   = UNetDoubleSmallWithoutGN(1, 1)
model_student.cuda()

optimizer_student  = optim.Adam(model_student.parameters(), lr=0.0001)
criterion          = nn.BCEWithLogitsLoss(torch.ones([1]).cuda())

val_dice_array   = []
train_dice_array = []
test_dice_array  = []

trainx, trainy   = sort_data(trainx_l, trainy_l)
total_epochs = 1000


# trainx = np.expand_dims(trainx, axis=1)
# trainy = np.expand_dims(trainy, axis=1)

# valx   = np.expand_dims(valx, axis=1)
# valy   = np.expand_dims(valy, axis=1)

# testx  = np.expand_dims(testx, axis=1)
# testy  = np.expand_dims(testy, axis=1)

teacher_dice_array = []
test_dice_array    = []


for epoch in range(total_epochs):
    if epoch%10 ==1:
        print(epoch)
    #train_model1(model, optimizer, criterion, trainx, trainy, augment=False)
    train_loss    = train_model(model_student, 4, optimizer_student, criterion, trainx, trainy, False)
    #train_loss    = train_model(model_student, 2, optimizer_student, criterion, trainx, trainy, False)
    
    val_dice      = evaluate_result(model_student, valx,   valy)
    student_dice1 = evaluate_result(model_student, trainx, trainy)
    student_dice2 = evaluate_result(model_student, testx,  testy)
    
    
    train_dice_array.append(np.mean(student_dice1))
    val_dice_array.append(np.mean(val_dice))
    test_dice_array.append(np.mean(student_dice2))

    model_save_name = "tmi-compare-unet-covid19"
    
    #if np.mean(val_dice) > prev_max:
    print("Step %d  Val Dice %.5f, Train Dice %f, Test Dice %f" % (epoch, np.mean(val_dice), np.mean(student_dice1), np.mean(student_dice2)))
    prev_max     = np.mean(val_dice)
    torch.save(model_student.state_dict(), basepath_models+model_save_name+'-'+str(epoch)+".pt")

    #np.save(model_save_name+'_train.npy',      train_dice_array)
    #np.save(model_save_name+'_validation.npy', val_dice_array)

In [ ]:
# [STAR] For training the Group Norm using un-labelled data


import torch.optim as optim
from tqdm import tqdm
import random
from skimage.transform import rotate, AffineTransform, warp
from scipy.stats import entropy
import numpy as np

import torch.optim as optim
from tqdm import tqdm
import random
from skimage.transform import rotate, AffineTransform, warp
from scipy.stats import entropy


basepath         = '/media/pranjal/2d33dff3-95f7-4dc0-9842-a9b18bcf1bf9/pranjal/COVID_MOSCOW/COVID_MOSCOW/COVID19_1110/'
basepath_models  = '/media/pranjal/2d33dff3-95f7-4dc0-9842-a9b18bcf1bf9/pranjal/COVID_MOSCOW/COVID_MOSCOW/COVID19_1110/models/single_models/'


def read_training_data(read_ids):
    x_array = []
    y_array = []
    
    for p in read_ids:
        name = basepath+'masks/'
        name = name+'study_'+p+'_mask.nii.gz'
        
        mask = sitk.GetArrayFromImage(sitk.ReadImage(name))
        vol  = sitk.GetArrayFromImage(sitk.ReadImage(name.replace('_mask.nii.gz', '.nii.gz').replace('masks', 'studies/CT-1')))
        
        for t in range(mask.shape[0]):
            temp  = np.count_nonzero(mask[t].flatten())
            if temp > 0:
                x_array.append(np.expand_dims(vol[t], axis=0))
                y_array.append(np.expand_dims(mask[t], axis=0))

    x_array = (np.array(x_array)+1024.0)/1024.0
    y_array = np.array(y_array)
    
    return x_array, y_array

def dice(im1, im2):
    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)
    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)
    return 2. * intersection.sum() / (im1.sum() + im2.sum()+0.00001)

def read_training_data_unlabelled(read_ids):
    x_array          = []
    x_array_lungmask = []
    
    names   = [x.split('_')[0] for x in read_ids]
    types   = [x.split('_')[1] for x in read_ids]
    count   = 0
    
    for p in names:
        name     = basepath+'studies/'+types[count]+'/'
        maskname = name+'study_'+p+'_mask.nii.gz'
        volname  = name+'study_'+p+'.nii.gz'
        
        mask = sitk.GetArrayFromImage(sitk.ReadImage(maskname))
        vol  = sitk.GetArrayFromImage(sitk.ReadImage(volname))
        mask[mask > 0] = 1
        
        for t in range(mask.shape[0]):
            if True:#t % 1 == 0:
                temp  = np.count_nonzero(mask[t].flatten())
                if temp > 0: # Check if lung region is present
                    x_array.append(np.expand_dims(vol[t], axis=0))
                    x_array_lungmask.append(np.expand_dims(mask[t], axis=0))
        
        count = count+1

    x_array          = (np.array(x_array)+1024.0)/1024.0
    x_array_lungmask = np.array(x_array_lungmask)
    
    return x_array, x_array_lungmask

def get_prediction(model, valx):
    output_array   = []
    batch_size     = 1
    
    model.eval()
    
    for ik in range(len(valx)//batch_size):
        x = valx[ik*batch_size:(ik+1)*batch_size, :, :, :]
        x = torch.tensor(x, device=device).float()

        output = model.forward(x)
        output = torch.sigmoid(output)
        output = output.data.cpu().numpy()
        
        for k in range(output.shape[0]):
            output_array.append(output[k, 0])
    
    output_array = np.array(output_array).astype('float16')
    output_array = np.expand_dims(output_array, 1)
    
    return output_array

def evaluate_result(model, valx, valy):
    model.eval()
    
    val_dice       = []
    batch_size     = 1
    for ik in range(len(valx)//batch_size):
        x = valx[ik*batch_size:(ik+1)*batch_size, :, :, :]
        y = valy[ik*batch_size:(ik+1)*batch_size, :, :, :]

        x = torch.tensor(x, device=device).float()

        output = model.forward(x)

        output = torch.sigmoid(output)        
        output = output.data.cpu().numpy()

        output[output < 0.5] = 0
        output[output > 0.5] = 1
        
        for pk in range(output.shape[0]):
            dt = dice(y[pk, 0, :, :], output[pk, 0, :, :])
            val_dice.append(dt)
    return val_dice

def train_model(model, optimizer, criterion, trainx, trainy):
    batch_size = 2
    loss_array = []
    
    model.train()
    
    for i in range(len(trainx)//batch_size):
        x = trainx[i*batch_size:(i+1)*batch_size, :, :, :]
        y = trainy[i*batch_size:(i+1)*batch_size, :, :, :]
        
        x = torch.tensor(x, device=device).float()
        y = torch.tensor(y, device=device).float()
        
        optimizer.zero_grad()
        output = model.forward(x)        
        loss   = criterion(output , y)
        loss.backward()
        
        loss_array.append(loss.item())
        optimizer.step()
    
    loss_array = np.mean(loss_array)
    return loss_array

def sort_data(trainx1, trainy1):
    # Sort the data
    X = trainx1
    Y = trainy1
    r = [t for t in sorted(zip(Y,X), key=lambda pair: np.sum(pair[0].flatten()))]
    
    trainx = []
    trainy = []
    
    for i in range(len(X)):
        trainy.append(r[i][0])
        trainx.append(r[i][1])
    
    trainx = np.array(trainx)
    trainy = np.array(trainy)
    
    return trainx, trainy
        
train_ids      = np.load(basepath+'TRAIN.npy')
val_ids        = np.load(basepath+'VALIDATION.npy')
test_ids       = np.load(basepath+'TEST.npy')
unlabelled_ids = np.load(basepath+'NOTLABELLED.npy')

train_ids           = train_ids

trainx_l, trainy_l = read_training_data(train_ids)
valx, valy         = read_training_data(val_ids)
testx, testy       = read_training_data(test_ids)

print(trainx_l.shape, valx.shape, testx.shape)

valx_img = sitk.GetImageFromArray(valx.astype('float32')[:, 0, :, :])
sitk.WriteImage(valx_img, basepath+'CT-img.nii.gz')


model_teacher = SUNet(1,1)#UNetDoubleSmall(1,1)
model_teacher.cuda()


p1         = torch.load(basepath_models+"tmi-f-3-93.pt")
model_teacher.load_state_dict(p1)


device             = torch.device("cuda:0")
optimizer_student  = optim.Adam(model_student.parameters(), lr=0.0005)
criterion          = nn.BCEWithLogitsLoss(torch.ones([1]).cuda())

val_loss_array   = []
train_loss_array = []

prev_max_teacher = -1000
prev_max    = -1000
model_count = 0
step_size   = 20
beta        = 0.9

val_dice_t   = evaluate_result(model_teacher, valx, valy)
print("Dice in the beginning ", np.mean(val_dice_t))

val_dice_t = np.mean(val_dice_t)
prev_max   = val_dice_t

teacher_dice_array = []

train_dice_array = []
val_dice_array   = []
test_dice_array  = []

model_save_name = "tmi-f-semi-sunet"

first_time = True

for epoch in range(300):
    temp_index                 = epoch%(int(len(unlabelled_ids)/step_size))
    trainx1, trainx1_lungmask  = read_training_data_unlabelled(unlabelled_ids[temp_index*step_size:temp_index*step_size+step_size])
    trainy1                    = get_prediction(model_teacher, trainx1)
    
    trainx, trainy = sort_data(trainx1, trainy1)
    if epoch%10 == 0:
        print(epoch, trainx.shape, trainy.shape)
    
    train_loss = train_model(model_student, optimizer_student, criterion, trainx, trainy)
    
    train_dice = evaluate_result(model_student, trainx_l, trainy_l)
    val_dice   = evaluate_result(model_student, valx, valy)
    
    # Update teacher weights
    if np.mean(val_dice) > val_dice_t:
        print(epoch, ' Updating Teacher Weights')
        torch.save(model_student.state_dict(), "temp.pt")
        torch.save(model_student.state_dict(), basepath_models+model_save_name+".pt")
        p1         = torch.load('temp.pt')
        
        model_teacher.load_state_dict(p1)
        val_dice_t = np.mean(val_dice)
    
    train_dice_array.append(np.mean(train_dice))
    val_dice_array.append(np.mean(val_dice))
    
    print("Step %d  Train Dice %.5f  Val Dice %.5f " % (epoch, np.mean(train_dice), np.mean(val_dice)))


In [ ]:
# [STAR] For selecting the most reliable weights

s  = 300
a1 = np.zeros([s, 236])
b1 = np.zeros([s, 126])

count = 0

x = []
y = []
z = []

model_save_name = 'tmi-f-single-semi'

def scoring_function(val_array, test_array, epoch):
    score_array = []
    step_size   = 5
    alpha       = 10
    
    temp_array     = val_array[epoch-step_size:epoch]
    sum_array      = 1-np.mean(temp_array, axis=0)
    var_array      = np.std(temp_array, axis=0)

    score_temp     = sum_array + alpha*var_array
    final_score    = score_temp*val_array[epoch]
    
    return final_score

for epoch in range(s):
    a = np.load('/home/yu-hao/AttentionDeepMIL/val_dice_array-'+model_save_name+'--'+str(epoch)+'.npy')
    a1[epoch] = a

score_array = []
for epoch in range(100, 300):
    final_score = scoring_function(a1, b1, epoch)
    score_array.append(np.mean(final_score))

index = np.argmax(score_array) 
print('Most reliable Weights Index ', index, score_array[index])
